Load modules

In [1]:
# Make sure modules are reimported each time to update changes
%reload_ext autoreload
%autoreload 2

# System packages
import os
from shutil import copyfile

import csv
import itertools
from difflib import get_close_matches
import warnings
warnings.simplefilter('error', UserWarning)

# Numerical packages
import numpy as np
import pandas as pd
from scipy import odr
from scipy.special import erfc
from scipy.interpolate import UnivariateSpline

# Plotting packages
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib import transforms
from matplotlib.offsetbox import AnchoredText
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredAuxTransformBox
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter, AutoMinorLocator)
from PIL import Image

# Set up dataframe display options and other formatting options
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
import pprint


# Local packages
from BoreFlow import general_python_functions
from BoreFlow import load_and_pass_data
from BoreFlow import process_climate_histories
from BoreFlow import monte_carlo_sampling
from BoreFlow import climatic_correction_functions

from BoreFlow import ben_mather_heat_flow_functions
from BoreFlow import heat_flow_functions

from BoreFlow import plotting_functions

Set up filenames and variables and import data

Filenames:
pc_ = palaeoclimate
rc_ = recent climate

Climate:
cc = climatic_correction
pcc = palaeoclimatic_correction 
rcc = recent_climatic_correction

Conductivities:
r87 = Rollin (1987) rollin1987
c82 = Cermak and Rybach (1982) cermak1982

mc = Monte Carlo monte_carlo

k = conductivity conds
T = temperature temps
z = depth
t = seconds seconds

years replace with y
seconds get rid of


cst = constant
replace const with cst

replace ukogl_well_tops_depth_error_m with ukogl_well_tops_z_error_m

Units:
z = metres
T = Kelvin or degrees Celsius
k = 
t = seconds (unless years are specified)

replace borehole with bh?

replace subsampled with ss

replace monte_carlo with mc

replace bgs_borehole_scan with bgs_bhscan
replace 



In [2]:
### SET UP FILENAMES ###

### Set up palaeoclimate datasets
# palaeoclimate data from Thomas Farrell's compilation
pc_csv = "./example_data/climate_data/palaeoclimate_data/palaeoclimate_surface_temperatures.csv"

# Warming data for the past 100+ years from NASA
rc_filename = "ZonAnn.Ts+dSST"
rc_base_folder = "./example_data/climate_data/recent_climate_data"
rc_calc_folder = rc_base_folder + "/calculated_data"
rc_hist_csv = rc_base_folder + "/raw_data/" + rc_filename + ".csv"
rc_smoother = "boxcar"
rc_smoothing_length = 9
rc_smoothed_filename = rc_filename + "_smoothedboxcar" + str(rc_smoothing_length)
rc_smoothed_outfile = rc_calc_folder + "/" + rc_smoothed_filename
# TODO Could get more local readings from Met Office?

### Compilations of average thermal conductivity and lithology/stratigrahy
british_k_comp_infile = "./example_data/thermal_conductivity_compilations_data/thermal_conductivity_tabulations_british"
british_k_comp_df = pd.read_excel(british_k_comp_infile + ".xlsx", comment="%!%#", mangle_dupe_cols=True, usecols=["commented","source_name","source_code","bgs_group_name","bgs_group_code","formation_name","bgs_formation_code","member_name","lith_type","bgs_lith_code","source_description","description_code","number_samples","median_k(Wm-1K-1)","mean_k(Wm-1K-1)","stdev_k(Wm-1K-1)","sterr_k(Wm-1K-1)","range_k(Wm-1K-1)","min_k(Wm-1K-1)","max_k(Wm-1K-1)","use?"])
# Extract conductivities from Rollin (1987) and turn into dictionary
r87_k_dict = british_k_comp_df[(british_k_comp_df['source_code'] == 'rollin1987') & (british_k_comp_df['use?'] == 'y')].dropna(subset=['description_code']).set_index('description_code').to_dict(orient='index')

global_k_comp_infile = "./example_data/thermal_conductivity_compilations_data/thermal_conductivity_tabulations_global"
global_k_comp_df = pd.read_excel(global_k_comp_infile + ".xlsx", comment="%!%#", mangle_dupe_cols=True, usecols=["commented","source_name","source_code","lith_type","source_description","number_samples","median_k(Wm-1K-1)","mean_k(Wm-1K-1)","stdev_k(Wm-1K-1)","sterr_k(Wm-1K-1)","range_k(Wm-1K-1)","min_k(Wm-1K-1)","max_k(Wm-1K-1)","use?"]).drop(columns=['commented'])
# Extract conductivities from Cermak and Rybach (1982) and turn into dictionary
c82_k_dict = global_k_comp_df[(global_k_comp_df['source_code'] == 'cermak1982') & (global_k_comp_df['use?'] == 'y')].dropna(subset=['lith_type']).set_index('lith_type').to_dict(orient='index')

# Load lookup dictionaries to translate literature names for lithology into standardised terms
liths_lookup_dict_infile = "./example_data/thermal_conductivity_compilations_data/thermal_conductivity_lookup_dictionary"
liths_lookup_dict_df = pd.read_excel(liths_lookup_dict_infile + ".xlsx", comment="%!%#", mangle_dupe_cols=True)

### Load useful dictionaries containing data and plotting preferences
# borehole_metadata_dict = background_dictionaries.import_borehole_metadata_dict()
# comp_k_dict = background_dictionaries.import_comp_k_dict()
# r87_lith_plot_format_dict = background_dictionaries.import_r87_lith_type_plot_format_dict()
# c82_lith_plot_format_dict = background_dictionaries.import_c82_lith_type_plot_format_dict()


r87_unit_plot_dict_infile = "./example_data/thermal_conductivity_compilations_data/rollin1987_unit_plot_dict"
r87_unit_plot_dict_df = pd.read_excel(r87_unit_plot_dict_infile + ".xlsx", comment="%!%#", mangle_dupe_cols=True)
# Extract plotting options for Rollin (1987) and turn into dictionary
r87_unit_plot_dict = r87_unit_plot_dict_df[(r87_unit_plot_dict_df['dictionary_name'] == 'rollin1987_unit_plot_dict')].drop(columns=['dictionary_name']).set_index('rollin1987_unit').to_dict(orient='index')

c82_unit_plot_dict_infile = "./example_data/thermal_conductivity_compilations_data/unit_lith_cermak1982_plot_dict"
c82_unit_plot_dict_df = pd.read_excel(c82_unit_plot_dict_infile + ".xlsx", comment="%!%#", mangle_dupe_cols=True)
# Extract plotting options for Cermak and Rybach (1982) and turn into dictionary
c82_unit_plot_dict = c82_unit_plot_dict_df[(c82_unit_plot_dict_df['dictionary_name'] == 'unit_lith_cermak1982_plot_dict')].drop(columns=['dictionary_name']).set_index('cermak1982_lith_type').to_dict(orient='index')


### Load surface temperature histories
# TODO Temp: specify uncertainty in times for palaeoclimate. TODO Add to file. Also have distribution type read from file
pc_sigma_t0y_cst = 1e3
pc_sigma_t1y_cst = 1e3
pc_sigma_t0_cst = general_python_functions.y2s(pc_sigma_t0y_cst)
pc_sigma_t1_cst = general_python_functions.y2s(pc_sigma_t1y_cst)
pc_t0, pc_t1, pc_deltaTs, pc_sigma_deltaTs, rc_ty, rc_deltaTs, rc_ty_smoothing_cutoff, rc_deltaTs_smoothed, rc_smoothed_df, pc_suffix, rc_suffix = load_and_pass_data.load_surface_temperature_histories(pc_csv, rc_hist_csv, rc_smoother, rc_smoothing_length, rc_smoothed_outfile)
pc_sigma_t0 = np.full(np.size(pc_t0), pc_sigma_t0_cst)
pc_sigma_t1 = np.full(np.size(pc_t1), pc_sigma_t1_cst)
pc_sigma_t_dist = 'normal'
pc_sigma_deltaTs_dist = 'normal'
## TODO What are uncertainties for recent climate history? What effect does smoothing have on uncertainty?
rc_sigma_ty_cst = 0.5
rc_sigma_ty_smoothing_cutoff_cst = rc_sigma_ty_cst
rc_sigma_deltaTs_cst = 0.1
rc_sigma_deltaTs_smoothed_cst = 0.1
rc_sigma_ty = np.full(np.size(rc_ty), rc_sigma_ty_cst)
rc_sigma_ty_smoothing_cutoff = np.full(np.size(rc_ty_smoothing_cutoff), rc_sigma_ty_smoothing_cutoff_cst)
rc_sigma_deltaTs = np.full(np.size(rc_deltaTs), rc_sigma_deltaTs_cst)
rc_sigma_deltaTs_smoothed = np.full(np.size(rc_deltaTs_smoothed), rc_sigma_deltaTs_smoothed_cst)
### TODO rc_sigma_ty_dist should be uniform
rc_sigma_ty_dist = 'normal'
rc_sigma_ty_smoothing_cutoff_dist = 'normal'
rc_sigma_deltaTs_dist = 'normal'
rc_sigma_deltaTs_smoothed_dist = 'normal'




### Set up paths for borehole data
boreholes_dir = "./example_data/borehole_data/standardised_borehole_records"

# Load input borehole summary - csv file generated from Excel spreadsheet
all_boreholes_spreadsheet_dir = str(boreholes_dir) + "/boreholes_overview_spreadsheet"
all_boreholes_overview_file = str(all_boreholes_spreadsheet_dir) + "/boreholes_overview_spreadsheet.xlsx"
all_borehole_overview_df = pd.read_excel(all_boreholes_overview_file, comment="%!%#", mangle_dupe_cols=True, usecols=["name","file_name","os_grid_code","ukng_easting","ukng_northing","temperature?","type_temp","number_uncorrected_temp_measurements","max_depth_temp(m)","conductivity?","max_depth_conductivities(m)","number_conductivity_measurements","max_depth_conductivities(m)","year_drilling_completed","year_temp_measurements","number_strat_interps","strat_interp1","strat_interp1_ext","strat_interp2","strat_interp2_ext","strat_interp3","strat_interp3_ext","strat_interp4","strat_interp4_ext","strat_interp5","strat_interp5_ext"])

individual_boreholes_dir = str(boreholes_dir) + "/individual_boreholes_compilation"

# Set up filename extensions for individual boreholes
conductivities_extension = "_conds"
temperatures_extension = "_temps"
z_T_k_extension = "_temps_k"
temperatures_sigma_extension = "_temps_uncertainties"
conductivities_sigma_extension = "_conds_uncertainties"

# TODO Where to put this? Set up filename for heat-flow results summary file. Should be in calcs folder
# all_boreholes_heat_flow_summary_file = boreholes_dir + '/all_boreholes_heat_flow_summary_nsim' + str(nsim) + '.dat'


### SET USER-SPECIFIED VARIABLES AND ANALYSIS OPTIONS ###

### Borehole constants ###
# Uncertainties for borehole records which don't have recorded uncertainties TODO Set all these in input files
sigma_z_T_cst = 1
sigma_T_cst = 1
sigma_z_k_cst = 1
sigma_k_cst = 0.25

# Rock properties - TODO add realistic uncertainties here. Could also do by lithology very easily
cp = 800.0
sigma_cp = 100.0
rho = 2700.0
sigma_rho = 300.0

# Choose whether to use quoted errors (i.e. errors recorded in original sources) or assigned errors (i.e. errors that reflect my confidence in the data)
errors_option = 'assigned'
# UKOGL well tops do not come with errors - set them here
ukogl_well_tops_z_error_m = 10

### TODO Add functionality to handle bottom-hole temperatures
### TODO Go through all raw data and make sure estimated uncertainties are consistent
# Set up list of all boreholes
borehole_list = [
    # "becklees",
    # "newbiggin",
    "silloth_no2",
    # "rowlands_gill",
    # "selkirk",
    # "ferneyrigg - doesn't currently work",
    # "rookhope",
    # "tocketts",
    # "woodland" - doesn't currently work",  
]

layer_option_list = [
    'whole_borehole',
    'bottom_hole',
    'cst_depth_intervals'
]


### TODO Set way to iterate over different options in lists for cut_top_m, subsample etc
### Set options for pre-processing of conductivity
# in_situ_k_preprocessing_dict = {
#     'cut_top_m_list':[None, 50],
#     'layer_division_list':[None, 100]
# }
in_situ_k_preprocessing_dict = {
    'cut_top_m_list':[None],
    # 'layer_division_list':[None, 100]
}
### Set options for pre-processing of temperature
# temps_preprocessing_dict = {
#     'cut_top_m_list':[None, 100],
#     'layer_division':{'layer_division_option':'no', 'layer_division_thickness_m':[100]},
# }
temps_preprocessing_dict = {
    'cut_top_m_list':[None],
    'cut_top_m_list':[100],
    'layer_division':{'layer_division_option':'no', 'layer_division_thickness_m':[100]},
}
### TODO One option for Monte Carlo analysis
### Set options for Monte Carlo sampling
### TODO Is it useful to just vary one variable (e.g. temperature) and keep others constant?
# monte_carlo_dict = {
#     'monte_carlo_option_list':['all', None, 'T', 'k', 'climate'],
#     'monte_carlo_nsim_list':[10],
#     'monte_carlo_T_subsample_factor_list':[2, None],
#     'monte_carlo_in_situ_k_subsample_factor_list':[2, None]
# }
monte_carlo_dict = {
    #'monte_carlo_option_list':[None],
    'monte_carlo_option_list':['all'],
    'monte_carlo_nsim_list':[100],
    'monte_carlo_T_subsample_factor_list':[2],
    'monte_carlo_in_situ_k_subsample_factor_list':[2]
}
# monte_carlo_T_subsample_type = 'regular'
monte_carlo_T_subsample_type = 'random'
# monte_carlo_in_situ_k_subsample_type = 'regular'
monte_carlo_in_situ_k_subsample_type = 'random'



### Set options for climatic corrections
# cc_type_list = [None, 'cst_borehole_conductivity', {'cst_specified_conductivity':[2]}, 'layer_diffusion']
cc_type_list = ['cst_borehole_conductivity']
pcc_option_list = ['yes']
rcc_option_list = ['yes']


# pc_options_dict = {
#     'no_pc_correction':{'no_pc_correction_option':'yes'},
#     'pc_correction_cst_borehole_conductivity':{
#         'pc_correction_cst_borehole_conductivity_option':'no',
#         'pc_correction_cst_borehole_conductivity_monte_carlo_option':'yes'},
#     'pc_correction_cst_conductivity_specified':{
#         'pc_correction_cst_conductivity_specified_option':'no',
#         'pc_correction_cst_conductivity_specified_k':[2, 3],
#         'pc_correction_cst_conductivity_specified_monte_carlo_option':'yes'},
#     'pc_correction_layer_diffusion':{
#         'pc_correction_layer_diffusion_option':'no',
#         'pc_correction_layer_diffusion_monte_carlo_option':'yes'}
# }



# cst_z_int_list = [
#     50,
#     # 100,
#     # 200,
#     # 1000
# ]
# orig_source_ref_dict = {
#     'Wheildon et al. (1985)':'wheildon1985'
# }

In [3]:
### RUN SELECTED FUNCTIONS FOR EACH BOREHOLE ###
for borehole in borehole_list:
    
    ### LOAD DATA AND CHECK FOR MISSING DATA ###
    borehole_path, raw_data_path, figures_path, borehole_year, temperatures_df, in_situ_conductivities_df, strat_interp_dict, skip_borehole, in_situ_conductivity_flag, borehole_name, number_strat_interps, strat_interp_names = load_and_pass_data.load_borehole_data(individual_boreholes_dir, borehole, temperatures_extension, conductivities_extension, all_borehole_overview_df, c82_k_dict, r87_k_dict, ukogl_well_tops_z_error_m, liths_lookup_dict_df, r87_unit_plot_dict, c82_unit_plot_dict)
    
        
    ### TODO Rename strat_interps as currently also includes in situ cond - this is confusing 
    print(borehole, borehole_name, borehole_year)
    print(strat_interp_names)
    
    ### RUN CALCULATIONS IF ALL DATA ARE PRESENT ###
    if skip_borehole == 'yes':
        continue
        
    ### SET UP DIRECTORIES ###
    calc_data_path = str(borehole_path) + "/calculated_data"
    general_python_functions.set_up_directory(calc_data_path)
    rc_path = calc_data_path + "/recent_climate"
    general_python_functions.set_up_directory(rc_path)    
        
    ### CUT RECENT SURFACE TEMPERATURE HISTORY TO BOREHOLE YEAR ###
    rc_t0, rc_sigma_t0, rc_t1, rc_sigma_t1, rc_ty_cut, rc_sigma_ty_cut, rc_t0_cut, rc_sigma_t0_cut, rc_t1_cut, rc_sigma_t1_cut, rc_deltaTs_cut, rc_sigma_deltaTs_cut, rc_t0_smoothing_cutoff, rc_sigma_t0_smoothing_cutoff, rc_t1_smoothing_cutoff, rc_sigma_t1_smoothing_cutoff, rc_ty_smoothing_cutoff_cut, rc_sigma_ty_smoothing_cutoff_cut, rc_t0_smoothing_cutoff_cut, rc_sigma_t0_smoothing_cutoff_cut, rc_t1_smoothing_cutoff_cut, rc_sigma_t1_smoothing_cutoff_cut, rc_deltaTs_smoothed_cut, rc_sigma_deltaTs_smoothed_cut = process_climate_histories.process_recent_climate_history(rc_path, rc_filename, borehole, rc_ty, rc_sigma_ty, rc_deltaTs, rc_sigma_deltaTs, borehole_year, rc_hist_csv, rc_smoothed_filename, rc_ty_smoothing_cutoff, rc_sigma_ty_smoothing_cutoff, rc_deltaTs_smoothed, rc_sigma_deltaTs_smoothed, rc_smoothed_outfile)    
        
    ### LOAD TEMPERATURE MEASUREMENTS AS NUMPY ARRAYS FOR EASE OF USE ###
    zT_m, T, zT_error_m, T_error = load_and_pass_data.read_temperature_dataframe_to_numpy_arrays(temperatures_df, errors_option)
        
    ### IF IN SITU CONDUCTIVITIES EXIST, LOAD AS NUMPY ARRAYS FOR EASE OF USE ###
    in_situ_zk_m, in_situ_k, in_situ_zk_error_m, in_situ_k_error = load_and_pass_data.read_in_situ_conductivity_dataframe_to_numpy_arrays(in_situ_conductivity_flag, in_situ_conductivities_df, errors_option)
        
        
    ### PLOT INPUT DATA ###
    ### Plot profile of temperature against depth
        
        
    ### Plot conductivity profiles, including stratigraphic interpretations when appropriate
    #                                         # # TODO Set max_depth_m_plot globally
    #                                         # max_depth_m_plot = np.max(strat_interp_lith_k_calcs_df['z1_m'].values)
    #                                         # # Plot only in situ conductivities
    #                                         # figure_name = figures_path + "_" + strat_interp_lith_k_name + '_in_situ_conds'
    #                                         # strat_k_option = 'no'
    #                                         # in_situ_k_option = 'yes'
    #                                         # spl_temp_option = 'no'
    #                                         # heat_flow_option = 'no'
    #                                         # heat_flow_file = 'na'
    #                                         # plotting_functions.plot_lithological_conductivity_temperature(max_depth_m_plot, plot_lith_fill_dict, plot_lith_fill_dict_keyword, strat_interp_lith_k_calcs_df, k_distribution, temperatures_df, zk_m, k, zk_error_m, k_error, np.nan, in_situ_k_option, spl_temp_option, strat_k_option, heat_flow_option, heat_flow_file, figure_name)


    ### Plot different profiles of conductivity with stratigraphy

    ### Plot palaeoclimate and recent temperature history
    # plotting_functions.plot_recent_climate_history(rc_ty, rc_deltaTs, rc_ty_smoothing_cutoff, rc_deltaTs_smoothed, rc_ty_cut, rc_deltaTs_cut, rc_ty_smoothing_cutoff_cut, rc_deltaTs_smoothed_cut, figures_path)




    ###### RUN ANALYSES FOR EACH CONDUCTIVITY PROFILE ######

    ### Set up top-level directories for calculations and save temperature dataframe within this structure ###
    cond_calcs_path, local_temps_calcs_path, local_temps_calcs_file = load_and_pass_data.set_up_calcs_folder_top(calc_data_path, borehole, temperatures_df)

    ### Load conductivity for stratigraphic interpretations ###
    ### Skip iteration if no stratigraphic interpretations corresponding to borehole ###
    if len(strat_interp_names) == 0:
        continue
        
    for strat_interp_name in strat_interp_names:
        # TODO temp
        if strat_interp_name != 'in_situ_conds':
            if 1 == 1: continue
                
        print("\n", strat_interp_name)
        ### Loop over different profiles of thermal conductivity (e.g. in situ, UKOGL well tops, BGS borehole scans)
        for strat_interp_lith_k_option in strat_interp_dict[strat_interp_name]['strat_interp_lith_cond'].keys():
            print(strat_interp_lith_k_option)
            
            
            ### Read in details of stratigraphic interpretation ###
            strat_interp_lith_k_dict, strat_interp_lith_k_name, strat_interp_lith_k_file, strat_interp_lith_k_calcs_file, run_calcs_option, plot_lith_fill_dict, plot_lith_fill_dict_keyword, k_distribution = load_and_pass_data.read_strat_interp_lith_cond_dict(strat_interp_dict, strat_interp_name, strat_interp_lith_k_option)
            
            ### Run calculations if specified ###
            if run_calcs_option != True:
                continue

            ### Set up input conductivity dataframe and destinations for saving calculations ###
            strat_interp_k_calcs_path, strat_interp_lith_k_calcs_path, local_strat_interp_lith_k_calcs_file, strat_interp_lith_k_calcs_file, strat_interp_lith_k_calcs_df = load_and_pass_data.set_up_calcs_folder_cond_top(cond_calcs_path, strat_interp_name, strat_interp_lith_k_name, borehole, strat_interp_lith_k_calcs_file)

            ### Read layered conductivity data into individual numpy arrays for ease of use ###
            layer_zmid_m, layer_zmid_error_m, layer_z0_m, layer_z1_m, layer_mean_k, layer_z0_error_m, layer_z1_error_m, layer_mean_k_error, layer_min_k, layer_max_k, layer_z_plot, layer_mean_k_plot, layer_stdev_k_plot, layer_min_k_plot, layer_max_k_plot = load_and_pass_data.read_conductivity_dataframe_to_numpy_arrays(k_distribution, strat_interp_lith_k_calcs_df, errors_option)

            # TODO Temp - set all uncertainties on k to 0.1
            # layer_mean_k_error[:] = 0.1
#                         layer_stdev_k_plot[:] = 0.1


            ### TODO Plot comparison between different inputs and outputs at end - need to save them to arrays for details of plotting

            #
            # plotting_functions.plot_lithological_conductivity_temperature(max_depth_m_plot, plot_lith_fill_dict, plot_lith_fill_dict_keyword, strat_interp_lith_k_calcs_df, k_distribution, temperatures_df, zk_m, k, zk_error_m, k_error, np.nan, in_situ_k_option, spl_temp_option, strat_k_option, heat_flow_option, heat_flow_file, figure_name)
            
            T_suffix = "_T"
            if strat_interp_name != 'in_situ_conds':
                k_suffix = "_k_" + strat_interp_name + "_" + strat_interp_lith_k_option
                print(strat_interp_lith_k_option, 'strat_interp_lith_k_option')
            else:
                k_suffix = "_k_" + strat_interp_name

           
                


            ### LOOP OVER OPTIONS FOR REMOVING CONDUCTIVITIES NEAR TOP OF BOREHOLE ###
            # TODO No point removing conductivities near top of borehole
            # # Only cut top conductivities if they are in situ measurements
            # if strat_interp_name == 'in_situ_conds':
            #     conds_cut_top_m_list = in_situ_conds_preprocessing_dict['cut_top_m_list']
            # else:
            #     conds_cut_top_m_list == [None]
            #
            # for conds_cut_top_m in conds_cut_top_m_list:
            #     zT_m_prep, T_prep, zT_error_m, T_error, T_suffix = load_and_pass_data.cut_top_depth(zT_m, T, zT_error_m, T_error, cut_top_m, suffix_root='_T')
            #
            # exit()
            #
            #
            #     # in_situ_conds_preprocessing_dict = {
            #     #     'cut_top_m_list':[None, 50],
            #     #     'layer_division_list':[None, 100]},
            #     # }
            #
            # ### TODO Add splitting into layers for in situ conductivities
            #
            #
            #
            k_cut_suffix = k_suffix


            ### LOOP OVER OPTIONS FOR REMOVING TEMPERATURES NEAR TOP OF BOREHOLE AND OVER OPTIONS FOR MONTE CARLO SUBSAMPLING ###
            for cut_top_m, monte_carlo_option in itertools.product(temps_preprocessing_dict['cut_top_m_list'], monte_carlo_dict['monte_carlo_option_list']):
                
                zT_m_cut, T_cut_suffix, T_cut, zT_error_m_cut, T_error_cut = load_and_pass_data.cut_top_depth(zT_m, cut_top_m, T_suffix, T, zT_error_m, T_error)
                monte_carlo_nsim_list, monte_carlo_T_subsample_factor_list, monte_carlo_in_situ_k_subsample_factor_list, monte_carlo_T_option, monte_carlo_k_option, monte_carlo_Tsurf_option = monte_carlo_sampling.set_up_monte_carlo_options(monte_carlo_dict, strat_interp_name, monte_carlo_option)
                
                ### TODO Make new functions/dictionaries to house lots of variables
                ### TODO Is this the best order of for loops? For instance, do I want to calculate all specified types of climate correction for each Monte Carlo sim?
                ### TODO Should averaging of Monte Carlo results be weighted by uncertainty in each MC estimate?
                ### TODO Make dictionary/outfile structure so can plot results from all conductivity types next to each other at end
                for cc_type, pcc_option, rcc_option, monte_carlo_T_subsample_factor, monte_carlo_in_situ_k_subsample_factor, monte_carlo_nsim in itertools.product(cc_type_list, pcc_option_list, rcc_option_list, monte_carlo_T_subsample_factor_list, monte_carlo_in_situ_k_subsample_factor_list, monte_carlo_nsim_list):
                    pprint.pprint((cc_type, pcc_option, rcc_option, monte_carlo_T_subsample_factor, monte_carlo_in_situ_k_subsample_factor, monte_carlo_nsim))
                    
                    ### CREATE STORAGE ARRAYS FOR MONTE CARLO OUTPUT ###
                    zk_interp_number_steps = 1000
                    cc_interp_number_steps = 1000
            
                    zT_m_input_mc_all, T_input_mc_all, T_input_cc_mc_all, zk_interp_number_steps, layer_zk_m_input_plot_mc_all, layer_k_input_plot_mc_all, whole_bullard_R1_downward_mc_all, whole_bullard_sigma_R1_downward_mc_all, whole_bullard_R1_upward_mc_all, whole_bullard_sigma_R1_upward_mc_all, whole_bullard_q_downward_TvR_mc_all, whole_bullard_sigma_q_downward_TvR_mc_all, whole_bullard_c_downward_TvR_mc_all, whole_bullard_sigma_c_downward_TvR_mc_all, whole_bullard_q_downward_RvT_mc_all, whole_bullard_sigma_q_downward_RvT_mc_all, whole_bullard_invc_downward_RvT_mc_all, whole_bullard_sigma_invc_downward_RvT_mc_all, whole_bullard_q_upward_TvR_mc_all, whole_bullard_sigma_q_upward_TvR_mc_all, whole_bullard_c_upward_TvR_mc_all, whole_bullard_sigma_c_upward_TvR_mc_all, whole_bullard_q_upward_RvT_mc_all, whole_bullard_sigma_q_upward_RvT_mc_all, whole_bullard_invc_upward_RvT_mc_all, whole_bullard_sigma_invc_upward_RvT_mc_all, whole_bullard_R1_downward_cc_mc_all, whole_bullard_sigma_R1_downward_cc_mc_all, whole_bullard_R1_upward_cc_mc_all, whole_bullard_sigma_R1_upward_cc_mc_all, whole_bullard_q_downward_TvR_cc_mc_all, whole_bullard_sigma_q_downward_TvR_cc_mc_all, whole_bullard_c_downward_TvR_cc_mc_all, whole_bullard_sigma_c_downward_TvR_cc_mc_all, whole_bullard_q_downward_RvT_cc_mc_all, whole_bullard_sigma_q_downward_RvT_cc_mc_all, whole_bullard_invc_downward_RvT_cc_mc_all, whole_bullard_sigma_invc_downward_RvT_cc_mc_all, whole_bullard_q_upward_TvR_cc_mc_all, whole_bullard_sigma_q_upward_TvR_cc_mc_all, whole_bullard_c_upward_TvR_cc_mc_all, whole_bullard_sigma_c_upward_TvR_cc_mc_all, whole_bullard_q_upward_RvT_cc_mc_all, whole_bullard_sigma_q_upward_RvT_cc_mc_all, whole_bullard_invc_upward_RvT_cc_mc_all, whole_bullard_sigma_invc_upward_RvT_cc_mc_all, pc_deltaTs_input_plot_mc_all, pc_t_input_plot_mc_all, rc_deltaTs_input_plot_mc_all, rc_t_input_plot_mc_all = monte_carlo_sampling.set_up_monte_carlo_output_arrays(monte_carlo_nsim, zT_m, zk_interp_number_steps, cc_interp_number_steps)
                    

                    for jsim in range(monte_carlo_nsim):

                        T_suffix_mc = T_cut_suffix
                        k_suffix_mc = k_cut_suffix
                        pc_suffix_mc = pc_suffix
                        rc_suffix_mc = rc_suffix
                        
#                         print('\n', k_suffix, '\n')


                        ### SUBSAMPLE AND PERTURB TEMPERATURE MEASUREMENTS ###
                        zT_m_subsampled, T_subsampled, zT_error_m_subsampled, T_error_subsampled, T_suffix_subsampled, zT_m_input, zT_error_m_input, T_input, T_error_input, T_suffix_mc = monte_carlo_sampling.perturb_T(T_suffix_mc, monte_carlo_T_subsample_type, monte_carlo_T_subsample_factor, zT_m_cut, T_cut, zT_error_m_cut, T_error_cut, monte_carlo_T_option)


                        ### SUBSAMPLE AND PERTURB CONDUCTIVITY MEASUREMENTS ###
                        # TODO Clip thermal conductivity to reasonable values - what are values to take?
                        k_low_clip = 1e-3

                        layer_zmid_m_subsampled, layer_zmid_error_m_subsampled, layer_z0_m_subsampled, layer_z0_error_m_subsampled, layer_z1_m_subsampled, layer_z1_error_m_subsampled, layer_mean_k_subsampled, layer_mean_k_error_subsampled, layer_min_k_subsampled, layer_max_k_subsampled, k_suffix_subsampled, layer_zmid_m_input, layer_zmid_error_m_input, layer_z0_m_input, layer_z0_error_m_input, layer_z1_m_input, layer_z1_error_m_input, layer_k_input, layer_k_error_input, layer_min_k_input, layer_max_k_input, layer_delta_z_m_input, layer_delta_z_error_m_input, layer_zk_m_input_plot, layer_k_input_plot, layer_k_error_input_plot, layer_min_k_input_plot, layer_max_k_input_plot, k_suffix_mc = monte_carlo_sampling.perturb_k(layer_zmid_m, k_suffix_mc, monte_carlo_in_situ_k_subsample_type, monte_carlo_in_situ_k_subsample_factor, layer_zmid_error_m, layer_z0_m, layer_z1_m, layer_mean_k, layer_z0_error_m, layer_z1_error_m, layer_mean_k_error, layer_min_k, layer_max_k, monte_carlo_k_option, k_low_clip, k_distribution)
                        


                        ### ESTIMATE MEAN CONDUCTIVITIES FOR WHOLE BOREHOLE ### TODO Need to account for whether Monte Carlo sampling or error combination
                        mean_k_all, sigma_mean_k_all, sigma_weighted_mean_k_all, stdev_sigma_weighted_mean_k_all, hmean_k_all, sigma_hmean_k_all, gmean_k_all, sdfactor_gmean_k_all, lower_bound_gmean_k_all, upper_bound_gmean_k_all, gmean_minus_k_all, gmean_plus_k_all, depth_weighted_mean_k_all, stdev_depth_weighted_mean_k_all, depth_weighted_mean_k_z1, sigma_depth_weighted_mean_k_z1, depth_weighted_hmean_k_z1, sigma_depth_weighted_hmean_k_z1 = heat_flow_functions.estimate_mean_conductivities(layer_k_input, layer_k_error_input, layer_min_k_input, layer_max_k_input, layer_delta_z_m_input, layer_delta_z_error_m_input, k_suffix_mc, k_distribution, monte_carlo_k_option)


                        ### TODO Add Monte Carlo sampling for density and heat capacity
                        mean_cp_all = 800.0
                        sigma_mean_cp_all = 100.0
                        mean_rho_all = 2700.0
                        sigma_mean_rho_all = 300.0


                        ### PERTURB RECORDS OF PALAEOCLIMATIC AND RECENT SURFACE TEMPERATURES ###
                        # TODO Need to add functionality to ensure that times are always montonically increasing
                        pc_t0_input, pc_sigma_t0_input, pc_t1_input, pc_sigma_t1_input, pc_deltaTs_input, pc_sigma_deltaTs_input, rc_t0_input, rc_sigma_t0_input, rc_t1_input, rc_sigma_t1_input, rc_deltaTs_input, rc_sigma_deltaTs_input, pc_suffix_mc, rc_suffix_mc = monte_carlo_sampling.perturb_climate(monte_carlo_Tsurf_option, pc_sigma_t_dist, pc_t0, pc_sigma_t0, pc_t1, pc_sigma_t1, pc_sigma_deltaTs_dist, pc_deltaTs, pc_sigma_deltaTs, rc_sigma_ty_smoothing_cutoff_dist, rc_t0_smoothing_cutoff_cut, rc_sigma_t0_smoothing_cutoff_cut, rc_t1_smoothing_cutoff_cut, rc_sigma_t1_smoothing_cutoff_cut, rc_sigma_deltaTs_smoothed_dist, rc_deltaTs_smoothed_cut, rc_sigma_deltaTs_smoothed_cut, pc_suffix_mc, rc_suffix_mc)
                        
                        
                        
                        ### ESTIMATE CORRECTIONS DUE TO SURFACE TEMPERATURES ###
                        # Estimate corrections at temperature depths
                        # Times are measured back from present. t1 are end of each interval (i.e. most recent), t0 are start of each interval
                        ### TODO Add uncertainties to climate correction
                        ### TODO Add calculation of climatic correction not assuming constant conductivity


                        # ### TODO Temp. Temperature spline to thermal conductivity depths
#                                                             # Spline interpolation to thermal conductivity data
#                                                             spl = UnivariateSpline(zT_m_input, T_input)
#                                                             T_k = spl(z_k)
                        z_m_palaeoclimatic_background = np.arange(0, 10000, 100)
                        z_error_m_palaeoclimatic_background = np.ones(np.size(z_m_palaeoclimatic_background))



                        if pcc_option == 'yes':
                            # Estimate palaeoclimatic corrections at regular depths for plotting
                            palaeoclimatic_delta_T_zT_palaeoclimatic_background, palaeoclimatic_sigma_delta_T_zT_palaeoclimatic_background = climatic_correction_functions.run_climatic_corrections(monte_carlo_k_option, monte_carlo_T_option, monte_carlo_Tsurf_option, cc_type, mean_rho_all, sigma_mean_rho_all, mean_cp_all, sigma_mean_cp_all, mean_k_all, sigma_mean_k_all, z_m_palaeoclimatic_background, z_error_m_palaeoclimatic_background, layer_z0_m_input, layer_z0_error_m_input, layer_z1_m_input, layer_z1_error_m_input, k_distribution, layer_k_input, layer_k_error_input, layer_min_k_input, layer_max_k_input, pc_t0_input, pc_sigma_t0_input, pc_t1_input, pc_sigma_t1_input, pc_deltaTs_input, pc_sigma_deltaTs_input)
                            # Estimate palaeoclimatic corrections at temperature depths
                            palaeoclimatic_delta_T_zT, palaeoclimatic_sigma_delta_T_zT = climatic_correction_functions.run_climatic_corrections(monte_carlo_k_option, monte_carlo_T_option, monte_carlo_Tsurf_option, cc_type, mean_rho_all, sigma_mean_rho_all, mean_cp_all, sigma_mean_cp_all, mean_k_all, sigma_mean_k_all, zT_m_input, zT_error_m_input, layer_z0_m_input, layer_z0_error_m_input, layer_z1_m_input, layer_z1_error_m_input, k_distribution, layer_k_input, layer_k_error_input, layer_min_k_input, layer_max_k_input, pc_t0_input, pc_sigma_t0_input, pc_t1_input, pc_sigma_t1_input, pc_deltaTs_input, pc_sigma_deltaTs_input)
                            # TODO Include in correction module. Add palaeoclimate corrections to input temperatures
                            T_input_pc = T_input + palaeoclimatic_delta_T_zT
                            if monte_carlo_k_option != 'yes' and monte_carlo_T_option != 'yes' and monte_carlo_Tsurf_option != 'yes':
                                T_error_input_pc = T_error_input + palaeoclimatic_sigma_delta_T_zT
                            else:
                                T_error_input_pc = None
                        else:
                            palaeoclimatic_delta_T_zT_palaeoclimatic_background, palaeoclimatic_sigma_delta_T_zT_palaeoclimatic_background, palaeoclimatic_delta_T_zT, palaeoclimatic_sigma_delta_T_zT = np.zeros(np.size(T_input)), np.zeros(np.size(T_input)), None, None

                        zT_m_rc_background = np.arange(0, 500, 1)
                        zT_error_m_rc_background = np.ones(np.size(zT_m_rc_background))
                        if rcc_option == 'yes':
                            # Estimate corrections due to recent climate at regular depths for plotting
                            recent_climatic_delta_T_zT_rc_background, recent_climatic_sigma_delta_T_zT_rc_background = climatic_correction_functions.run_climatic_corrections(monte_carlo_k_option, monte_carlo_T_option, monte_carlo_Tsurf_option, cc_type, mean_rho_all, sigma_mean_rho_all, mean_cp_all, sigma_mean_cp_all, mean_k_all, sigma_mean_k_all, zT_m_rc_background, zT_error_m_rc_background, layer_z0_m_input, layer_z0_error_m_input, layer_z1_m_input, layer_z1_error_m_input, k_distribution, layer_k_input, layer_k_error_input, layer_min_k_input, layer_max_k_input, rc_t0_input, rc_sigma_t0_input, rc_t1_input, rc_sigma_t1_input, rc_deltaTs_input, rc_sigma_deltaTs_input)
                            # Estimate corrections due to recent climate at temperature depths
                            recent_climatic_delta_T_zT, recent_climatic_sigma_delta_T_zT = climatic_correction_functions.run_climatic_corrections(monte_carlo_k_option, monte_carlo_T_option, monte_carlo_Tsurf_option, cc_type, mean_rho_all, sigma_mean_rho_all, mean_cp_all, sigma_mean_cp_all, mean_k_all, sigma_mean_k_all, zT_m_input, zT_error_m_input, layer_z0_m_input, layer_z0_error_m_input, layer_z1_m_input, layer_z1_error_m_input, k_distribution, layer_k_input, layer_k_error_input, layer_min_k_input, layer_max_k_input, rc_t0_input, rc_sigma_t0_input, rc_t1_input, rc_sigma_t1_input, rc_deltaTs_input, rc_sigma_deltaTs_input)
                            # Add recent climatic corrections to input temperatures
                            T_input_rc = T_input + recent_climatic_delta_T_zT
                            # TODO Include in correction module
                            if monte_carlo_k_option != 'yes' and monte_carlo_T_option != 'yes' and monte_carlo_Tsurf_option != 'yes':
                                T_error_input_rc = T_error_input + recent_climatic_sigma_delta_T_zT
                            else:
                                T_error_input_rc = None
                        else:
                            recent_climatic_delta_T_zT, recent_climatic_delta_T_zT, recent_climatic_delta_T_zT_rc_background, recent_climatic_sigma_delta_T_zT_rc_background = np.zeros(np.size(T_input)), np.zeros(np.size(T_input)), None, None

                        T_input_cc = T_input + palaeoclimatic_delta_T_zT + recent_climatic_delta_T_zT
                        # TODO Include in correction module
                        if monte_carlo_k_option != 'yes' and monte_carlo_T_option != 'yes' and monte_carlo_Tsurf_option != 'yes':
                            T_error_input_cc = T_error_input + palaeoclimatic_sigma_delta_T_zT + recent_climatic_sigma_delta_T_zT
                        else:
                            T_error_input_cc = None


                        ### ESTIMATE HEAT FLOWS USING ALL AVAILABLE TEMPERATURE MEASUREMENTS ###
                        ### Without climate corrections
                        ### Estimate heat flows using Bullard plot
                        R1_downward, sigma_R1_downward, R1_upward, sigma_R1_upward, q_downward_TvR, sigma_q_downward_TvR, q_round_downward_TvR, sigma_q_round_downward_TvR, Q_downward_TvR, sigma_Q_downward_TvR, Q_round_downward_TvR, sigma_Q_round_downward_TvR, c_downward_TvR, sigma_c_downward_TvR, c_round_downward_TvR, sigma_c_round_downward_TvR, q_downward_RvT, sigma_q_downward_RvT, q_round_downward_RvT, sigma_q_round_downward_RvT, Q_downward_RvT, sigma_Q_downward_RvT, Q_round_downward_RvT, sigma_Q_round_downward_RvT, invq_downward_RvT, sigma_invq_downward_RvT, invc_downward_RvT, sigma_invc_downward_RvT, q_upward_TvR, sigma_q_upward_TvR, q_round_upward_TvR, sigma_q_round_upward_TvR, Q_upward_TvR, sigma_Q_upward_TvR, Q_round_upward_TvR, sigma_Q_round_upward_TvR, c_upward_TvR, sigma_c_upward_TvR, c_round_upward_TvR, sigma_c_round_upward_TvR, q_upward_RvT, sigma_q_upward_RvT, q_round_upward_RvT, sigma_q_round_upward_RvT, Q_upward_RvT, sigma_Q_upward_RvT, Q_round_upward_RvT, sigma_Q_round_upward_RvT, invq_upward_RvT, sigma_invq_upward_RvT, invc_upward_RvT, sigma_invc_upward_RvT = heat_flow_functions.estimate_heat_flows_from_k_layers_and_T_depths_bullard_plot(layer_z0_m_input, layer_z0_error_m_input, layer_z1_m_input, layer_z1_error_m_input, layer_k_input, layer_k_error_input, zT_m_input, zT_error_m_input, T_input, T_error_input)
                        ### With climate corrections
                        ### Estimate heat flows using Bullard plot
                        R1_downward_cc, sigma_R1_downward_cc, R1_upward_cc, sigma_R1_upward_cc, q_downward_TvR_cc, sigma_q_downward_TvR_cc, q_round_downward_TvR_cc, sigma_q_round_downward_TvR_cc, Q_downward_TvR_cc, sigma_Q_downward_TvR_cc, Q_round_downward_TvR_cc, sigma_Q_round_downward_TvR_cc, c_downward_TvR_cc, sigma_c_downward_TvR_cc, c_round_downward_TvR_cc, sigma_c_round_downward_TvR_cc, q_downward_RvT_cc, sigma_q_downward_RvT_cc, q_round_downward_RvT_cc, sigma_q_round_downward_RvT_cc, Q_downward_RvT_cc, sigma_Q_downward_RvT_cc, Q_round_downward_RvT_cc, sigma_Q_round_downward_RvT_cc, invq_downward_RvT_cc, sigma_invq_downward_RvT_cc, invc_downward_RvT_cc, sigma_invc_downward_RvT_cc, q_upward_TvR_cc, sigma_q_upward_TvR_cc, q_round_upward_TvR_cc, sigma_q_round_upward_TvR_cc, Q_upward_TvR_cc, sigma_Q_upward_TvR_cc, Q_round_upward_TvR_cc, sigma_Q_round_upward_TvR_cc, c_upward_TvR_cc, sigma_c_upward_TvR_cc, c_round_upward_TvR_cc, sigma_c_round_upward_TvR_cc, q_upward_RvT_cc, sigma_q_upward_RvT_cc, q_round_upward_RvT_cc, sigma_q_round_upward_RvT_cc, Q_upward_RvT_cc, sigma_Q_upward_RvT_cc, Q_round_upward_RvT_cc, sigma_Q_round_upward_RvT_cc, invq_upward_RvT_cc, sigma_invq_upward_RvT_cc, invc_upward_RvT_cc, sigma_invc_upward_RvT_cc = heat_flow_functions.estimate_heat_flows_from_k_layers_and_T_depths_bullard_plot(layer_z0_m_input, layer_z0_error_m_input, layer_z1_m_input, layer_z1_error_m_input, layer_k_input, layer_k_error_input, zT_m_input, zT_error_m_input, T_input_cc, T_error_input_cc)

                        ### TODO Put in new function
                        zT_m_input_mc_all[jsim, 0:np.size(zT_m_input)] = zT_m_input
                        T_input_mc_all[jsim, 0:np.size(zT_m_input)] = T_input
                        T_input_cc_mc_all[jsim, 0:np.size(zT_m_input)] = T_input_cc

                            # if monte_carlo_k_option is None:
                            # layer_k_error_input_plot_mc_all[jsim, :] = layer_k_error_input_plot
                            # layer_min_k_input_plot_mc_all[jsim, :] = layer_min_k_input_plot
                            # layer_max_k_input_plot_mc_all[jsim, :] = layer_max_k_input_plot


                        ### TODO Check whether these arrays are set up correctly. Fill in arrays for plotting perturbed conductivity
                        # layer_zk_m_input_plot_mc_all[jsim,:] = np.arange(0, np.max(layer_z_plot), zk_interp_step_m)
                        layer_zk_m_input_plot_mc_all[jsim,:] = np.linspace(0, np.max(layer_z_plot), num=zk_interp_number_steps)
                        for layer_z0_m_input_index in range(np.size(layer_z0_m_input)):
                            if np.size(np.where(layer_zk_m_input_plot_mc_all[jsim,:] >= layer_z0_m_input[layer_z0_m_input_index])[0]) != 0:
                                layer_k_input_plot_mc_all_index = np.where(layer_zk_m_input_plot_mc_all[jsim,:] >= layer_z0_m_input[layer_z0_m_input_index])[0][0]
                                layer_k_input_plot_mc_all[jsim, layer_k_input_plot_mc_all_index::] = layer_k_input[layer_z0_m_input_index]
                                            
                        
                        # Write out climatic histories for plotting at end of MC simulation
                        pc_t_input_plot_mc_all[jsim,:] = np.linspace(0, np.max(pc_t0), num=cc_interp_number_steps)
                        for pc_t_input_index in range(np.size(pc_t0_input)):
                            if np.size(np.where(pc_t_input_plot_mc_all[jsim,:] >= np.flip(pc_t0_input)[pc_t_input_index])[0]) != 0:
                                pc_deltaTs_input_plot_mc_all_index = np.where(pc_t_input_plot_mc_all[jsim,:] >= np.flip(pc_t0_input)[pc_t_input_index])[0][0]
                                pc_deltaTs_input_plot_mc_all[jsim, pc_deltaTs_input_plot_mc_all_index::] = np.flip(pc_deltaTs_input)[pc_t_input_index]
                        pc_deltaTs_input_plot_mc_all[jsim, 0] = pc_deltaTs_input_plot_mc_all[jsim, 1]

                        
                        ### Write out uncorrected results to storage arrays
                        whole_bullard_R1_downward_mc_all[jsim, 0:np.size(zT_m_input)] = R1_downward
                        whole_bullard_sigma_R1_downward_mc_all[jsim, 0:np.size(zT_m_input)]  = sigma_R1_downward
                        whole_bullard_R1_upward_mc_all[jsim, 0:np.size(zT_m_input)] = R1_upward
                        whole_bullard_sigma_R1_upward_mc_all[jsim, 0:np.size(zT_m_input)]  = sigma_R1_upward

                        whole_bullard_q_downward_TvR_mc_all[jsim] = q_downward_TvR
                        whole_bullard_sigma_q_downward_TvR_mc_all[jsim] = sigma_q_downward_TvR
                        whole_bullard_c_downward_TvR_mc_all[jsim] = c_downward_TvR
                        whole_bullard_sigma_c_downward_TvR_mc_all[jsim] = sigma_c_downward_TvR
                        whole_bullard_q_downward_RvT_mc_all[jsim] = q_downward_RvT
                        whole_bullard_sigma_q_downward_RvT_mc_all[jsim] = sigma_q_downward_RvT
                        whole_bullard_invc_downward_RvT_mc_all[jsim] = invc_downward_RvT
                        whole_bullard_sigma_invc_downward_RvT_mc_all[jsim] = sigma_invc_downward_RvT

                        whole_bullard_q_upward_TvR_mc_all[jsim] = q_upward_TvR
                        whole_bullard_sigma_q_upward_TvR_mc_all[jsim] = sigma_q_upward_TvR
                        whole_bullard_c_upward_TvR_mc_all[jsim] = c_upward_TvR
                        whole_bullard_sigma_c_upward_TvR_mc_all[jsim] = sigma_c_upward_TvR
                        whole_bullard_q_upward_RvT_mc_all[jsim] = q_upward_RvT
                        whole_bullard_sigma_q_upward_RvT_mc_all[jsim] = sigma_q_upward_RvT
                        whole_bullard_invc_upward_RvT_mc_all[jsim] = invc_upward_RvT
                        whole_bullard_sigma_invc_upward_RvT_mc_all[jsim] = sigma_invc_upward_RvT


                        ### Write out climate-corrected results to storage arrays
                        whole_bullard_R1_downward_cc_mc_all[jsim, 0:np.size(zT_m_input)] = R1_downward_cc
                        whole_bullard_sigma_R1_downward_cc_mc_all[jsim, 0:np.size(zT_m_input)]  = sigma_R1_downward_cc
                        whole_bullard_R1_upward_cc_mc_all[jsim, 0:np.size(zT_m_input)] = R1_upward_cc
                        whole_bullard_sigma_R1_upward_cc_mc_all[jsim, 0:np.size(zT_m_input)]  = sigma_R1_upward_cc

                        whole_bullard_q_downward_TvR_cc_mc_all[jsim] = q_downward_TvR_cc
                        whole_bullard_sigma_q_downward_TvR_cc_mc_all[jsim] = sigma_q_downward_TvR_cc
                        whole_bullard_c_downward_TvR_cc_mc_all[jsim] = c_downward_TvR_cc
                        whole_bullard_sigma_c_downward_TvR_cc_mc_all[jsim] = sigma_c_downward_TvR_cc
                        whole_bullard_q_downward_RvT_cc_mc_all[jsim] = q_downward_RvT_cc
                        whole_bullard_sigma_q_downward_RvT_cc_mc_all[jsim] = sigma_q_downward_RvT_cc
                        whole_bullard_invc_downward_RvT_cc_mc_all[jsim] = invc_downward_RvT_cc
                        whole_bullard_sigma_invc_downward_RvT_cc_mc_all[jsim] = sigma_invc_downward_RvT_cc
                        
                        whole_bullard_q_upward_TvR_cc_mc_all[jsim] = q_upward_TvR_cc
                        whole_bullard_sigma_q_upward_TvR_cc_mc_all[jsim] = sigma_q_upward_TvR_cc
                        whole_bullard_c_upward_TvR_cc_mc_all[jsim] = c_upward_TvR_cc
                        whole_bullard_sigma_c_upward_TvR_cc_mc_all[jsim] = sigma_c_upward_TvR_cc
                        whole_bullard_q_upward_RvT_cc_mc_all[jsim] = q_upward_RvT_cc
                        whole_bullard_sigma_q_upward_RvT_cc_mc_all[jsim] = sigma_q_upward_RvT_cc
                        whole_bullard_invc_upward_RvT_cc_mc_all[jsim] = invc_upward_RvT_cc
                        whole_bullard_sigma_invc_upward_RvT_cc_mc_all[jsim] = sigma_invc_upward_RvT_cc


                        # Plot Monte Carlo results: temperature, conductivity and resulting thermal resistivity, perturbed climatic histories, and climate-corrected temperatures
                        ### PLOT RESULTS OF MONTE CARLO ANALYSIS ###
                        if monte_carlo_option is not None and jsim == monte_carlo_nsim-1:
                            print('estimating Monte Carlo averages\n')
                            # TODO Add weighted averages
                            whole_bullard_q_downward_TvR_mc_mean, whole_bullard_q_downward_TvR_mc_stdev, whole_bullard_c_downward_TvR_mc_median, whole_bullard_Q_downward_TvR_mc_mean_round, whole_bullard_Q_downward_TvR_mc_stdev_round, whole_bullard_q_downward_TvR_mc_P10, whole_bullard_q_downward_TvR_mc_P50, whole_bullard_q_downward_TvR_mc_P90 = heat_flow_functions. estimate_mc_heat_flow_distribution_statistics(whole_bullard_q_downward_TvR_mc_all, whole_bullard_sigma_q_downward_TvR_mc_all, whole_bullard_c_downward_TvR_mc_all)
                            
                            # Sort all storage arrays by value of heat flow
                            
#                             print(np.shape(whole_bullard_q_downward_TvR_mc_all))
#                             print(np.shape(zT_m_input_mc_all))
                            
#                             print(np.shape(a))
#                             print(np.shape(b))
#                             print(np.shape(c))
                            
                            
                            whole_bullard_q_downward_TvR_mc_all_sorted, zT_m_input_mc_all_sorted, T_input_mc_all_sorted, T_input_cc_mc_all_sorted, zk_interp_number_steps_sorted, layer_zk_m_input_plot_mc_all_sorted, layer_k_input_plot_mc_all_sorted, whole_bullard_R1_downward_mc_all_sorted, whole_bullard_sigma_R1_downward_mc_all_sorted, whole_bullard_R1_upward_mc_all_sorted, whole_bullard_sigma_R1_upward_mc_all_sorted, whole_bullard_q_downward_TvR_mc_all_sorted, whole_bullard_sigma_q_downward_TvR_mc_all_sorted, whole_bullard_c_downward_TvR_mc_all_sorted, whole_bullard_sigma_c_downward_TvR_mc_all_sorted, whole_bullard_q_downward_RvT_mc_all_sorted, whole_bullard_sigma_q_downward_RvT_mc_all_sorted, whole_bullard_invc_downward_RvT_mc_all_sorted, whole_bullard_sigma_invc_downward_RvT_mc_all_sorted, whole_bullard_q_upward_TvR_mc_all_sorted, whole_bullard_sigma_q_upward_TvR_mc_all_sorted, whole_bullard_c_upward_TvR_mc_all_sorted, whole_bullard_sigma_c_upward_TvR_mc_all_sorted, whole_bullard_q_upward_RvT_mc_all_sorted, whole_bullard_sigma_q_upward_RvT_mc_all_sorted, whole_bullard_invc_upward_RvT_mc_all_sorted, whole_bullard_sigma_invc_upward_RvT_mc_all_sorted, whole_bullard_R1_downward_cc_mc_all_sorted, whole_bullard_sigma_R1_downward_cc_mc_all_sorted, whole_bullard_R1_upward_cc_mc_all_sorted, whole_bullard_sigma_R1_upward_cc_mc_all_sorted, whole_bullard_q_downward_TvR_cc_mc_all_sorted, whole_bullard_sigma_q_downward_TvR_cc_mc_all_sorted, whole_bullard_c_downward_TvR_cc_mc_all_sorted, whole_bullard_sigma_c_downward_TvR_cc_mc_all_sorted, whole_bullard_q_downward_RvT_cc_mc_all_sorted, whole_bullard_sigma_q_downward_RvT_cc_mc_all_sorted, whole_bullard_invc_downward_RvT_cc_mc_all_sorted, whole_bullard_sigma_invc_downward_RvT_cc_mc_all_sorted, whole_bullard_q_upward_TvR_cc_mc_all_sorted, whole_bullard_sigma_q_upward_TvR_cc_mc_all_sorted, whole_bullard_c_upward_TvR_cc_mc_all_sorted, whole_bullard_sigma_c_upward_TvR_cc_mc_all_sorted, whole_bullard_q_upward_RvT_cc_mc_all_sorted, whole_bullard_sigma_q_upward_RvT_cc_mc_all_sorted, whole_bullard_invc_upward_RvT_cc_mc_all_sorted, whole_bullard_sigma_invc_upward_RvT_cc_mc_all_sorted, pc_deltaTs_input_plot_mc_all_sorted, pc_t_input_plot_mc_all_sorted, rc_deltaTs_input_plot_mc_all_sorted, rc_t_input_plot_mc_all_sorted = general_python_functions.sort_arrays(whole_bullard_q_downward_TvR_mc_all, zT_m_input_mc_all, T_input_mc_all, T_input_cc_mc_all, zk_interp_number_steps, layer_zk_m_input_plot_mc_all, layer_k_input_plot_mc_all, whole_bullard_R1_downward_mc_all, whole_bullard_sigma_R1_downward_mc_all, whole_bullard_R1_upward_mc_all, whole_bullard_sigma_R1_upward_mc_all, whole_bullard_q_downward_TvR_mc_all, whole_bullard_sigma_q_downward_TvR_mc_all, whole_bullard_c_downward_TvR_mc_all, whole_bullard_sigma_c_downward_TvR_mc_all, whole_bullard_q_downward_RvT_mc_all, whole_bullard_sigma_q_downward_RvT_mc_all, whole_bullard_invc_downward_RvT_mc_all, whole_bullard_sigma_invc_downward_RvT_mc_all, whole_bullard_q_upward_TvR_mc_all, whole_bullard_sigma_q_upward_TvR_mc_all, whole_bullard_c_upward_TvR_mc_all, whole_bullard_sigma_c_upward_TvR_mc_all, whole_bullard_q_upward_RvT_mc_all, whole_bullard_sigma_q_upward_RvT_mc_all, whole_bullard_invc_upward_RvT_mc_all, whole_bullard_sigma_invc_upward_RvT_mc_all, whole_bullard_R1_downward_cc_mc_all, whole_bullard_sigma_R1_downward_cc_mc_all, whole_bullard_R1_upward_cc_mc_all, whole_bullard_sigma_R1_upward_cc_mc_all, whole_bullard_q_downward_TvR_cc_mc_all, whole_bullard_sigma_q_downward_TvR_cc_mc_all, whole_bullard_c_downward_TvR_cc_mc_all, whole_bullard_sigma_c_downward_TvR_cc_mc_all, whole_bullard_q_downward_RvT_cc_mc_all, whole_bullard_sigma_q_downward_RvT_cc_mc_all, whole_bullard_invc_downward_RvT_cc_mc_all, whole_bullard_sigma_invc_downward_RvT_cc_mc_all, whole_bullard_q_upward_TvR_cc_mc_all, whole_bullard_sigma_q_upward_TvR_cc_mc_all, whole_bullard_c_upward_TvR_cc_mc_all, whole_bullard_sigma_c_upward_TvR_cc_mc_all, whole_bullard_q_upward_RvT_cc_mc_all, whole_bullard_sigma_q_upward_RvT_cc_mc_all, whole_bullard_invc_upward_RvT_cc_mc_all, whole_bullard_sigma_invc_upward_RvT_cc_mc_all, pc_deltaTs_input_plot_mc_all, pc_t_input_plot_mc_all, rc_deltaTs_input_plot_mc_all, rc_t_input_plot_mc_all)
                            
#                             print(whole_bullard_q_downward_TvR_mc_all)
#                             print(whole_bullard_q_downward_TvR_mc_all_sorted)
#                             print(zT_m_input_mc_all)
#                             print(zT_m_input_mc_all_sorted)
                            
                            whole_bullard_q_downward_TvR_mc_mean_index = np.max(np.where(whole_bullard_q_downward_TvR_mc_all_sorted <= whole_bullard_q_downward_TvR_mc_mean))
                            whole_bullard_q_downward_TvR_mc_P10_index = np.max(np.where(whole_bullard_q_downward_TvR_mc_all_sorted <= whole_bullard_q_downward_TvR_mc_P10))
                            whole_bullard_q_downward_TvR_mc_P50_index = np.max(np.where(whole_bullard_q_downward_TvR_mc_all_sorted <= whole_bullard_q_downward_TvR_mc_P50))
                            whole_bullard_q_downward_TvR_mc_P90_index = np.max(np.where(whole_bullard_q_downward_TvR_mc_all_sorted <= whole_bullard_q_downward_TvR_mc_P90))
                            
                            print(whole_bullard_q_downward_TvR_mc_mean_index)
                            print(whole_bullard_q_downward_TvR_mc_all_sorted[whole_bullard_q_downward_TvR_mc_mean_index])
                            print(whole_bullard_q_downward_TvR_mc_mean)
                            
                            probdens, edges = np.histogram(whole_bullard_q_downward_TvR_mc_all_sorted, bins=50, density=True)
                            
                            plt.plot()
                        
                        
                            plt.bar(edges[:-1], probdens, width=np.diff(edges), align="edge")
                            plt.axvline(x=whole_bullard_q_downward_TvR_mc_mean, color='black')
                            plt.axvline(x=whole_bullard_q_downward_TvR_mc_P10, color='black')
                            plt.axvline(x=whole_bullard_q_downward_TvR_mc_P50, color='black')
                            plt.axvline(x=whole_bullard_q_downward_TvR_mc_P90, color='black')
                            plt.axvline(x=whole_bullard_q_downward_TvR_mc_all_sorted[whole_bullard_q_downward_TvR_mc_mean_index], color='red', alpha=50)
                            plt.axvline(x=whole_bullard_q_downward_TvR_mc_all_sorted[whole_bullard_q_downward_TvR_mc_P10_index], color='red', alpha=50)
                            plt.axvline(x=whole_bullard_q_downward_TvR_mc_all_sorted[whole_bullard_q_downward_TvR_mc_P50_index], color='red', alpha=50)
                            plt.axvline(x=whole_bullard_q_downward_TvR_mc_all_sorted[whole_bullard_q_downward_TvR_mc_P90_index], color='red', alpha=50)
                            
                            plt.plot()
                            plt.show()
                            
                            
                            
                            if 1==1:
                                continue
                            
                            a_sort, b_sort = general_python_functions.sort_arrays(whole_bullard_q_downward_TvR_mc_all, whole_bullard_sigma_q_downward_TvR_mc_all, whole_bullard_c_downward_TvR_mc_all)
                            
                            
                            
                            zT_m_input_mc_all, T_input_mc_all, T_input_cc_mc_all, zk_interp_number_steps, layer_zk_m_input_plot_mc_all, layer_k_input_plot_mc_all, whole_bullard_R1_downward_mc_all, whole_bullard_sigma_R1_downward_mc_all, whole_bullard_R1_upward_mc_all, whole_bullard_sigma_R1_upward_mc_all, whole_bullard_q_downward_TvR_mc_all, whole_bullard_sigma_q_downward_TvR_mc_all, whole_bullard_c_downward_TvR_mc_all, whole_bullard_sigma_c_downward_TvR_mc_all, whole_bullard_q_downward_RvT_mc_all, whole_bullard_sigma_q_downward_RvT_mc_all, whole_bullard_invc_downward_RvT_mc_all, whole_bullard_sigma_invc_downward_RvT_mc_all, whole_bullard_q_upward_TvR_mc_all, whole_bullard_sigma_q_upward_TvR_mc_all, whole_bullard_c_upward_TvR_mc_all, whole_bullard_sigma_c_upward_TvR_mc_all, whole_bullard_q_upward_RvT_mc_all, whole_bullard_sigma_q_upward_RvT_mc_all, whole_bullard_invc_upward_RvT_mc_all, whole_bullard_sigma_invc_upward_RvT_mc_all, whole_bullard_R1_downward_cc_mc_all, whole_bullard_sigma_R1_downward_cc_mc_all, whole_bullard_R1_upward_cc_mc_all, whole_bullard_sigma_R1_upward_cc_mc_all, whole_bullard_q_downward_TvR_cc_mc_all, whole_bullard_sigma_q_downward_TvR_cc_mc_all, whole_bullard_c_downward_TvR_cc_mc_all, whole_bullard_sigma_c_downward_TvR_cc_mc_all, whole_bullard_q_downward_RvT_cc_mc_all, whole_bullard_sigma_q_downward_RvT_cc_mc_all, whole_bullard_invc_downward_RvT_cc_mc_all, whole_bullard_sigma_invc_downward_RvT_cc_mc_all, whole_bullard_q_upward_TvR_cc_mc_all, whole_bullard_sigma_q_upward_TvR_cc_mc_all, whole_bullard_c_upward_TvR_cc_mc_all, whole_bullard_sigma_c_upward_TvR_cc_mc_all, whole_bullard_q_upward_RvT_cc_mc_all, whole_bullard_sigma_q_upward_RvT_cc_mc_all, whole_bullard_invc_upward_RvT_cc_mc_all, whole_bullard_sigma_invc_upward_RvT_cc_mc_all, pc_deltaTs_input_plot_mc_all, pc_t_input_plot_mc_all, rc_deltaTs_input_plot_mc_all, rc_t_input_plot_mc_all
                            
                            
                            whole_bullard_q_downward_TvR_mc_all_sort_index = np.argsort(whole_bullard_q_downward_TvR_mc_all)
                            print(whole_bullard_q_downward_TvR_mc_all_sort_index)
                            print(whole_bullard_q_downward_TvR_mc_all[whole_bullard_q_downward_TvR_mc_all_sort_index])
                            if 1==1:
                                continue
                            
                            # Find iterations closest to mean, P10, P50, P90 distributions
                            print(whole_bullard_q_downward_TvR_mc_mean)
                            print(whole_bullard_q_downward_TvR_mc_all)
                            
                            whole_bullard_q_downward_TvR_mc_mean_index = np.max(np.where(whole_bullard_q_downward_TvR_mc_all < whole_bullard_q_downward_TvR_mc_mean))
                            print(whole_bullard_q_downward_TvR_mc_mean_index)

                            print(whole_bullard_q_downward_TvR_mc_mean)
                            
                            if 1==1:
                                continue
                            
                            
                            whole_bullard_q_downward_RvT_mc_mean, whole_bullard_q_downward_RvT_mc_stdev, whole_bullard_invc_downward_RvT_mc_median, whole_bullard_Q_downward_RvT_mc_mean_round, whole_bullard_Q_downward_RvT_mc_stdev_round, whole_bullard_q_downward_RvT_mc_P10, whole_bullard_q_downward_RvT_mc_P50, whole_bullard_q_downward_RvT_mc_P90 = heat_flow_functions. estimate_mc_heat_flow_distribution_statistics(whole_bullard_q_downward_RvT_mc_all, whole_bullard_sigma_q_downward_RvT_mc_all, whole_bullard_invc_downward_RvT_mc_all)
                            
                            whole_bullard_q_upward_TvR_mc_mean, whole_bullard_q_upward_TvR_mc_stdev, whole_bullard_c_upward_TvR_mc_median, whole_bullard_Q_upward_TvR_mc_mean_round, whole_bullard_Q_upward_TvR_mc_stdev_round, whole_bullard_q_upward_TvR_mc_P10, whole_bullard_q_upward_TvR_mc_P50, whole_bullard_q_upward_TvR_mc_P90 = heat_flow_functions. estimate_mc_heat_flow_distribution_statistics(whole_bullard_q_upward_TvR_mc_all, whole_bullard_sigma_q_upward_TvR_mc_all, whole_bullard_c_upward_TvR_mc_all)
                            
                            whole_bullard_q_upward_RvT_mc_mean, whole_bullard_q_upward_RvT_mc_stdev, whole_bullard_invc_upward_RvT_mc_median, whole_bullard_Q_upward_RvT_mc_mean_round, whole_bullard_Q_upward_RvT_mc_stdev_round, whole_bullard_q_upward_RvT_mc_P10, whole_bullard_q_upward_RvT_mc_P50, whole_bullard_q_upward_RvT_mc_P90 = heat_flow_functions. estimate_mc_heat_flow_distribution_statistics(whole_bullard_q_upward_RvT_mc_all, whole_bullard_sigma_q_upward_RvT_mc_all, whole_bullard_invc_upward_RvT_mc_all)
                            
                            
                            
                            whole_bullard_q_downward_TvR_cc_mc_mean, whole_bullard_q_downward_TvR_cc_mc_stdev, whole_bullard_c_downward_TvR_cc_mc_median, whole_bullard_Q_downward_TvR_cc_mc_mean_round, whole_bullard_Q_downward_TvR_cc_mc_stdev_round, whole_bullard_q_downward_TvR_cc_mc_P10, whole_bullard_q_downward_TvR_cc_mc_P50, whole_bullard_q_downward_TvR_cc_mc_P90 = heat_flow_functions. estimate_mc_heat_flow_distribution_statistics(whole_bullard_q_downward_TvR_cc_mc_all, whole_bullard_sigma_q_downward_TvR_cc_mc_all, whole_bullard_c_downward_TvR_cc_mc_all)
                            
                            whole_bullard_q_downward_RvT_cc_mc_mean, whole_bullard_q_downward_RvT_cc_mc_stdev, whole_bullard_invc_downward_RvT_cc_mc_median, whole_bullard_Q_downward_RvT_cc_mc_mean_round, whole_bullard_Q_downward_RvT_cc_mc_stdev_round, whole_bullard_q_downward_RvT_cc_mc_P10, whole_bullard_q_downward_RvT_cc_mc_P50, whole_bullard_q_downward_RvT_cc_mc_P90 = heat_flow_functions. estimate_mc_heat_flow_distribution_statistics(whole_bullard_q_downward_RvT_cc_mc_all, whole_bullard_sigma_q_downward_RvT_cc_mc_all, whole_bullard_invc_downward_RvT_cc_mc_all)
                            
                            whole_bullard_q_upward_TvR_cc_mc_mean, whole_bullard_q_upward_TvR_cc_mc_stdev, whole_bullard_c_upward_TvR_cc_mc_median, whole_bullard_Q_upward_TvR_cc_mc_mean_round, whole_bullard_Q_upward_TvR_cc_mc_stdev_round, whole_bullard_q_upward_TvR_cc_mc_P10, whole_bullard_q_upward_TvR_cc_mc_P50, whole_bullard_q_upward_TvR_cc_mc_P90 = heat_flow_functions. estimate_mc_heat_flow_distribution_statistics(whole_bullard_q_upward_TvR_cc_mc_all, whole_bullard_sigma_q_upward_TvR_cc_mc_all, whole_bullard_c_upward_TvR_cc_mc_all)
                            
                            whole_bullard_q_upward_RvT_cc_mc_mean, whole_bullard_q_upward_RvT_cc_mc_stdev, whole_bullard_invc_upward_RvT_cc_mc_median, whole_bullard_Q_upward_RvT_cc_mc_mean_round, whole_bullard_Q_upward_RvT_cc_mc_stdev_round, whole_bullard_q_upward_RvT_cc_mc_P10, whole_bullard_q_upward_RvT_cc_mc_P50, whole_bullard_q_upward_RvT_cc_mc_P90 = heat_flow_functions. estimate_mc_heat_flow_distribution_statistics(whole_bullard_q_upward_RvT_cc_mc_all, whole_bullard_sigma_q_upward_RvT_cc_mc_all, whole_bullard_invc_upward_RvT_cc_mc_all)
                            
                            
                            print('plotting last')
                            
                            ### Plot palaeoclimatic history before and after Monte Carlo sampling
                            figure_name = figures_path + '_' + 'palaeoclimate.jpg'
                            pc_plot_input_option = 'no'
                            plotting_functions.plot_palaeoclimate(pc_deltaTs, pc_sigma_deltaTs, pc_t0, pc_t1, pc_sigma_t0, pc_sigma_t1, pc_deltaTs_input, pc_t0_input, pc_t1_input, pc_plot_input_option, figure_name)
                            figure_name = figures_path + '_' + 'palaeoclimate_mc.jpg'
                            pc_plot_input_option = 'yes'
                            plotting_functions.plot_palaeoclimate(pc_deltaTs, pc_sigma_deltaTs, pc_t0, pc_t1, pc_sigma_t0, pc_sigma_t1, pc_deltaTs_input, pc_t0_input, pc_t1_input, pc_plot_input_option, figure_name)

                            ### Plot recent temperature history before and after Monte Carlo sampling
                            figure_name = figures_path + '_' + 'recent_climate_history.jpg'
                            rc_plot_input_option='no'
                            plotting_functions.plot_recent_climate_history_new(rc_deltaTs, rc_t0, rc_t1, rc_deltaTs_cut, rc_t0_cut, rc_t1_cut, rc_deltaTs_smoothed_cut, rc_t0_smoothing_cutoff_cut, rc_t1_smoothing_cutoff_cut, rc_deltaTs_input, rc_t0_input, rc_t1_input, rc_ty, rc_plot_input_option, figure_name)
                            figure_name = figures_path + '_' + 'recent_climate_history_mc.jpg'
                            rc_plot_input_option='yes'
                            plotting_functions.plot_recent_climate_history_new(rc_deltaTs, rc_t0, rc_t1, rc_deltaTs_cut, rc_t0_cut, rc_t1_cut, rc_deltaTs_smoothed_cut, rc_t0_smoothing_cutoff_cut, rc_t1_smoothing_cutoff_cut, rc_deltaTs_input, rc_t0_input, rc_t1_input, rc_ty, rc_plot_input_option, figure_name)
                            
                            
                            
                            ### PLOT TEMPERATURE, STRATIGRAPHY AND CONDUCTIVITY ###
                            max_depth_m_plot = 100 * np.ceil(np.max(np.array([np.max(zT_m), np.max(layer_z1_m)]))/100)
                            T_range_min = np.min(np.array([np.min(T), np.min(T_input_cc)]))
                            T_range_max = np.max(np.array([np.max(T), np.max(T_input_cc)]))
                            T_range = np.array([T_range_min, T_range_max])

                            ### Plot input temperature, stratigraphy (if necessary), conductivity and resistivity (as empty box) in top row. Plot climatic records in lower row.
                            # Set up options for plotting temperature
                            figure_name = figures_path + '_a' + T_suffix + k_suffix + '_climate'
                            T_plotting_dict = {'number_lines':1, 'line0':{'line_type':'errorbar', 'zT_m_plot':zT_m, 'T_plot':T, 'zT_error_m_plot':zT_error_m, 'T_error_plot':T_error, 'fmt':'.k', 'alpha':1, 'markeredgecolor':None, 'figure_label':'a'}}
                            # Set up options for plotting thermal conductivity
                            if k_distribution != 'in_situ_normal':
                                k_plotting_dict = {'number_lines':1, 'line0':{'line_type':'strat_interp_lith_k', 'z0_m':strat_interp_lith_k_calcs_df['z0_m'], 'z1_m':strat_interp_lith_k_calcs_df['z1_m'], 'min_k':strat_interp_lith_k_calcs_df['min_k'], 'max_k':strat_interp_lith_k_calcs_df['max_k'], 'mean_k':strat_interp_lith_k_calcs_df['mean_k'], 'k_assigned_error':strat_interp_lith_k_calcs_df['k_assigned_error'], 'color':'black', 'alpha':0.5, 'zorder':5, 'figure_label':'b'}}
                            else:
                                k_plotting_dict = {'number_lines':1, 'line0':{'line_type':'mean_errorbar', 'layer_zk_m_plot':layer_zmid_m, 'layer_k_plot':layer_mean_k, 'layer_zk_error_m_plot':layer_zmid_error_m, 'layer_k_error_plot':layer_mean_k_error, 'fmt':'.k', 'alpha':1, 'zorder':5, 'figure_label':'b'}}
                            # Set up options for plotting thermal resistivity
                            res_plotting_dict = {'number_lines':1, 'line0':{'line_type':'errorbar', 'zR_m_plot':zT_m_input, 'R_plot':R1_downward, 'zR_error_m_plot':zT_error_m_input, 'R_error_plot':sigma_R1_downward, 'fmt':'.r', 'alpha':0, 'markeredgecolor':None, 'figure_label':'c'}}
                            # Set up options for plotting Bullard plots - plot as empty frames
                            bullard_TvR_plotting_dict = {'number_lines':1, 'line0':{'line_type':'emptyframe', 'empty_x':1, 'empty_y':1, 'figure_label':'e'}}
                            bullard_RvT_plotting_dict = None
                            # Set up options for plotting palaeoclimate
                            pc_plotting_dict = {'pc_plot_input_option':'no', 'pc_deltaTs':pc_deltaTs, 'pc_sigma_deltaTs':pc_sigma_deltaTs, 'pc_t0':pc_t0, 'pc_t1':pc_t1, 'pc_sigma_t0':pc_sigma_t0, 'pc_sigma_t1':pc_sigma_t1, 'pc_deltaTs_input':pc_deltaTs_input, 'pc_t0_input':pc_t0_input, 'pc_t1_input':pc_t1_input, 'fmt':'-b', 'figure_label':'d'}
                            # Set up options for plotting recent climate
                            rc_plotting_dict = {'rc_plot_input_option':'no', 'rc_deltaTs':rc_deltaTs, 'rc_t0':rc_t0, 'rc_t1':rc_t1, 'rc_deltaTs_cut':rc_deltaTs_cut, 'rc_t0_cut':rc_t0_cut, 'rc_t1_cut':rc_t1_cut, 'rc_deltaTs_smoothed_cut':rc_deltaTs_smoothed_cut, 'rc_t0_smoothing_cutoff_cut':rc_t0_smoothing_cutoff_cut, 'rc_t1_smoothing_cutoff_cut':rc_t1_smoothing_cutoff_cut, 'rc_deltaTs_input':rc_deltaTs_input, 'rc_t0_input':rc_t0_input, 'rc_t1_input':rc_t1_input, 'rc_ty':rc_ty, 'color':'blue','figure_label':'e'}
                            # Set up options for plotting heat-flow histogram
                            qhist_plotting_dict = {'number_hists':2,
                            'hist0':{'hist_type':'unweighted_histogram', 'set':whole_bullard_q_downward_TvR_mc_all, 'sigma_set':whole_bullard_sigma_q_downward_TvR_mc_all, 'color':'red', 'alpha':0, 'number_bins':50, 'figure_label':None},
                            'hist1':{'hist_type':'unweighted_histogram', 'set':whole_bullard_q_downward_TvR_cc_mc_all, 'sigma_set':whole_bullard_sigma_q_downward_TvR_cc_mc_all, 'color':'blue', 'alpha':0, 'number_bins':50, 'figure_label':'f'}}
                            plotting_functions.plot_temperature_stratigraphy_conductivity(max_depth_m_plot, k_distribution, plot_lith_fill_dict, plot_lith_fill_dict_keyword, strat_interp_lith_k_calcs_df, T_plotting_dict, k_plotting_dict, res_plotting_dict, bullard_TvR_plotting_dict, bullard_RvT_plotting_dict, pc_plotting_dict, rc_plotting_dict, qhist_plotting_dict, T_range, figure_name)
                            
                            
                           
                            
                            
                            ### Plot temperature, stratigraphy (if necessary) and conductivity after top values have been cut
                            # TODO Add cutting of conductivity near top of hole
                            figure_name = figures_path + '_b' + T_cut_suffix + k_cut_suffix + '_climate'
                            T_plotting_dict = {'number_lines':2,
                                                'line0':{'line_type':'errorbar', 'zT_m_plot':zT_m, 'T_plot':T, 'zT_error_m_plot':zT_error_m, 'T_error_plot':T_error, 'fmt':'.k', 'alpha':0.125, 'markeredgecolor':None, 'figure_label':None},
                                                'line1':{'line_type':'errorbar', 'zT_m_plot':zT_m_cut, 'T_plot':T_cut, 'zT_error_m_plot':zT_error_m_cut, 'T_error_plot':T_error_cut, 'fmt':'.k', 'alpha':1, 'markeredgecolor':None, 'figure_label':'a'}}
                            if k_distribution != 'in_situ_normal':
                                k_plotting_dict = {'number_lines':1, 'line0':{'line_type':'strat_interp_lith_k', 'z0_m':strat_interp_lith_k_calcs_df['z0_m'], 'z1_m':strat_interp_lith_k_calcs_df['z1_m'], 'min_k':strat_interp_lith_k_calcs_df['min_k'], 'max_k':strat_interp_lith_k_calcs_df['max_k'], 'mean_k':strat_interp_lith_k_calcs_df['mean_k'], 'k_assigned_error':strat_interp_lith_k_calcs_df['k_assigned_error'], 'color':'black', 'alpha':0.5, 'zorder':5, 'figure_label':'b'}}
                            else:
                                k_plotting_dict = {'number_lines':1, 'line0':{'line_type':'mean_errorbar', 'layer_zk_m_plot':layer_zmid_m, 'layer_k_plot':layer_mean_k, 'layer_zk_error_m_plot':layer_zmid_error_m, 'layer_k_error_plot':layer_mean_k_error, 'fmt':'.k', 'alpha':1, 'zorder':5, 'figure_label':'b'}}
                            # Set up options for plotting thermal resistivity
                            res_plotting_dict = {'number_lines':1, 'line0':{'line_type':'errorbar', 'zR_m_plot':zT_m_input, 'R_plot':R1_downward, 'zR_error_m_plot':zT_error_m_input, 'R_error_plot':sigma_R1_downward, 'fmt':'.r', 'alpha':0, 'markeredgecolor':None, 'figure_label':'c'}}
                            # Set up options for plotting Bullard plots - plot as empty frames
                            bullard_TvR_plotting_dict = {'number_lines':1, 'line0':{'line_type':'emptyframe', 'empty_x':1, 'empty_y':1, 'figure_label':'e'}}
                            bullard_RvT_plotting_dict = None
                            # Set up options for plotting palaeoclimate
                            pc_plotting_dict = {'pc_plot_input_option':'no', 'pc_deltaTs':pc_deltaTs, 'pc_sigma_deltaTs':pc_sigma_deltaTs, 'pc_t0':pc_t0, 'pc_t1':pc_t1, 'pc_sigma_t0':pc_sigma_t0, 'pc_sigma_t1':pc_sigma_t1, 'pc_deltaTs_input':pc_deltaTs_input, 'pc_t0_input':pc_t0_input, 'pc_t1_input':pc_t1_input, 'fmt':'-b', 'figure_label':'d'}
                            # Set up options for plotting recent climate
                            rc_plotting_dict = {'rc_plot_input_option':'no', 'rc_deltaTs':rc_deltaTs, 'rc_t0':rc_t0, 'rc_t1':rc_t1, 'rc_deltaTs_cut':rc_deltaTs_cut, 'rc_t0_cut':rc_t0_cut, 'rc_t1_cut':rc_t1_cut, 'rc_deltaTs_smoothed_cut':rc_deltaTs_smoothed_cut, 'rc_t0_smoothing_cutoff_cut':rc_t0_smoothing_cutoff_cut, 'rc_t1_smoothing_cutoff_cut':rc_t1_smoothing_cutoff_cut, 'rc_deltaTs_input':rc_deltaTs_input, 'rc_t0_input':rc_t0_input, 'rc_t1_input':rc_t1_input, 'rc_ty':rc_ty, 'color':'blue','figure_label':'e'}
                            # Set up options for plotting heat-flow histogram
                            qhist_plotting_dict = {'number_hists':2,                             
                            'hist0':{'hist_type':'unweighted_histogram', 'set':whole_bullard_q_downward_TvR_mc_all, 'sigma_set':whole_bullard_sigma_q_downward_TvR_mc_all, 'color':'red', 'alpha':0, 'number_bins':50, 'figure_label':None},                             
                            'hist1':{'hist_type':'unweighted_histogram', 'set':whole_bullard_q_downward_TvR_cc_mc_all, 'sigma_set':whole_bullard_sigma_q_downward_TvR_cc_mc_all, 'color':'blue', 'alpha':0, 'number_bins':50, 'figure_label':'f'}}
                            plotting_functions.plot_temperature_stratigraphy_conductivity(max_depth_m_plot, k_distribution, plot_lith_fill_dict, plot_lith_fill_dict_keyword, strat_interp_lith_k_calcs_df, T_plotting_dict, k_plotting_dict, res_plotting_dict, bullard_TvR_plotting_dict, bullard_RvT_plotting_dict, pc_plotting_dict, rc_plotting_dict, qhist_plotting_dict, T_range, figure_name)
                            
                            # Plot subsampled temperature 
                            if monte_carlo_T_subsample_factor != None:
                                figure_name = figures_path + '_c' + T_suffix_subsampled + k_suffix
                                T_plotting_dict = {'number_lines':2,
                                'line0':{'line_type':'errorbar', 'zT_m_plot':zT_m, 'T_plot':T, 'zT_error_m_plot':zT_error_m, 'T_error_plot':T_error, 'fmt':'.k', 'alpha':0.125, 'markeredgecolor':None, 'figure_label':None}, 
#                                 'line1':{'line_type':'errorbar', 'zT_m_plot':zT_m_cut, 'T_plot':T_cut, 'zT_error_m_plot':zT_error_m_cut, 'T_error_plot':T_error_cut, 'fmt':'.k', 'alpha':0.25, 'markeredgecolor':None},
                                'line1':{'line_type':'errorbar', 'zT_m_plot':zT_m_subsampled, 'T_plot':T_subsampled, 'zT_error_m_plot':zT_error_m_subsampled, 'T_error_plot':T_error_subsampled, 'fmt':'.k', 'alpha':1, 'markeredgecolor':None, 'figure_label':'a'}}
                                if k_distribution != 'in_situ_normal':
                                    k_plotting_dict = {'number_lines':1, 'line0':{'line_type':'strat_interp_lith_k', 'z0_m':strat_interp_lith_k_calcs_df['z0_m'], 'z1_m':strat_interp_lith_k_calcs_df['z1_m'], 'min_k':strat_interp_lith_k_calcs_df['min_k'], 'max_k':strat_interp_lith_k_calcs_df['max_k'], 'mean_k':strat_interp_lith_k_calcs_df['mean_k'], 'k_assigned_error':strat_interp_lith_k_calcs_df['k_assigned_error'], 'color':'black', 'alpha':0.5, 'zorder':5, 'figure_label':'b'}}
                                else:
                                    k_plotting_dict = {'number_lines':1, 'line0':{'line_type':'mean_errorbar', 'layer_zk_m_plot':layer_zmid_m, 'layer_k_plot':layer_mean_k, 'layer_zk_error_m_plot':layer_zmid_error_m, 'layer_k_error_plot':layer_mean_k_error, 'fmt':'.k', 'alpha':1, 'zorder':5, 'figure_label':'b'}}
                                # Set up options for plotting thermal resistivity
                                res_plotting_dict = {'number_lines':1, 'line0':{'line_type':'errorbar', 'zR_m_plot':zT_m_input, 'R_plot':R1_downward, 'zR_error_m_plot':zT_error_m_input, 'R_error_plot':sigma_R1_downward, 'fmt':'.r', 'alpha':0, 'markeredgecolor':None, 'figure_label':'c'}}
                                # Set up options for plotting Bullard plots - plot as empty frames
                                bullard_TvR_plotting_dict = {'number_lines':1, 'line0':{'line_type':'emptyframe', 'empty_x':1, 'empty_y':1, 'figure_label':'e'}}
                                bullard_RvT_plotting_dict = None
                                # Set up options for plotting palaeoclimate
                                pc_plotting_dict = {'pc_plot_input_option':'no', 'pc_deltaTs':pc_deltaTs, 'pc_sigma_deltaTs':pc_sigma_deltaTs, 'pc_t0':pc_t0, 'pc_t1':pc_t1, 'pc_sigma_t0':pc_sigma_t0, 'pc_sigma_t1':pc_sigma_t1, 'pc_deltaTs_input':pc_deltaTs_input, 'pc_t0_input':pc_t0_input, 'pc_t1_input':pc_t1_input, 'fmt':'-b', 'figure_label':'d'}
                                # Set up options for plotting recent climate
                                rc_plotting_dict = {'rc_plot_input_option':'no', 'rc_deltaTs':rc_deltaTs, 'rc_t0':rc_t0, 'rc_t1':rc_t1, 'rc_deltaTs_cut':rc_deltaTs_cut, 'rc_t0_cut':rc_t0_cut, 'rc_t1_cut':rc_t1_cut, 'rc_deltaTs_smoothed_cut':rc_deltaTs_smoothed_cut, 'rc_t0_smoothing_cutoff_cut':rc_t0_smoothing_cutoff_cut, 'rc_t1_smoothing_cutoff_cut':rc_t1_smoothing_cutoff_cut, 'rc_deltaTs_input':rc_deltaTs_input, 'rc_t0_input':rc_t0_input, 'rc_t1_input':rc_t1_input, 'rc_ty':rc_ty, 'color':'blue','figure_label':'e'}
                                # Set up options for plotting heat-flow histogram
                                qhist_plotting_dict = {'number_hists':2,                             
                                'hist0':{'hist_type':'unweighted_histogram', 'set':whole_bullard_q_downward_TvR_mc_all, 'sigma_set':whole_bullard_sigma_q_downward_TvR_mc_all, 'color':'red', 'alpha':0, 'number_bins':50, 'figure_label':None},                             
                                'hist1':{'hist_type':'unweighted_histogram', 'set':whole_bullard_q_downward_TvR_cc_mc_all, 'sigma_set':whole_bullard_sigma_q_downward_TvR_cc_mc_all, 'color':'blue', 'alpha':0, 'number_bins':50, 'figure_label':'f'}}
                                plotting_functions.plot_temperature_stratigraphy_conductivity(max_depth_m_plot, k_distribution, plot_lith_fill_dict, plot_lith_fill_dict_keyword, strat_interp_lith_k_calcs_df, T_plotting_dict, k_plotting_dict, res_plotting_dict, bullard_TvR_plotting_dict, bullard_RvT_plotting_dict, pc_plotting_dict, rc_plotting_dict, qhist_plotting_dict, T_range, figure_name)
                            
                            # Plot perturbed temperature
                            figure_name = figures_path + '_d' + '_jsim' + str(jsim) + T_suffix_mc + k_suffix
                            T_plotting_dict = {'number_lines':3, 
                            'line0':{'line_type':'errorbar', 'zT_m_plot':zT_m, 'T_plot':T, 'zT_error_m_plot':zT_error_m, 'T_error_plot':T_error, 'fmt':'.k', 'alpha':0.125, 'markeredgecolor':None, 'figure_label':None}, 
                            'line1':{'line_type':'errorbar', 'zT_m_plot':zT_m_subsampled, 'T_plot':T_subsampled, 'zT_error_m_plot':zT_error_m_subsampled, 'T_error_plot':T_error_subsampled, 'fmt':'.k', 'alpha':0.125, 'markeredgecolor':None, 'figure_label':None},
                            'line2':{'line_type':'errorbar', 'zT_m_plot':zT_m_input, 'T_plot':T_input, 'zT_error_m_plot':zT_error_m_input, 'T_error_plot':T_error_input, 'fmt':'.r', 'alpha':1, 'markeredgecolor':None, 'figure_label':'a'}}
                            if k_distribution != 'in_situ_normal':
                                k_plotting_dict = {'number_lines':1, 'line0':{'line_type':'strat_interp_lith_k', 'z0_m':strat_interp_lith_k_calcs_df['z0_m'], 'z1_m':strat_interp_lith_k_calcs_df['z1_m'], 'min_k':strat_interp_lith_k_calcs_df['min_k'], 'max_k':strat_interp_lith_k_calcs_df['max_k'], 'mean_k':strat_interp_lith_k_calcs_df['mean_k'], 'k_assigned_error':strat_interp_lith_k_calcs_df['k_assigned_error'], 'color':'black', 'alpha':0.5, 'zorder':5, 'figure_label':'b'}}
                            else:
                                k_plotting_dict = {'number_lines':1, 'line0':{'line_type':'mean_errorbar', 'layer_zk_m_plot':layer_zmid_m, 'layer_k_plot':layer_mean_k, 'layer_zk_error_m_plot':layer_zmid_error_m, 'layer_k_error_plot':layer_mean_k_error, 'fmt':'.k', 'alpha':1, 'zorder':5, 'figure_label':'b'}}
                            # Set up options for plotting thermal resistivity
                            res_plotting_dict = {'number_lines':1, 'line0':{'line_type':'errorbar', 'zR_m_plot':zT_m_input, 'R_plot':R1_downward, 'zR_error_m_plot':zT_error_m_input, 'R_error_plot':sigma_R1_downward, 'fmt':'.r', 'alpha':0, 'markeredgecolor':None, 'figure_label':'c'}}
                            # Set up options for plotting Bullard plots - plot as empty frames
                            bullard_TvR_plotting_dict = {'number_lines':1, 'line0':{'line_type':'emptyframe', 'empty_x':1, 'empty_y':1, 'figure_label':'e'}}
                            bullard_RvT_plotting_dict = None
                            # Set up options for plotting palaeoclimate
                            pc_plotting_dict = {'pc_plot_input_option':'no', 'pc_deltaTs':pc_deltaTs, 'pc_sigma_deltaTs':pc_sigma_deltaTs, 'pc_t0':pc_t0, 'pc_t1':pc_t1, 'pc_sigma_t0':pc_sigma_t0, 'pc_sigma_t1':pc_sigma_t1, 'pc_deltaTs_input':pc_deltaTs_input, 'pc_t0_input':pc_t0_input, 'pc_t1_input':pc_t1_input, 'fmt':'-b', 'figure_label':'d'}
                            # Set up options for plotting recent climate
                            rc_plotting_dict = {'rc_plot_input_option':'no', 'rc_deltaTs':rc_deltaTs, 'rc_t0':rc_t0, 'rc_t1':rc_t1, 'rc_deltaTs_cut':rc_deltaTs_cut, 'rc_t0_cut':rc_t0_cut, 'rc_t1_cut':rc_t1_cut, 'rc_deltaTs_smoothed_cut':rc_deltaTs_smoothed_cut, 'rc_t0_smoothing_cutoff_cut':rc_t0_smoothing_cutoff_cut, 'rc_t1_smoothing_cutoff_cut':rc_t1_smoothing_cutoff_cut, 'rc_deltaTs_input':rc_deltaTs_input, 'rc_t0_input':rc_t0_input, 'rc_t1_input':rc_t1_input, 'rc_ty':rc_ty, 'color':'blue','figure_label':'e'}
                            # Set up options for plotting heat-flow histogram
                            qhist_plotting_dict = {'number_hists':2,                             
                            'hist0':{'hist_type':'unweighted_histogram', 'set':whole_bullard_q_downward_TvR_mc_all, 'sigma_set':whole_bullard_sigma_q_downward_TvR_mc_all, 'color':'red', 'alpha':0, 'number_bins':50, 'figure_label':None},                             
                            'hist1':{'hist_type':'unweighted_histogram', 'set':whole_bullard_q_downward_TvR_cc_mc_all, 'sigma_set':whole_bullard_sigma_q_downward_TvR_cc_mc_all, 'color':'blue', 'alpha':0, 'number_bins':50, 'figure_label':'f'}}
                            plotting_functions.plot_temperature_stratigraphy_conductivity(max_depth_m_plot, k_distribution, plot_lith_fill_dict, plot_lith_fill_dict_keyword, strat_interp_lith_k_calcs_df, T_plotting_dict, k_plotting_dict, res_plotting_dict, bullard_TvR_plotting_dict, bullard_RvT_plotting_dict, pc_plotting_dict, rc_plotting_dict, qhist_plotting_dict, T_range, figure_name)
                            
                            # Plot perturbed temperature and subsampled conductivity (if in situ; otherwise plot conductivity distribution)
                            if monte_carlo_in_situ_k_subsample_factor != None:
                                figure_name = figures_path + '_e' + '_jsim' + str(jsim) + T_suffix_mc + k_suffix_subsampled
                                T_plotting_dict = {'number_lines':3, 
                            'line0':{'line_type':'errorbar', 'zT_m_plot':zT_m, 'T_plot':T, 'zT_error_m_plot':zT_error_m, 'T_error_plot':T_error, 'fmt':'.k', 'alpha':0.125, 'markeredgecolor':None, 'figure_label':None}, 
                            'line1':{'line_type':'errorbar', 'zT_m_plot':zT_m_subsampled, 'T_plot':T_subsampled, 'zT_error_m_plot':zT_error_m_subsampled, 'T_error_plot':T_error_subsampled, 'fmt':'.k', 'alpha':0.125, 'markeredgecolor':None, 'figure_label':None},
                            'line2':{'line_type':'errorbar', 'zT_m_plot':zT_m_input, 'T_plot':T_input, 'zT_error_m_plot':zT_error_m_input, 'T_error_plot':T_error_input, 'fmt':'.r', 'alpha':1, 'markeredgecolor':None, 'figure_label':'a'}}
                                if k_distribution == 'in_situ_normal':
                                    k_plotting_dict = {'number_lines':2,
                                    'line0':{'line_type':'mean_errorbar', 'layer_zk_m_plot':layer_zmid_m, 'layer_k_plot':layer_mean_k, 'layer_zk_error_m_plot':layer_zmid_error_m, 'layer_k_error_plot':layer_mean_k_error, 'fmt':'.k', 'alpha':0.125, 'zorder':5, 'figure_label':None},
                                    'line1':{'line_type':'mean_errorbar', 'layer_zk_m_plot':layer_zmid_m_subsampled, 'layer_k_plot':layer_mean_k_subsampled, 'layer_zk_error_m_plot':layer_zmid_error_m_subsampled, 'layer_k_error_plot':layer_mean_k_error_subsampled, 'fmt':'.k', 'alpha':1, 'markeredgecolor':None, 'zorder':6, 'figure_label':'b'}}
                                else:
                                     k_plotting_dict = {'number_lines':1, 'line0':{'line_type':'strat_interp_lith_k', 'z0_m':strat_interp_lith_k_calcs_df['z0_m'], 'z1_m':strat_interp_lith_k_calcs_df['z1_m'], 'min_k':strat_interp_lith_k_calcs_df['min_k'], 'max_k':strat_interp_lith_k_calcs_df['max_k'], 'mean_k':strat_interp_lith_k_calcs_df['mean_k'], 'k_assigned_error':strat_interp_lith_k_calcs_df['k_assigned_error'], 'color':'black', 'alpha':0.5, 'zorder':5, 'figure_label':'b'}}
                                # Set up options for plotting thermal resistivity
                                res_plotting_dict = {'number_lines':1, 'line0':{'line_type':'errorbar', 'zR_m_plot':zT_m_input, 'R_plot':R1_downward, 'zR_error_m_plot':zT_error_m_input, 'R_error_plot':sigma_R1_downward, 'fmt':'.r', 'alpha':0, 'markeredgecolor':None, 'figure_label':'c'}}
                                # Set up options for plotting Bullard plots - plot as empty frames
                                bullard_TvR_plotting_dict = {'number_lines':1, 'line0':{'line_type':'emptyframe', 'empty_x':1, 'empty_y':1, 'figure_label':'e'}}
                                bullard_RvT_plotting_dict = None
                                # Set up options for plotting palaeoclimate
                                pc_plotting_dict = {'pc_plot_input_option':'no', 'pc_deltaTs':pc_deltaTs, 'pc_sigma_deltaTs':pc_sigma_deltaTs, 'pc_t0':pc_t0, 'pc_t1':pc_t1, 'pc_sigma_t0':pc_sigma_t0, 'pc_sigma_t1':pc_sigma_t1, 'pc_deltaTs_input':pc_deltaTs_input, 'pc_t0_input':pc_t0_input, 'pc_t1_input':pc_t1_input, 'fmt':'-b', 'figure_label':'d'}
                                # Set up options for plotting recent climate
                                rc_plotting_dict = {'rc_plot_input_option':'no', 'rc_deltaTs':rc_deltaTs, 'rc_t0':rc_t0, 'rc_t1':rc_t1, 'rc_deltaTs_cut':rc_deltaTs_cut, 'rc_t0_cut':rc_t0_cut, 'rc_t1_cut':rc_t1_cut, 'rc_deltaTs_smoothed_cut':rc_deltaTs_smoothed_cut, 'rc_t0_smoothing_cutoff_cut':rc_t0_smoothing_cutoff_cut, 'rc_t1_smoothing_cutoff_cut':rc_t1_smoothing_cutoff_cut, 'rc_deltaTs_input':rc_deltaTs_input, 'rc_t0_input':rc_t0_input, 'rc_t1_input':rc_t1_input, 'rc_ty':rc_ty, 'color':'blue','figure_label':'e'}
                                # Set up options for plotting heat-flow histogram
                                qhist_plotting_dict = {'number_hists':2,                             
                                'hist0':{'hist_type':'unweighted_histogram', 'set':whole_bullard_q_downward_TvR_mc_all, 'sigma_set':whole_bullard_sigma_q_downward_TvR_mc_all, 'color':'red', 'alpha':0, 'number_bins':50, 'figure_label':None},                             
                                'hist1':{'hist_type':'unweighted_histogram', 'set':whole_bullard_q_downward_TvR_cc_mc_all, 'sigma_set':whole_bullard_sigma_q_downward_TvR_cc_mc_all, 'color':'blue', 'alpha':0, 'number_bins':50, 'figure_label':'f'}}
                                plotting_functions.plot_temperature_stratigraphy_conductivity(max_depth_m_plot, k_distribution, plot_lith_fill_dict, plot_lith_fill_dict_keyword, strat_interp_lith_k_calcs_df, T_plotting_dict, k_plotting_dict, res_plotting_dict, bullard_TvR_plotting_dict, bullard_RvT_plotting_dict, pc_plotting_dict, rc_plotting_dict, qhist_plotting_dict, T_range, figure_name)
                    
                            

                            ### Plot in situ conductivity measurements divided into layers, with measurement points at centre of each layer
#                             if k_distribution == 'in_situ_normal':
#                                 figure_name = figures_path + '_jsim' + str(jsim) + T_suffix_mc + k_suffix_subsampled + '_layers'
#                                 T_plotting_dict = {'number_lines':3, 
#                             'line0':{'line_type':'errorbar', 'zT_m_plot':zT_m, 'T_plot':T, 'zT_error_m_plot':zT_error_m, 'T_error_plot':T_error, 'fmt':'.k', 'alpha':0.125, 'markeredgecolor':None, 'figure_label':None}, 
#                             'line1':{'line_type':'errorbar', 'zT_m_plot':zT_m_subsampled, 'T_plot':T_subsampled, 'zT_error_m_plot':zT_error_m_subsampled, 'T_error_plot':T_error_subsampled, 'fmt':'.k', 'alpha':0.125, 'markeredgecolor':None, 'figure_label':None},
#                             'line2':{'line_type':'errorbar', 'zT_m_plot':zT_m_input, 'T_plot':T_input, 'zT_error_m_plot':zT_error_m_input, 'T_error_plot':T_error_input, 'fmt':'.r', 'alpha':1, 'markeredgecolor':None, 'figure_label':'a'}}
#                                 k_plotting_dict = {'number_lines':2,
#                                 'line0':{'line_type':'strat_interp_lith_k', 'z0_m':layer_z0_m_subsampled, 'z1_m':layer_z1_m_subsampled, 'min_k':layer_min_k_subsampled, 'max_k':layer_max_k_subsampled, 'mean_k':layer_mean_k_subsampled, 'k_assigned_error':layer_mean_k_error_subsampled, 'color':'black', 'alpha':1, 'zorder':5, 'figure_label':None},
#                                 'line1':{'line_type':'mean_errorbar', 'layer_zk_m_plot':layer_zmid_m_subsampled, 'layer_k_plot':layer_mean_k_subsampled, 'layer_zk_error_m_plot':layer_zmid_error_m_subsampled, 'layer_k_error_plot':layer_mean_k_error_subsampled, 'fmt':'.k', 'alpha':0.5, 'zorder':5, 'figure_label':'b'}} 
#                                 # Set up options for plotting thermal resistivity
#                                 res_plotting_dict = {'number_lines':1, 'line0':{'line_type':'errorbar', 'zR_m_plot':zT_m_input, 'R_plot':R1_downward, 'zR_error_m_plot':zT_error_m_input, 'R_error_plot':sigma_R1_downward, 'fmt':'.r', 'alpha':0, 'markeredgecolor':None, 'figure_label':'c'}}
#                                 # Set up options for plotting Bullard plots - plot as empty frames
#                                 bullard_TvR_plotting_dict = {'number_lines':1, 'line0':{'line_type':'emptyframe', 'empty_x':1, 'empty_y':1, 'figure_label':'e'}}
#                                 bullard_RvT_plotting_dict = None
#                                 # Set up options for plotting palaeoclimate
#                                 pc_plotting_dict = {'pc_plot_input_option':'no', 'pc_deltaTs':pc_deltaTs, 'pc_sigma_deltaTs':pc_sigma_deltaTs, 'pc_t0':pc_t0, 'pc_t1':pc_t1, 'pc_sigma_t0':pc_sigma_t0, 'pc_sigma_t1':pc_sigma_t1, 'pc_deltaTs_input':pc_deltaTs_input, 'pc_t0_input':pc_t0_input, 'pc_t1_input':pc_t1_input, 'fmt':'-b', 'figure_label':'d'}
#                                 # Set up options for plotting recent climate
#                                 rc_plotting_dict = {'rc_plot_input_option':'no', 'rc_deltaTs':rc_deltaTs, 'rc_t0':rc_t0, 'rc_t1':rc_t1, 'rc_deltaTs_cut':rc_deltaTs_cut, 'rc_t0_cut':rc_t0_cut, 'rc_t1_cut':rc_t1_cut, 'rc_deltaTs_smoothed_cut':rc_deltaTs_smoothed_cut, 'rc_t0_smoothing_cutoff_cut':rc_t0_smoothing_cutoff_cut, 'rc_t1_smoothing_cutoff_cut':rc_t1_smoothing_cutoff_cut, 'rc_deltaTs_input':rc_deltaTs_input, 'rc_t0_input':rc_t0_input, 'rc_t1_input':rc_t1_input, 'rc_ty':rc_ty, 'color':'blue','figure_label':'e'}
#                                 # Set up options for plotting heat-flow histogram
#                                 qhist_plotting_dict = {'number_hists':2,                             
#                                 'hist0':{'hist_type':'unweighted_histogram', 'set':whole_bullard_q_downward_TvR_mc_all, 'sigma_set':whole_bullard_sigma_q_downward_TvR_mc_all, 'color':'red', 'alpha':0, 'number_bins':50, 'figure_label':None},                             
#                                 'hist1':{'hist_type':'unweighted_histogram', 'set':whole_bullard_q_downward_TvR_cc_mc_all, 'sigma_set':whole_bullard_sigma_q_downward_TvR_cc_mc_all, 'color':'blue', 'alpha':0, 'number_bins':50, 'figure_label':'f'}}
#                                 plotting_functions.plot_temperature_stratigraphy_conductivity(max_depth_m_plot, k_distribution, plot_lith_fill_dict, plot_lith_fill_dict_keyword, strat_interp_lith_k_calcs_df, T_plotting_dict, k_plotting_dict, res_plotting_dict, bullard_TvR_plotting_dict, bullard_RvT_plotting_dict, pc_plotting_dict, rc_plotting_dict, qhist_plotting_dict, T_range, figure_name)

                            
                                
                            # Plot perturbed temperature and perturbed conductivity
                            figure_name = figures_path + '_f' + '_jsim' + str(jsim) + T_suffix_mc + k_suffix_mc
                            T_plotting_dict = {'number_lines':3, 
                            'line0':{'line_type':'errorbar', 'zT_m_plot':zT_m, 'T_plot':T, 'zT_error_m_plot':zT_error_m, 'T_error_plot':T_error, 'fmt':'.k', 'alpha':0.125, 'markeredgecolor':None, 'figure_label':None}, 
                            'line1':{'line_type':'errorbar', 'zT_m_plot':zT_m_subsampled, 'T_plot':T_subsampled, 'zT_error_m_plot':zT_error_m_subsampled, 'T_error_plot':T_error_subsampled, 'fmt':'.k', 'alpha':0.125, 'markeredgecolor':None, 'figure_label':None},
                            'line2':{'line_type':'errorbar', 'zT_m_plot':zT_m_input, 'T_plot':T_input, 'zT_error_m_plot':zT_error_m_input, 'T_error_plot':T_error_input, 'fmt':'.r', 'alpha':1, 'markeredgecolor':None, 'figure_label':'a'}}
                            if k_distribution != 'in_situ_normal':
                                k_plotting_dict = {'number_lines':2, 
                                'line0':{'line_type':'strat_interp_lith_k', 'z0_m':layer_z0_m_subsampled, 'z1_m':layer_z1_m_subsampled, 'min_k':layer_min_k_subsampled, 'max_k':layer_max_k_subsampled, 'mean_k':layer_mean_k_subsampled, 'k_assigned_error':layer_mean_k_error_subsampled, 'color':'black', 'alpha':0.125, 'zorder':5, 'figure_label':None},
                                'line1':{'line_type':'mc_input', 'layer_zk_m_input_plot':layer_zk_m_input_plot, 'layer_k_input_plot':layer_k_input_plot, 'color':'black', 'alpha':1, 'zorder':6, 'figure_label':'b'}}
                            else:
                                k_plotting_dict = {'number_lines':2,
                                'line0':{'line_type':'mean_errorbar', 'layer_zk_m_plot':layer_zmid_m, 'layer_k_plot':layer_mean_k, 'layer_zk_error_m_plot':layer_zmid_error_m, 'layer_k_error_plot':layer_mean_k_error, 'fmt':'.k', 'alpha':0.125, 'zorder':5, 'figure_label':'b'},
                                'line1':{'line_type':'mc_input', 'layer_zk_m_input_plot':layer_zk_m_input_plot, 'layer_k_input_plot':layer_k_input_plot, 'color':'black', 'alpha':1, 'zorder':6, 'figure_label':'b'}}
                            # Set up options for plotting thermal resistivity
                            res_plotting_dict = {'number_lines':1, 'line0':{'line_type':'errorbar', 'zR_m_plot':zT_m_input, 'R_plot':R1_downward, 'zR_error_m_plot':zT_error_m_input, 'R_error_plot':sigma_R1_downward, 'fmt':'.r', 'alpha':0, 'markeredgecolor':None, 'figure_label':'c'}}
                            # Set up options for plotting Bullard plots - plot as empty frames
                            bullard_TvR_plotting_dict = {'number_lines':1, 'line0':{'line_type':'emptyframe', 'empty_x':1, 'empty_y':1, 'figure_label':'e'}}
                            bullard_RvT_plotting_dict = None
                            # Set up options for plotting palaeoclimate
                            pc_plotting_dict = {'pc_plot_input_option':'no', 'pc_deltaTs':pc_deltaTs, 'pc_sigma_deltaTs':pc_sigma_deltaTs, 'pc_t0':pc_t0, 'pc_t1':pc_t1, 'pc_sigma_t0':pc_sigma_t0, 'pc_sigma_t1':pc_sigma_t1, 'pc_deltaTs_input':pc_deltaTs_input, 'pc_t0_input':pc_t0_input, 'pc_t1_input':pc_t1_input, 'fmt':'-b', 'figure_label':'d'}
                            # Set up options for plotting recent climate
                            rc_plotting_dict = {'rc_plot_input_option':'no', 'rc_deltaTs':rc_deltaTs, 'rc_t0':rc_t0, 'rc_t1':rc_t1, 'rc_deltaTs_cut':rc_deltaTs_cut, 'rc_t0_cut':rc_t0_cut, 'rc_t1_cut':rc_t1_cut, 'rc_deltaTs_smoothed_cut':rc_deltaTs_smoothed_cut, 'rc_t0_smoothing_cutoff_cut':rc_t0_smoothing_cutoff_cut, 'rc_t1_smoothing_cutoff_cut':rc_t1_smoothing_cutoff_cut, 'rc_deltaTs_input':rc_deltaTs_input, 'rc_t0_input':rc_t0_input, 'rc_t1_input':rc_t1_input, 'rc_ty':rc_ty, 'color':'blue','figure_label':'e'}
                            # Set up options for plotting heat-flow histogram
                            qhist_plotting_dict = {'number_hists':2,                             
                            'hist0':{'hist_type':'unweighted_histogram', 'set':whole_bullard_q_downward_TvR_mc_all, 'sigma_set':whole_bullard_sigma_q_downward_TvR_mc_all, 'color':'red', 'alpha':0, 'number_bins':50, 'figure_label':None},                             
                            'hist1':{'hist_type':'unweighted_histogram', 'set':whole_bullard_q_downward_TvR_cc_mc_all, 'sigma_set':whole_bullard_sigma_q_downward_TvR_cc_mc_all, 'color':'blue', 'alpha':0, 'number_bins':50, 'figure_label':'f'}}
                            plotting_functions.plot_temperature_stratigraphy_conductivity(max_depth_m_plot, k_distribution, plot_lith_fill_dict, plot_lith_fill_dict_keyword, strat_interp_lith_k_calcs_df, T_plotting_dict, k_plotting_dict, res_plotting_dict, bullard_TvR_plotting_dict, bullard_RvT_plotting_dict, pc_plotting_dict, rc_plotting_dict, qhist_plotting_dict, T_range, figure_name)
                            
                            
                            
                            
                            # Plot perturbed temperature and conductivity and resulting thermal resistivity
                            figure_name = figures_path + '_g' + '_jsim' + str(jsim) + T_suffix_mc + k_suffix_mc + '_res'
                            T_plotting_dict = {'number_lines':3, 
                            'line0':{'line_type':'errorbar', 'zT_m_plot':zT_m, 'T_plot':T, 'zT_error_m_plot':zT_error_m, 'T_error_plot':T_error, 'fmt':'.k', 'alpha':0.125, 'markeredgecolor':None, 'figure_label':None}, 
                            'line1':{'line_type':'errorbar', 'zT_m_plot':zT_m_subsampled, 'T_plot':T_subsampled, 'zT_error_m_plot':zT_error_m_subsampled, 'T_error_plot':T_error_subsampled, 'fmt':'.k', 'alpha':0.5, 'markeredgecolor':None, 'figure_label':None},
                            'line2':{'line_type':'errorbar', 'zT_m_plot':zT_m_input, 'T_plot':T_input, 'zT_error_m_plot':zT_error_m_input, 'T_error_plot':T_error_input, 'fmt':'.r', 'alpha':1, 'markeredgecolor':None, 'figure_label':'a'}}
                            if k_distribution != 'in_situ_normal':
                                k_plotting_dict = {'number_lines':2, 
                                'line0':{'line_type':'strat_interp_lith_k', 'z0_m':layer_z0_m_subsampled, 'z1_m':layer_z1_m_subsampled, 'min_k':layer_min_k_subsampled, 'max_k':layer_max_k_subsampled, 'mean_k':layer_mean_k_subsampled, 'k_assigned_error':layer_mean_k_error_subsampled, 'color':'black', 'alpha':0.125, 'zorder':5, 'figure_label':None},
                                'line1':{'line_type':'mc_input', 'layer_zk_m_input_plot':layer_zk_m_input_plot, 'layer_k_input_plot':layer_k_input_plot, 'color':'black', 'alpha':1, 'zorder':6, 'figure_label':'b'}}
                            else:
                                k_plotting_dict = {'number_lines':2,
                                'line0':{'line_type':'mean_errorbar', 'layer_zk_m_plot':layer_zmid_m, 'layer_k_plot':layer_mean_k, 'layer_zk_error_m_plot':layer_zmid_error_m, 'layer_k_error_plot':layer_mean_k_error, 'fmt':'.k', 'alpha':0.125, 'zorder':5, 'figure_label':'b'},
                                'line1':{'line_type':'mc_input', 'layer_zk_m_input_plot':layer_zk_m_input_plot, 'layer_k_input_plot':layer_k_input_plot, 'color':'black', 'alpha':1, 'zorder':6, 'figure_label':'b'}}
                            # Set up options for plotting thermal resistivity
                            res_plotting_dict = {'number_lines':1,
                            'line0':{'line_type':'errorbar', 'zR_m_plot':zT_m_input, 'R_plot':R1_downward, 'zR_error_m_plot':zT_error_m_input, 'R_error_plot':sigma_R1_downward, 'fmt':'.k', 'alpha':1, 'markeredgecolor':None, 'figure_label':'c'}}
                                                 #'line1':{'line_type':'errorbar', 'zR_m_plot':np.flip(zT_m_input), 'R_plot':np.max(R1_downward)-1*R1_upward, 'zR_error_m_plot':np.flip(zT_error_m_input), 'R_error_plot':sigma_R1_upward, 'fmt':'.g', 'alpha':1, 'markeredgecolor':None, 'figure_label':'c'}                   }
                            # Set up options for plotting Bullard plots - plot as empty frames
                            bullard_TvR_plotting_dict = {'number_lines':1, 'line0':{'line_type':'emptyframe', 'empty_x':1, 'empty_y':1, 'figure_label':'e'}}
                            bullard_RvT_plotting_dict = None
                            # Set up options for plotting palaeoclimate
                            pc_plotting_dict = {'pc_plot_input_option':'no', 'pc_deltaTs':pc_deltaTs, 'pc_sigma_deltaTs':pc_sigma_deltaTs, 'pc_t0':pc_t0, 'pc_t1':pc_t1, 'pc_sigma_t0':pc_sigma_t0, 'pc_sigma_t1':pc_sigma_t1, 'pc_deltaTs_input':pc_deltaTs_input, 'pc_t0_input':pc_t0_input, 'pc_t1_input':pc_t1_input, 'fmt':'-b', 'figure_label':'d'}
                            # Set up options for plotting recent climate
                            rc_plotting_dict = {'rc_plot_input_option':'no', 'rc_deltaTs':rc_deltaTs, 'rc_t0':rc_t0, 'rc_t1':rc_t1, 'rc_deltaTs_cut':rc_deltaTs_cut, 'rc_t0_cut':rc_t0_cut, 'rc_t1_cut':rc_t1_cut, 'rc_deltaTs_smoothed_cut':rc_deltaTs_smoothed_cut, 'rc_t0_smoothing_cutoff_cut':rc_t0_smoothing_cutoff_cut, 'rc_t1_smoothing_cutoff_cut':rc_t1_smoothing_cutoff_cut, 'rc_deltaTs_input':rc_deltaTs_input, 'rc_t0_input':rc_t0_input, 'rc_t1_input':rc_t1_input, 'rc_ty':rc_ty, 'color':'blue','figure_label':'e'}
                            # Set up options for plotting heat-flow histogram
                            qhist_plotting_dict = {'number_hists':2,                             
                            'hist0':{'hist_type':'unweighted_histogram', 'set':whole_bullard_q_downward_TvR_mc_all, 'sigma_set':whole_bullard_sigma_q_downward_TvR_mc_all, 'color':'red', 'alpha':0, 'number_bins':50, 'figure_label':None},                             
                            'hist1':{'hist_type':'unweighted_histogram', 'set':whole_bullard_q_downward_TvR_cc_mc_all, 'sigma_set':whole_bullard_sigma_q_downward_TvR_cc_mc_all, 'color':'blue', 'alpha':0, 'number_bins':50, 'figure_label':'f'}}
                            plotting_functions.plot_temperature_stratigraphy_conductivity(max_depth_m_plot, k_distribution, plot_lith_fill_dict, plot_lith_fill_dict_keyword, strat_interp_lith_k_calcs_df, T_plotting_dict, k_plotting_dict, res_plotting_dict, bullard_TvR_plotting_dict, bullard_RvT_plotting_dict, pc_plotting_dict, rc_plotting_dict, qhist_plotting_dict, T_range, figure_name)
                            
                            # Plot perturbed temperature and conductivity and resulting thermal resistivity, and perturbed palaeoclimate
                            figure_name = figures_path + '_h' + '_jsim' + str(jsim) + T_suffix_mc + k_suffix_mc + '_res' + pc_suffix_mc
                            T_plotting_dict = {'number_lines':3, 
                            'line0':{'line_type':'errorbar', 'zT_m_plot':zT_m, 'T_plot':T, 'zT_error_m_plot':zT_error_m, 'T_error_plot':T_error, 'fmt':'.k', 'alpha':0.125, 'markeredgecolor':None, 'figure_label':None}, 
                            'line1':{'line_type':'errorbar', 'zT_m_plot':zT_m_subsampled, 'T_plot':T_subsampled, 'zT_error_m_plot':zT_error_m_subsampled, 'T_error_plot':T_error_subsampled, 'fmt':'.k', 'alpha':0.5, 'markeredgecolor':None, 'figure_label':None},
                            'line2':{'line_type':'errorbar', 'zT_m_plot':zT_m_input, 'T_plot':T_input, 'zT_error_m_plot':zT_error_m_input, 'T_error_plot':T_error_input, 'fmt':'.r', 'alpha':1, 'markeredgecolor':None, 'figure_label':'a'}}
                            if k_distribution != 'in_situ_normal':
                                k_plotting_dict = {'number_lines':2, 
                                'line0':{'line_type':'strat_interp_lith_k', 'z0_m':layer_z0_m_subsampled, 'z1_m':layer_z1_m_subsampled, 'min_k':layer_min_k_subsampled, 'max_k':layer_max_k_subsampled, 'mean_k':layer_mean_k_subsampled, 'k_assigned_error':layer_mean_k_error_subsampled, 'color':'black', 'alpha':0.125, 'zorder':5, 'figure_label':None},
                                'line1':{'line_type':'mc_input', 'layer_zk_m_input_plot':layer_zk_m_input_plot, 'layer_k_input_plot':layer_k_input_plot, 'color':'black', 'alpha':1, 'zorder':6, 'figure_label':'b'}}
                            else:
                                k_plotting_dict = {'number_lines':2,
                                'line0':{'line_type':'mean_errorbar', 'layer_zk_m_plot':layer_zmid_m, 'layer_k_plot':layer_mean_k, 'layer_zk_error_m_plot':layer_zmid_error_m, 'layer_k_error_plot':layer_mean_k_error, 'fmt':'.k', 'alpha':0.125, 'zorder':5, 'figure_label':'b'},
                                'line1':{'line_type':'mc_input', 'layer_zk_m_input_plot':layer_zk_m_input_plot, 'layer_k_input_plot':layer_k_input_plot, 'color':'black', 'alpha':1, 'zorder':6, 'figure_label':'b'}}
                            # Set up options for plotting thermal resistivity
                            res_plotting_dict = {'number_lines':1, 'line0':{'line_type':'errorbar', 'zR_m_plot':zT_m_input, 'R_plot':R1_downward, 'zR_error_m_plot':zT_error_m_input, 'R_error_plot':sigma_R1_downward, 'fmt':'.k', 'alpha':1, 'markeredgecolor':None, 'figure_label':'c'}}
                            # Set up options for plotting Bullard plots - plot as empty frames
                            bullard_TvR_plotting_dict = {'number_lines':1, 'line0':{'line_type':'emptyframe', 'empty_x':1, 'empty_y':1, 'figure_label':'e'}}
                            bullard_RvT_plotting_dict = None
                            # Set up options for plotting palaeoclimate
                            pc_plotting_dict = {'pc_plot_input_option':'yes', 'pc_deltaTs':pc_deltaTs, 'pc_sigma_deltaTs':pc_sigma_deltaTs, 'pc_t0':pc_t0, 'pc_t1':pc_t1, 'pc_sigma_t0':pc_sigma_t0, 'pc_sigma_t1':pc_sigma_t1, 'pc_deltaTs_input':pc_deltaTs_input, 'pc_t0_input':pc_t0_input, 'pc_t1_input':pc_t1_input, 'fmt':'-b', 'figure_label':'d'}
                            # Set up options for plotting recent climate
                            rc_plotting_dict = {'rc_plot_input_option':'no', 'rc_deltaTs':rc_deltaTs, 'rc_t0':rc_t0, 'rc_t1':rc_t1, 'rc_deltaTs_cut':rc_deltaTs_cut, 'rc_t0_cut':rc_t0_cut, 'rc_t1_cut':rc_t1_cut, 'rc_deltaTs_smoothed_cut':rc_deltaTs_smoothed_cut, 'rc_t0_smoothing_cutoff_cut':rc_t0_smoothing_cutoff_cut, 'rc_t1_smoothing_cutoff_cut':rc_t1_smoothing_cutoff_cut, 'rc_deltaTs_input':rc_deltaTs_input, 'rc_t0_input':rc_t0_input, 'rc_t1_input':rc_t1_input, 'rc_ty':rc_ty, 'color':'blue','figure_label':'e'}
                            # Set up options for plotting heat-flow histogram
                            qhist_plotting_dict = {'number_hists':2,                             
                            'hist0':{'hist_type':'unweighted_histogram', 'set':whole_bullard_q_downward_TvR_mc_all, 'sigma_set':whole_bullard_sigma_q_downward_TvR_mc_all, 'color':'red', 'alpha':0, 'number_bins':50, 'figure_label':None},                             
                            'hist1':{'hist_type':'unweighted_histogram', 'set':whole_bullard_q_downward_TvR_cc_mc_all, 'sigma_set':whole_bullard_sigma_q_downward_TvR_cc_mc_all, 'color':'blue', 'alpha':0, 'number_bins':50, 'figure_label':'f'}}
                            plotting_functions.plot_temperature_stratigraphy_conductivity(max_depth_m_plot, k_distribution, plot_lith_fill_dict, plot_lith_fill_dict_keyword, strat_interp_lith_k_calcs_df, T_plotting_dict, k_plotting_dict, res_plotting_dict, bullard_TvR_plotting_dict, bullard_RvT_plotting_dict, pc_plotting_dict, rc_plotting_dict, qhist_plotting_dict, T_range, figure_name)
                            
                            # Plot perturbed temperature and conductivity and resulting thermal resistivity, perturbed palaeoclimate and smoothed recent climatic history
                            figure_name = figures_path + '_i' + '_jsim' + str(jsim) + T_suffix_mc + k_suffix_mc + '_res' + pc_suffix_mc + rc_suffix
                            T_plotting_dict = {'number_lines':3, 
                            'line0':{'line_type':'errorbar', 'zT_m_plot':zT_m, 'T_plot':T, 'zT_error_m_plot':zT_error_m, 'T_error_plot':T_error, 'fmt':'.k', 'alpha':0.125, 'markeredgecolor':None, 'figure_label':None}, 
                            'line1':{'line_type':'errorbar', 'zT_m_plot':zT_m_subsampled, 'T_plot':T_subsampled, 'zT_error_m_plot':zT_error_m_subsampled, 'T_error_plot':T_error_subsampled, 'fmt':'.k', 'alpha':0.5, 'markeredgecolor':None, 'figure_label':None},
                            'line2':{'line_type':'errorbar', 'zT_m_plot':zT_m_input, 'T_plot':T_input, 'zT_error_m_plot':zT_error_m_input, 'T_error_plot':T_error_input, 'fmt':'.r', 'alpha':1, 'markeredgecolor':None, 'figure_label':'a'}}
                            if k_distribution != 'in_situ_normal':
                                k_plotting_dict = {'number_lines':2, 
                                'line0':{'line_type':'strat_interp_lith_k', 'z0_m':layer_z0_m_subsampled, 'z1_m':layer_z1_m_subsampled, 'min_k':layer_min_k_subsampled, 'max_k':layer_max_k_subsampled, 'mean_k':layer_mean_k_subsampled, 'k_assigned_error':layer_mean_k_error_subsampled, 'color':'black', 'alpha':0.125, 'zorder':5, 'figure_label':None},
                                'line1':{'line_type':'mc_input', 'layer_zk_m_input_plot':layer_zk_m_input_plot, 'layer_k_input_plot':layer_k_input_plot, 'color':'black', 'alpha':1, 'zorder':6, 'figure_label':'b'}}
                            else:
                                k_plotting_dict = {'number_lines':2,
                                'line0':{'line_type':'mean_errorbar', 'layer_zk_m_plot':layer_zmid_m, 'layer_k_plot':layer_mean_k, 'layer_zk_error_m_plot':layer_zmid_error_m, 'layer_k_error_plot':layer_mean_k_error, 'fmt':'.k', 'alpha':0.125, 'zorder':5, 'figure_label':'b'},
                                'line1':{'line_type':'mc_input', 'layer_zk_m_input_plot':layer_zk_m_input_plot, 'layer_k_input_plot':layer_k_input_plot, 'color':'black', 'alpha':1, 'zorder':6, 'figure_label':'b'}}
                            # Set up options for plotting thermal resistivity
                            res_plotting_dict = {'number_lines':1, 'line0':{'line_type':'errorbar', 'zR_m_plot':zT_m_input, 'R_plot':R1_downward, 'zR_error_m_plot':zT_error_m_input, 'R_error_plot':sigma_R1_downward, 'fmt':'.k', 'alpha':1, 'markeredgecolor':None, 'figure_label':'c'}}
                            # Set up options for plotting Bullard plots - plot as empty frames
                            bullard_TvR_plotting_dict = {'number_lines':1, 'line0':{'line_type':'emptyframe', 'empty_x':1, 'empty_y':1, 'figure_label':'e'}}
                            bullard_RvT_plotting_dict = None
                            # Set up options for plotting palaeoclimate
                            pc_plotting_dict = {'pc_plot_input_option':'yes', 'pc_deltaTs':pc_deltaTs, 'pc_sigma_deltaTs':pc_sigma_deltaTs, 'pc_t0':pc_t0, 'pc_t1':pc_t1, 'pc_sigma_t0':pc_sigma_t0, 'pc_sigma_t1':pc_sigma_t1, 'pc_deltaTs_input':pc_deltaTs_input, 'pc_t0_input':pc_t0_input, 'pc_t1_input':pc_t1_input, 'fmt':'-b', 'figure_label':'d'}
                            # Set up options for plotting recent climate
                            rc_plotting_dict = {'rc_plot_input_option':'smoothed', 'rc_deltaTs':rc_deltaTs, 'rc_t0':rc_t0, 'rc_t1':rc_t1, 'rc_deltaTs_cut':rc_deltaTs_cut, 'rc_t0_cut':rc_t0_cut, 'rc_t1_cut':rc_t1_cut, 'rc_deltaTs_smoothed_cut':rc_deltaTs_smoothed_cut, 'rc_t0_smoothing_cutoff_cut':rc_t0_smoothing_cutoff_cut, 'rc_t1_smoothing_cutoff_cut':rc_t1_smoothing_cutoff_cut, 'rc_deltaTs_input':rc_deltaTs_input, 'rc_t0_input':rc_t0_input, 'rc_t1_input':rc_t1_input, 'rc_ty':rc_ty, 'color':'blue','figure_label':'e'}
                            # Set up options for plotting heat-flow histogram
                            qhist_plotting_dict = {'number_hists':2,                             
                            'hist0':{'hist_type':'unweighted_histogram', 'set':whole_bullard_q_downward_TvR_mc_all, 'sigma_set':whole_bullard_sigma_q_downward_TvR_mc_all, 'color':'red', 'alpha':0, 'number_bins':50, 'figure_label':None},                             
                            'hist1':{'hist_type':'unweighted_histogram', 'set':whole_bullard_q_downward_TvR_cc_mc_all, 'sigma_set':whole_bullard_sigma_q_downward_TvR_cc_mc_all, 'color':'blue', 'alpha':0, 'number_bins':50, 'figure_label':'f'}}
                            plotting_functions.plot_temperature_stratigraphy_conductivity(max_depth_m_plot, k_distribution, plot_lith_fill_dict, plot_lith_fill_dict_keyword, strat_interp_lith_k_calcs_df, T_plotting_dict, k_plotting_dict, res_plotting_dict, bullard_TvR_plotting_dict, bullard_RvT_plotting_dict, pc_plotting_dict, rc_plotting_dict, qhist_plotting_dict, T_range, figure_name)
                            
                            # Plot perturbed temperature and conductivity and resulting thermal resistivity, and perturbed climatic histories
                            figure_name = figures_path + '_j' + '_jsim' + str(jsim) + T_suffix_mc + k_suffix_mc + '_res' + pc_suffix_mc + rc_suffix_mc
                            T_plotting_dict = {'number_lines':3, 
                            'line0':{'line_type':'errorbar', 'zT_m_plot':zT_m, 'T_plot':T, 'zT_error_m_plot':zT_error_m, 'T_error_plot':T_error, 'fmt':'.k', 'alpha':0.125, 'markeredgecolor':None, 'figure_label':None}, 
                            'line1':{'line_type':'errorbar', 'zT_m_plot':zT_m_subsampled, 'T_plot':T_subsampled, 'zT_error_m_plot':zT_error_m_subsampled, 'T_error_plot':T_error_subsampled, 'fmt':'.k', 'alpha':0.5, 'markeredgecolor':None, 'figure_label':None},
                            'line2':{'line_type':'errorbar', 'zT_m_plot':zT_m_input, 'T_plot':T_input, 'zT_error_m_plot':zT_error_m_input, 'T_error_plot':T_error_input, 'fmt':'.r', 'alpha':1, 'markeredgecolor':None, 'figure_label':'a'}}
                            if k_distribution != 'in_situ_normal':
                                k_plotting_dict = {'number_lines':2, 
                                'line0':{'line_type':'strat_interp_lith_k', 'z0_m':layer_z0_m_subsampled, 'z1_m':layer_z1_m_subsampled, 'min_k':layer_min_k_subsampled, 'max_k':layer_max_k_subsampled, 'mean_k':layer_mean_k_subsampled, 'k_assigned_error':layer_mean_k_error_subsampled, 'color':'black', 'alpha':0.125, 'zorder':5, 'figure_label':None},
                                'line1':{'line_type':'mc_input', 'layer_zk_m_input_plot':layer_zk_m_input_plot, 'layer_k_input_plot':layer_k_input_plot, 'color':'black', 'alpha':1, 'zorder':6, 'figure_label':'b'}}
                            else:
                                k_plotting_dict = {'number_lines':2,
                                'line0':{'line_type':'mean_errorbar', 'layer_zk_m_plot':layer_zmid_m, 'layer_k_plot':layer_mean_k, 'layer_zk_error_m_plot':layer_zmid_error_m, 'layer_k_error_plot':layer_mean_k_error, 'fmt':'.k', 'alpha':0.125, 'zorder':5, 'figure_label':'b'},
                                'line1':{'line_type':'mc_input', 'layer_zk_m_input_plot':layer_zk_m_input_plot, 'layer_k_input_plot':layer_k_input_plot, 'color':'black', 'alpha':1, 'zorder':6, 'figure_label':'b'}}
                            # Set up options for plotting thermal resistivity
                            res_plotting_dict = {'number_lines':1, 'line0':{'line_type':'errorbar', 'zR_m_plot':zT_m_input, 'R_plot':R1_downward, 'zR_error_m_plot':zT_error_m_input, 'R_error_plot':sigma_R1_downward, 'fmt':'.k', 'alpha':1, 'markeredgecolor':None, 'figure_label':'c'}}
                            # Set up options for plotting Bullard plots - plot as empty frames
                            bullard_TvR_plotting_dict = {'number_lines':1, 'line0':{'line_type':'emptyframe', 'empty_x':1, 'empty_y':1, 'figure_label':'e'}}
                            bullard_RvT_plotting_dict = None
                            # Set up options for plotting palaeoclimate
                            pc_plotting_dict = {'pc_plot_input_option':'yes', 'pc_deltaTs':pc_deltaTs, 'pc_sigma_deltaTs':pc_sigma_deltaTs, 'pc_t0':pc_t0, 'pc_t1':pc_t1, 'pc_sigma_t0':pc_sigma_t0, 'pc_sigma_t1':pc_sigma_t1, 'pc_deltaTs_input':pc_deltaTs_input, 'pc_t0_input':pc_t0_input, 'pc_t1_input':pc_t1_input, 'fmt':'-b', 'figure_label':'d'}
                            # Set up options for plotting recent climate
                            rc_plotting_dict = {'rc_plot_input_option':'mc', 'rc_deltaTs':rc_deltaTs, 'rc_t0':rc_t0, 'rc_t1':rc_t1, 'rc_deltaTs_cut':rc_deltaTs_cut, 'rc_t0_cut':rc_t0_cut, 'rc_t1_cut':rc_t1_cut, 'rc_deltaTs_smoothed_cut':rc_deltaTs_smoothed_cut, 'rc_t0_smoothing_cutoff_cut':rc_t0_smoothing_cutoff_cut, 'rc_t1_smoothing_cutoff_cut':rc_t1_smoothing_cutoff_cut, 'rc_deltaTs_input':rc_deltaTs_input, 'rc_t0_input':rc_t0_input, 'rc_t1_input':rc_t1_input, 'rc_ty':rc_ty, 'color':'blue', 'figure_label':'e'}
                            # Set up options for plotting heat-flow histogram
                            qhist_plotting_dict = {'number_hists':2,                             
                            'hist0':{'hist_type':'unweighted_histogram', 'set':whole_bullard_q_downward_TvR_mc_all, 'sigma_set':whole_bullard_sigma_q_downward_TvR_mc_all, 'color':'red', 'alpha':0, 'number_bins':50, 'figure_label':None},                             
                            'hist1':{'hist_type':'unweighted_histogram', 'set':whole_bullard_q_downward_TvR_cc_mc_all, 'sigma_set':whole_bullard_sigma_q_downward_TvR_cc_mc_all, 'color':'blue', 'alpha':0, 'number_bins':50, 'figure_label':'f'}}
                            plotting_functions.plot_temperature_stratigraphy_conductivity(max_depth_m_plot, k_distribution, plot_lith_fill_dict, plot_lith_fill_dict_keyword, strat_interp_lith_k_calcs_df, T_plotting_dict, k_plotting_dict, res_plotting_dict, bullard_TvR_plotting_dict, bullard_RvT_plotting_dict, pc_plotting_dict, rc_plotting_dict, qhist_plotting_dict, T_range, figure_name)
                            
                            # Plot perturbed temperature and conductivity and resulting thermal resistivity, perturbed climatic histories, and climate-corrected temperatures
                            figure_name = figures_path + '_k' + '_jsim' + str(jsim) + T_suffix_mc + '_cc' + k_suffix_mc + '_res' + pc_suffix_mc + rc_suffix_mc
                            T_plotting_dict = {'number_lines':4, 
                            'line0':{'line_type':'errorbar', 'zT_m_plot':zT_m, 'T_plot':T, 'zT_error_m_plot':zT_error_m, 'T_error_plot':T_error, 'fmt':'.k', 'alpha':0.125, 'markeredgecolor':None, 'figure_label':None}, 
                            'line1':{'line_type':'errorbar', 'zT_m_plot':zT_m_subsampled, 'T_plot':T_subsampled, 'zT_error_m_plot':zT_error_m_subsampled, 'T_error_plot':T_error_subsampled, 'fmt':'.k', 'alpha':0.5, 'markeredgecolor':None, 'figure_label':None},
                            'line2':{'line_type':'errorbar', 'zT_m_plot':zT_m_input, 'T_plot':T_input, 'zT_error_m_plot':zT_error_m_input, 'T_error_plot':T_error_input, 'fmt':'.r', 'alpha':1, 'markeredgecolor':None, 'figure_label':'a'},
                            'line3':{'line_type':'errorbar', 'zT_m_plot':zT_m_input, 'T_plot':T_input_cc, 'zT_error_m_plot':zT_error_m_input, 'T_error_plot':T_error_input, 'fmt':'.b', 'alpha':1, 'markeredgecolor':None, 'figure_label':'a'}}
                            if k_distribution != 'in_situ_normal':
                                k_plotting_dict = {'number_lines':2, 
                                'line0':{'line_type':'strat_interp_lith_k', 'z0_m':layer_z0_m_subsampled, 'z1_m':layer_z1_m_subsampled, 'min_k':layer_min_k_subsampled, 'max_k':layer_max_k_subsampled, 'mean_k':layer_mean_k_subsampled, 'k_assigned_error':layer_mean_k_error_subsampled, 'color':'black', 'alpha':0.125, 'zorder':5, 'figure_label':None},
                                'line1':{'line_type':'mc_input', 'layer_zk_m_input_plot':layer_zk_m_input_plot, 'layer_k_input_plot':layer_k_input_plot, 'color':'black', 'alpha':1, 'zorder':6, 'figure_label':'b'}}
                            else:
                                k_plotting_dict = {'number_lines':2,
                                'line0':{'line_type':'mean_errorbar', 'layer_zk_m_plot':layer_zmid_m, 'layer_k_plot':layer_mean_k, 'layer_zk_error_m_plot':layer_zmid_error_m, 'layer_k_error_plot':layer_mean_k_error, 'fmt':'.k', 'alpha':0.125, 'zorder':5, 'figure_label':'b'},
                                'line1':{'line_type':'mc_input', 'layer_zk_m_input_plot':layer_zk_m_input_plot, 'layer_k_input_plot':layer_k_input_plot, 'color':'black', 'alpha':1, 'zorder':6, 'figure_label':'b'}}
                            # Set up options for plotting thermal resistivity
                            res_plotting_dict = {'number_lines':1, 'line0':{'line_type':'errorbar', 'zR_m_plot':zT_m_input, 'R_plot':R1_downward, 'zR_error_m_plot':zT_error_m_input, 'R_error_plot':sigma_R1_downward, 'fmt':'.k', 'alpha':1, 'markeredgecolor':None, 'figure_label':'c'}}
                            # Set up options for plotting Bullard plots - plot as empty frames
                            bullard_TvR_plotting_dict = {'number_lines':1, 'line0':{'line_type':'emptyframe', 'empty_x':1, 'empty_y':1, 'figure_label':'e'}}
                            bullard_RvT_plotting_dict = None
                            # Set up options for plotting palaeoclimate
                            pc_plotting_dict = {'pc_plot_input_option':'yes', 'pc_deltaTs':pc_deltaTs, 'pc_sigma_deltaTs':pc_sigma_deltaTs, 'pc_t0':pc_t0, 'pc_t1':pc_t1, 'pc_sigma_t0':pc_sigma_t0, 'pc_sigma_t1':pc_sigma_t1, 'pc_deltaTs_input':pc_deltaTs_input, 'pc_t0_input':pc_t0_input, 'pc_t1_input':pc_t1_input, 'fmt':'-b', 'figure_label':'d'}
                            # Set up options for plotting recent climate
                            rc_plotting_dict = {'rc_plot_input_option':'mc', 'rc_deltaTs':rc_deltaTs, 'rc_t0':rc_t0, 'rc_t1':rc_t1, 'rc_deltaTs_cut':rc_deltaTs_cut, 'rc_t0_cut':rc_t0_cut, 'rc_t1_cut':rc_t1_cut, 'rc_deltaTs_smoothed_cut':rc_deltaTs_smoothed_cut, 'rc_t0_smoothing_cutoff_cut':rc_t0_smoothing_cutoff_cut, 'rc_t1_smoothing_cutoff_cut':rc_t1_smoothing_cutoff_cut, 'rc_deltaTs_input':rc_deltaTs_input, 'rc_t0_input':rc_t0_input, 'rc_t1_input':rc_t1_input, 'rc_ty':rc_ty, 'color':'blue', 'figure_label':'e'}
                            # Set up options for plotting heat-flow histogram
                            qhist_plotting_dict = {'number_hists':2,                             
                            'hist0':{'hist_type':'unweighted_histogram', 'set':whole_bullard_q_downward_TvR_mc_all, 'sigma_set':whole_bullard_sigma_q_downward_TvR_mc_all, 'color':'red', 'alpha':0, 'number_bins':50, 'figure_label':None},                             
                            'hist1':{'hist_type':'unweighted_histogram', 'set':whole_bullard_q_downward_TvR_cc_mc_all, 'sigma_set':whole_bullard_sigma_q_downward_TvR_cc_mc_all, 'color':'blue', 'alpha':0, 'number_bins':50, 'figure_label':'f'}}
                            plotting_functions.plot_temperature_stratigraphy_conductivity(max_depth_m_plot, k_distribution, plot_lith_fill_dict, plot_lith_fill_dict_keyword, strat_interp_lith_k_calcs_df, T_plotting_dict, k_plotting_dict, res_plotting_dict, bullard_TvR_plotting_dict, bullard_RvT_plotting_dict, pc_plotting_dict, rc_plotting_dict, qhist_plotting_dict, T_range, figure_name)
                            
                            
                            
                            
                            # Plot perturbed temperature and conductivity and resulting thermal resistivity, perturbed climatic histories, and climate-corrected temperatures
                            figure_name = figures_path + '_l' + '_jsim' + str(jsim) + T_suffix_mc + '_cc' + k_suffix_mc + '_res' + pc_suffix_mc + rc_suffix_mc + '_bul'
                            T_plotting_dict = {'number_lines':4, 
                            'line0':{'line_type':'errorbar', 'zT_m_plot':zT_m, 'T_plot':T, 'zT_error_m_plot':zT_error_m, 'T_error_plot':T_error, 'fmt':'.k', 'alpha':0.125, 'markeredgecolor':None, 'figure_label':None}, 
                            'line1':{'line_type':'errorbar', 'zT_m_plot':zT_m_subsampled, 'T_plot':T_subsampled, 'zT_error_m_plot':zT_error_m_subsampled, 'T_error_plot':T_error_subsampled, 'fmt':'.k', 'alpha':0.5, 'markeredgecolor':None, 'figure_label':None},
                            'line2':{'line_type':'errorbar', 'zT_m_plot':zT_m_input, 'T_plot':T_input, 'zT_error_m_plot':zT_error_m_input, 'T_error_plot':T_error_input, 'fmt':'.r', 'alpha':1, 'markeredgecolor':None, 'figure_label':'a'},
                            'line3':{'line_type':'errorbar', 'zT_m_plot':zT_m_input, 'T_plot':T_input_cc, 'zT_error_m_plot':zT_error_m_input, 'T_error_plot':T_error_input, 'fmt':'.b', 'alpha':1, 'markeredgecolor':None, 'figure_label':'a'}}
                            if k_distribution != 'in_situ_normal':
                                k_plotting_dict = {'number_lines':2, 
                                'line0':{'line_type':'strat_interp_lith_k', 'z0_m':layer_z0_m_subsampled, 'z1_m':layer_z1_m_subsampled, 'min_k':layer_min_k_subsampled, 'max_k':layer_max_k_subsampled, 'mean_k':layer_mean_k_subsampled, 'k_assigned_error':layer_mean_k_error_subsampled, 'color':'black', 'alpha':0.125, 'zorder':5, 'figure_label':None},
                                'line1':{'line_type':'mc_input', 'layer_zk_m_input_plot':layer_zk_m_input_plot, 'layer_k_input_plot':layer_k_input_plot, 'color':'black', 'alpha':1, 'zorder':6, 'figure_label':'b'}}
                            else:
                                k_plotting_dict = {'number_lines':2,
                                'line0':{'line_type':'mean_errorbar', 'layer_zk_m_plot':layer_zmid_m, 'layer_k_plot':layer_mean_k, 'layer_zk_error_m_plot':layer_zmid_error_m, 'layer_k_error_plot':layer_mean_k_error, 'fmt':'.k', 'alpha':0.125, 'zorder':5, 'figure_label':'b'},
                                'line1':{'line_type':'mc_input', 'layer_zk_m_input_plot':layer_zk_m_input_plot, 'layer_k_input_plot':layer_k_input_plot, 'color':'black', 'alpha':1, 'zorder':6, 'figure_label':'b'}}
                            # Set up options for plotting thermal resistivity
                            res_plotting_dict = {'number_lines':1, 'line0':{'line_type':'errorbar', 'zR_m_plot':zT_m_input, 'R_plot':R1_downward, 'zR_error_m_plot':zT_error_m_input, 'R_error_plot':sigma_R1_downward, 'fmt':'.k', 'alpha':1, 'markeredgecolor':None, 'figure_label':'c'}}
                            # Set up options for plotting Bullard plots
                            bullard_TvR_plotting_dict = {'number_lines':6, 
                            'line0':{'line_type':'TvR_errorbar', 'T_plot':T_input, 'R_plot':R1_downward, 'T_error_plot':T_error_input, 'R_error_plot':sigma_R1_downward, 'fmt':'.r', 'alpha':1, 'markeredgecolor':None, 'figure_label':None},
                            'line1':{'line_type':'TvR_errorbar', 'T_plot':T_input_cc, 'R_plot':R1_downward, 'T_error_plot':T_error_input_cc, 'R_error_plot':sigma_R1_downward, 'fmt':'.b', 'alpha':1, 'markeredgecolor':None, 'figure_label':None},
                            'line2':{'line_type':'fitted_line_x', 'x_plot':R1_downward, 'gradient':q_downward_TvR, 'intercept':c_downward_TvR, 'color':'red', 'linestyle':'dotted', 'alpha':1, 'legend_label':'Q = ' + str(Q_round_downward_TvR) + r' $\pm$ ' + str(sigma_Q_round_downward_TvR) + r' W m$^{-2}$', 'figure_label':None},
                            'line3':{'line_type':'fitted_line_y', 'y_plot':T_input, 'gradient':invq_downward_RvT, 'intercept':invc_downward_RvT, 'color':'red', 'linestyle':'dashed', 'alpha':1, 'legend_label':'Q = ' + str(Q_round_downward_RvT) + r' $\pm$ ' + str(sigma_Q_round_downward_RvT) + r' W m$^{-2}$', 'figure_label':None},
                            'line4':{'line_type':'fitted_line_x', 'x_plot':R1_downward, 'gradient':q_downward_TvR_cc, 'intercept':c_downward_TvR_cc, 'color':'blue', 'linestyle':'dotted', 'alpha':1, 'legend_label':'Q = ' + str(Q_round_downward_TvR_cc) + r' $\pm$ ' + str(sigma_Q_round_downward_TvR_cc) + r' W m$^{-2}$', 'figure_label':None},
                            'line5':{'line_type':'fitted_line_y', 'y_plot':T_input_cc, 'gradient':invq_downward_RvT_cc, 'intercept':invc_downward_RvT_cc, 'color':'blue', 'linestyle':'dashed', 'alpha':1, 'legend_label':'Q = ' + str(Q_round_downward_RvT_cc) + r' $\pm$ ' + str(sigma_Q_round_downward_RvT_cc) + r' W m$^{-2}$', 'figure_label':'e'}}
                            bullard_RvT_plotting_dict = None
                            # Set up options for plotting palaeoclimate
                            pc_plotting_dict = {'pc_plot_input_option':'yes', 'pc_deltaTs':pc_deltaTs, 'pc_sigma_deltaTs':pc_sigma_deltaTs, 'pc_t0':pc_t0, 'pc_t1':pc_t1, 'pc_sigma_t0':pc_sigma_t0, 'pc_sigma_t1':pc_sigma_t1, 'pc_deltaTs_input':pc_deltaTs_input, 'pc_t0_input':pc_t0_input, 'pc_t1_input':pc_t1_input, 'fmt':'-b', 'figure_label':'d'}
                            # Set up options for plotting recent climate
                            rc_plotting_dict = {'rc_plot_input_option':'mc', 'rc_deltaTs':rc_deltaTs, 'rc_t0':rc_t0, 'rc_t1':rc_t1, 'rc_deltaTs_cut':rc_deltaTs_cut, 'rc_t0_cut':rc_t0_cut, 'rc_t1_cut':rc_t1_cut, 'rc_deltaTs_smoothed_cut':rc_deltaTs_smoothed_cut, 'rc_t0_smoothing_cutoff_cut':rc_t0_smoothing_cutoff_cut, 'rc_t1_smoothing_cutoff_cut':rc_t1_smoothing_cutoff_cut, 'rc_deltaTs_input':rc_deltaTs_input, 'rc_t0_input':rc_t0_input, 'rc_t1_input':rc_t1_input, 'rc_ty':rc_ty, 'color':'blue', 'figure_label':'e'}
                            # Set up options for plotting heat-flow histogram
                            qhist_plotting_dict = {'number_hists':2,                             
                            'hist0':{'hist_type':'unweighted_histogram', 'set':whole_bullard_q_downward_TvR_mc_all, 'sigma_set':whole_bullard_sigma_q_downward_TvR_mc_all, 'color':'red', 'alpha':0, 'number_bins':50, 'figure_label':None},                             
                            'hist1':{'hist_type':'unweighted_histogram', 'set':whole_bullard_q_downward_TvR_cc_mc_all, 'sigma_set':whole_bullard_sigma_q_downward_TvR_cc_mc_all, 'color':'blue', 'alpha':0, 'number_bins':50, 'figure_label':'f'}}
                            plotting_functions.plot_temperature_stratigraphy_conductivity(max_depth_m_plot, k_distribution, plot_lith_fill_dict, plot_lith_fill_dict_keyword, strat_interp_lith_k_calcs_df, T_plotting_dict, k_plotting_dict, res_plotting_dict, bullard_TvR_plotting_dict, bullard_RvT_plotting_dict, pc_plotting_dict, rc_plotting_dict, qhist_plotting_dict, T_range, figure_name)
                                  
  
                            
                            ### Plot last Monte Carlo iteration together with overall heat-flow histograms
                            figure_name = figures_path + '_m' + '_jsim' + str(jsim) + T_suffix_mc + '_cc' + k_suffix_mc + '_res' + pc_suffix_mc + rc_suffix_mc + '_bul' + '_qhist'
                            T_plotting_dict = {'number_lines':4, 
                            'line0':{'line_type':'errorbar', 'zT_m_plot':zT_m, 'T_plot':T, 'zT_error_m_plot':zT_error_m, 'T_error_plot':T_error, 'fmt':'.k', 'alpha':0.125, 'markeredgecolor':None, 'figure_label':None}, 
                            'line1':{'line_type':'errorbar', 'zT_m_plot':zT_m_subsampled, 'T_plot':T_subsampled, 'zT_error_m_plot':zT_error_m_subsampled, 'T_error_plot':T_error_subsampled, 'fmt':'.k', 'alpha':0.5, 'markeredgecolor':None, 'figure_label':None},
                            'line2':{'line_type':'errorbar', 'zT_m_plot':zT_m_input, 'T_plot':T_input, 'zT_error_m_plot':zT_error_m_input, 'T_error_plot':T_error_input, 'fmt':'.r', 'alpha':1, 'markeredgecolor':None, 'figure_label':'a'},
                            'line3':{'line_type':'errorbar', 'zT_m_plot':zT_m_input, 'T_plot':T_input_cc, 'zT_error_m_plot':zT_error_m_input, 'T_error_plot':T_error_input, 'fmt':'.b', 'alpha':1, 'markeredgecolor':None, 'figure_label':'a'}}
                            if k_distribution != 'in_situ_normal':
                                k_plotting_dict = {'number_lines':2, 
                                'line0':{'line_type':'strat_interp_lith_k', 'z0_m':layer_z0_m_subsampled, 'z1_m':layer_z1_m_subsampled, 'min_k':layer_min_k_subsampled, 'max_k':layer_max_k_subsampled, 'mean_k':layer_mean_k_subsampled, 'k_assigned_error':layer_mean_k_error_subsampled, 'color':'black', 'alpha':0.125, 'zorder':5, 'figure_label':None},
                                'line1':{'line_type':'mc_input', 'layer_zk_m_input_plot':layer_zk_m_input_plot, 'layer_k_input_plot':layer_k_input_plot, 'color':'black', 'alpha':1, 'zorder':6, 'figure_label':'b'}}
                            else:
                                k_plotting_dict = {'number_lines':2,
                                'line0':{'line_type':'mean_errorbar', 'layer_zk_m_plot':layer_zmid_m, 'layer_k_plot':layer_mean_k, 'layer_zk_error_m_plot':layer_zmid_error_m, 'layer_k_error_plot':layer_mean_k_error, 'fmt':'.k', 'alpha':0.125, 'zorder':5, 'figure_label':'b'},
                                'line1':{'line_type':'mc_input', 'layer_zk_m_input_plot':layer_zk_m_input_plot, 'layer_k_input_plot':layer_k_input_plot, 'color':'black', 'alpha':1, 'zorder':6, 'figure_label':'b'}}
                            # Set up options for plotting thermal resistivity
                            res_plotting_dict = {'number_lines':1, 'line0':{'line_type':'errorbar', 'zR_m_plot':zT_m_input, 'R_plot':R1_downward, 'zR_error_m_plot':zT_error_m_input, 'R_error_plot':sigma_R1_downward, 'fmt':'.k', 'alpha':1, 'markeredgecolor':None, 'figure_label':'c'}}
                            # Set up options for plotting Bullard plots
                            bullard_TvR_plotting_dict = {'number_lines':6, 
                            'line0':{'line_type':'TvR_errorbar', 'T_plot':T_input, 'R_plot':R1_downward, 'T_error_plot':T_error_input, 'R_error_plot':sigma_R1_downward, 'fmt':'.r', 'alpha':1, 'markeredgecolor':None, 'figure_label':None},
                            'line1':{'line_type':'TvR_errorbar', 'T_plot':T_input_cc, 'R_plot':R1_downward, 'T_error_plot':T_error_input_cc, 'R_error_plot':sigma_R1_downward, 'fmt':'.b', 'alpha':1, 'markeredgecolor':None, 'figure_label':None},
                            'line2':{'line_type':'fitted_line_x', 'x_plot':R1_downward, 'gradient':q_downward_TvR, 'intercept':c_downward_TvR, 'color':'red', 'linestyle':'dotted', 'alpha':1, 'legend_label':'Q = ' + str(Q_round_downward_TvR) + r' $\pm$ ' + str(sigma_Q_round_downward_TvR) + r' W m$^{-2}$', 'figure_label':None},
                            'line3':{'line_type':'fitted_line_y', 'y_plot':T_input, 'gradient':invq_downward_RvT, 'intercept':invc_downward_RvT, 'color':'red', 'linestyle':'dashed', 'alpha':1, 'legend_label':'Q = ' + str(Q_round_downward_RvT) + r' $\pm$ ' + str(sigma_Q_round_downward_RvT) + r' W m$^{-2}$', 'figure_label':None},
                            'line4':{'line_type':'fitted_line_x', 'x_plot':R1_downward, 'gradient':q_downward_TvR_cc, 'intercept':c_downward_TvR_cc, 'color':'blue', 'linestyle':'dotted', 'alpha':1, 'legend_label':'Q = ' + str(Q_round_downward_TvR_cc) + r' $\pm$ ' + str(sigma_Q_round_downward_TvR_cc) + r' W m$^{-2}$', 'figure_label':None},
                            'line5':{'line_type':'fitted_line_y', 'y_plot':T_input_cc, 'gradient':invq_downward_RvT_cc, 'intercept':invc_downward_RvT_cc, 'color':'blue', 'linestyle':'dashed', 'alpha':1, 'legend_label':'Q = ' + str(Q_round_downward_RvT_cc) + r' $\pm$ ' + str(sigma_Q_round_downward_RvT_cc) + r' W m$^{-2}$', 'figure_label':'e'}}
                            bullard_RvT_plotting_dict = None
                            # Set up options for plotting palaeoclimate
                            pc_plotting_dict = {'pc_plot_input_option':'yes', 'pc_deltaTs':pc_deltaTs, 'pc_sigma_deltaTs':pc_sigma_deltaTs, 'pc_t0':pc_t0, 'pc_t1':pc_t1, 'pc_sigma_t0':pc_sigma_t0, 'pc_sigma_t1':pc_sigma_t1, 'pc_deltaTs_input':pc_deltaTs_input, 'pc_t0_input':pc_t0_input, 'pc_t1_input':pc_t1_input, 'fmt':'-b', 'figure_label':'d'}
                            # Set up options for plotting recent climate
                            rc_plotting_dict = {'rc_plot_input_option':'mc', 'rc_deltaTs':rc_deltaTs, 'rc_t0':rc_t0, 'rc_t1':rc_t1, 'rc_deltaTs_cut':rc_deltaTs_cut, 'rc_t0_cut':rc_t0_cut, 'rc_t1_cut':rc_t1_cut, 'rc_deltaTs_smoothed_cut':rc_deltaTs_smoothed_cut, 'rc_t0_smoothing_cutoff_cut':rc_t0_smoothing_cutoff_cut, 'rc_t1_smoothing_cutoff_cut':rc_t1_smoothing_cutoff_cut, 'rc_deltaTs_input':rc_deltaTs_input, 'rc_t0_input':rc_t0_input, 'rc_t1_input':rc_t1_input, 'rc_ty':rc_ty, 'color':'blue', 'figure_label':'e'}
                            # Set up options for plotting heat-flow histograms
                            qhist_plotting_dict = {'number_hists':2,
                            'hist0':{'hist_type':'unweighted_histogram', 'set':whole_bullard_q_downward_TvR_mc_all, 'sigma_set':whole_bullard_sigma_q_downward_TvR_mc_all, 'color':'red', 'alpha':0.8, 'number_bins':50, 'legend_label':'Q = ' + str(whole_bullard_Q_downward_TvR_mc_mean_round) + r' $\pm$ ' + str(whole_bullard_Q_downward_TvR_mc_stdev_round) + r' W m$^{-2}$', 'figure_label':None},
                            'hist1':{'hist_type':'unweighted_histogram', 'set':whole_bullard_q_downward_TvR_cc_mc_all, 'sigma_set':whole_bullard_sigma_q_downward_TvR_cc_mc_all, 'color':'blue', 'alpha':0.8, 'number_bins':50, 'legend_label':'Q = ' + str(whole_bullard_Q_downward_TvR_cc_mc_mean_round) + r' $\pm$ ' + str(whole_bullard_Q_downward_TvR_cc_mc_stdev_round) + r' W m$^{-2}$', 'figure_label':'f'}}
                            plotting_functions.plot_temperature_stratigraphy_conductivity(max_depth_m_plot, k_distribution, plot_lith_fill_dict, plot_lith_fill_dict_keyword, strat_interp_lith_k_calcs_df, T_plotting_dict, k_plotting_dict, res_plotting_dict, bullard_TvR_plotting_dict, bullard_RvT_plotting_dict, pc_plotting_dict, rc_plotting_dict, qhist_plotting_dict, T_range, figure_name)
                            
                            if 1==1:continue


                            figure_name = figures_path + '_mc_results' + T_suffix_mc + '_cc' + k_suffix_mc + '_res' + pc_suffix_mc + rc_suffix_mc + '_bul'
                            T_plotting_dict = {'number_lines':1, 
                            'line0':{'line_type':'mc_results', 'zT_m_plot':zT_m_input_mc_all, 'T_plot':T_input_cc_mc_all, 'colors':'inferno', 'figure_label':'a'}}                 
#                             'line1':{'line_type':'errorbar', 'zT_m_plot':zT_m_subsampled, 'T_plot':T_subsampled, 'zT_error_m_plot':zT_error_m_subsampled, 'T_error_plot':T_error_subsampled, 'fmt':'.k', 'alpha':0.5, 'markeredgecolor':None, 'figure_label':None},
#                             'line2':{'line_type':'errorbar', 'zT_m_plot':zT_m_input, 'T_plot':T_input, 'zT_error_m_plot':zT_error_m_input, 'T_error_plot':T_error_input, 'fmt':'.r', 'alpha':1, 'markeredgecolor':None, 'figure_label':'a'},
#                             'line3':{'line_type':'errorbar', 'zT_m_plot':zT_m_input, 'T_plot':T_input_cc, 'zT_error_m_plot':zT_error_m_input, 'T_error_plot':T_error_input, 'fmt':'.b', 'alpha':1, 'markeredgecolor':None, 'figure_label':'a'}}
                            k_plotting_dict = {'number_lines':1, 
                            'line0':{'line_type':'mc_results', 'k_plot':layer_k_input_plot_mc_all, 'zk_m_plot':layer_zk_m_input_plot_mc_all, 'colors':'inferno', 'figure_label':'b'}}
                            # Set up options for plotting thermal resistivity
                            res_plotting_dict = {'number_lines':1,
                            'line0':{'line_type':'mc_results', 'zR_m_plot':zT_m_input_mc_all, 'R_plot':whole_bullard_R1_downward_mc_all, 'alpha':1, 'colors':'inferno', 'figure_label':'c'}}
                            # Set up options for plotting Bullard plots
                            bullard_TvR_plotting_dict = {'number_lines':1, 
                            'line0':{'line_type':'mc_results', 'T_plot':T_input_cc_mc_all, 'R_plot':whole_bullard_R1_downward_mc_all, 'alpha':1, 'colors':'inferno', 'figure_label':'d'}}
#                             'line1':{'line_type':'TvR_errorbar', 'T_plot':T_input_cc, 'R_plot':R1_downward, 'T_error_plot':T_error_input_cc, 'R_error_plot':sigma_R1_downward, 'fmt':'.b', 'alpha':1, 'markeredgecolor':None, 'figure_label':None},
#                             'line2':{'line_type':'fitted_line_x', 'x_plot':R1_downward, 'gradient':q_downward_TvR, 'intercept':c_downward_TvR, 'color':'red', 'linestyle':'dotted', 'alpha':1, 'legend_label':'Q = ' + str(Q_round_downward_TvR) + r' $\pm$ ' + str(sigma_Q_round_downward_TvR) + r' W m$^{-2}$', 'figure_label':None},
#                             'line3':{'line_type':'fitted_line_y', 'y_plot':T_input, 'gradient':invq_downward_RvT, 'intercept':invc_downward_RvT, 'color':'red', 'linestyle':'dashed', 'alpha':1, 'legend_label':'Q = ' + str(Q_round_downward_RvT) + r' $\pm$ ' + str(sigma_Q_round_downward_RvT) + r' W m$^{-2}$', 'figure_label':None},
#                             'line4':{'line_type':'fitted_line_x', 'x_plot':R1_downward, 'gradient':q_downward_TvR_cc, 'intercept':c_downward_TvR_cc, 'color':'blue', 'linestyle':'dotted', 'alpha':1, 'legend_label':'Q = ' + str(Q_round_downward_TvR_cc) + r' $\pm$ ' + str(sigma_Q_round_downward_TvR_cc) + r' W m$^{-2}$', 'figure_label':None},
#                             'line5':{'line_type':'fitted_line_y', 'y_plot':T_input_cc, 'gradient':invq_downward_RvT_cc, 'intercept':invc_downward_RvT_cc, 'color':'blue', 'linestyle':'dashed', 'alpha':1, 'legend_label':'Q = ' + str(Q_round_downward_RvT_cc) + r' $\pm$ ' + str(sigma_Q_round_downward_RvT_cc) + r' W m$^{-2}$', 'figure_label':'e'}}
                            bullard_RvT_plotting_dict = None
                            # Set up options for plotting palaeoclimate
                            pc_plotting_dict = {'pc_plot_input_option':'mc_results', 'pc_deltaTs':pc_deltaTs_input_plot_mc_all, 'pc_t':pc_t_input_plot_mc_all, 'colors':'inferno', 'figure_label':'d'}
                            # Set up options for plotting recent climate
                            rc_plotting_dict = {'rc_plot_input_option':'mc', 'rc_deltaTs':rc_deltaTs, 'rc_t0':rc_t0, 'rc_t1':rc_t1, 'rc_deltaTs_cut':rc_deltaTs_cut, 'rc_t0_cut':rc_t0_cut, 'rc_t1_cut':rc_t1_cut, 'rc_deltaTs_smoothed_cut':rc_deltaTs_smoothed_cut, 'rc_t0_smoothing_cutoff_cut':rc_t0_smoothing_cutoff_cut, 'rc_t1_smoothing_cutoff_cut':rc_t1_smoothing_cutoff_cut, 'rc_deltaTs_input':rc_deltaTs_input, 'rc_t0_input':rc_t0_input, 'rc_t1_input':rc_t1_input, 'rc_ty':rc_ty, 'figure_label':'e'}
                            plotting_functions.plot_temperature_stratigraphy_conductivity(max_depth_m_plot, k_distribution, plot_lith_fill_dict, plot_lith_fill_dict_keyword, strat_interp_lith_k_calcs_df, T_plotting_dict, k_plotting_dict, res_plotting_dict, bullard_TvR_plotting_dict, bullard_RvT_plotting_dict, pc_plotting_dict, rc_plotting_dict, T_range, figure_name)
                            
                           
                            if 1==1: continue
                            
                            


#                             # Plot perturbed and climate-corrected temperature and perturbed conductivity
#                             T_plotting_dict = {'number_lines':4, 
#                             'line0':{'line_type':'errorbar', 'zT_m_plot':zT_m, 'T_plot':T, 'zT_error_m_plot':zT_error_m, 'T_error_plot':T_error, 'fmt':'.k', 'alpha':0.125, 'markeredgecolor':None, 'figure_label':None},
#                             'line1':{'line_type':'errorbar', 'zT_m_plot':zT_m_subsampled, 'T_plot':T_subsampled, 'zT_error_m_plot':zT_error_m_subsampled, 'T_error_plot':T_error_subsampled, 'fmt':'.k', 'alpha':0.5, 'markeredgecolor':None, 'figure_label':None},
#                             'line2':{'line_type':'errorbar', 'zT_m_plot':zT_m_input, 'T_plot':T_input, 'zT_error_m_plot':zT_error_m_input, 'T_error_plot':T_error_input, 'fmt':'.r', 'alpha':1, 'markeredgecolor':None, 'figure_label':None},
#                             'line3':{'line_type':'errorbar', 'zT_m_plot':zT_m_input, 'T_plot':T_input_cc, 'zT_error_m_plot':zT_error_m_input, 'T_error_plot':T_error_input, 'fmt':'.b', 'alpha':1, 'markeredgecolor':None, 'figure_label':'a'}}
#                             k_plotting_dict = {'number_lines':2, 
#                             'line0':{'line_type':'strat_interp_lith_k', 'z0_m':layer_z0_m_subsampled, 'z1_m':layer_z1_m_subsampled, 'min_k':layer_min_k_subsampled, 'max_k':layer_max_k_subsampled, 'mean_k':layer_mean_k_subsampled, 'k_assigned_error':layer_mean_k_error_subsampled, 'color':'black', 'alpha':0.5, 'zorder':5, 'figure_label':None}, 
#                             'line1':{'line_type':'mc_input', 'layer_zk_m_input_plot':layer_zk_m_input_plot, 'layer_k_input_plot':layer_k_input_plot, 'color':'red', 'alpha':1, 'zorder':6, 'figure_label':'b'}}
#                             res_plotting_dict = None
#                             bullard_TvR_plotting_dict = None
#                             bullard_RvT_plotting_dict = None
#                             figure_name = figures_path + '_jsim' + str(jsim) + T_suffix_mc + '_cc' + pc_suffix_mc + rc_suffix_mc + k_suffix_mc
#                             plotting_functions.plot_temperature_stratigraphy_conductivity(max_depth_m_plot, k_distribution, plot_lith_fill_dict, plot_lith_fill_dict_keyword, strat_interp_lith_k_calcs_df, T_plotting_dict, k_plotting_dict, res_plotting_dict, bullard_TvR_plotting_dict, bullard_RvT_plotting_dict, T_range, figure_name)
                            
                            
#                             # Plot perturbed and climate-corrected temperature and perturbed conductivity and resistivity
#                             T_plotting_dict = {'number_lines':4, 
#                             'line0':{'line_type':'errorbar', 'zT_m_plot':zT_m, 'T_plot':T, 'zT_error_m_plot':zT_error_m, 'T_error_plot':T_error, 'fmt':'.k', 'alpha':0.125, 'markeredgecolor':None, 'figure_label':None, 'figure_label':None},
#                             'line1':{'line_type':'errorbar', 'zT_m_plot':zT_m_subsampled, 'T_plot':T_subsampled, 'zT_error_m_plot':zT_error_m_subsampled, 'T_error_plot':T_error_subsampled, 'fmt':'.k', 'alpha':0.5, 'markeredgecolor':None, 'figure_label':None},
#                             'line2':{'line_type':'errorbar', 'zT_m_plot':zT_m_input, 'T_plot':T_input, 'zT_error_m_plot':zT_error_m_input, 'T_error_plot':T_error_input, 'fmt':'.r', 'alpha':1, 'markeredgecolor':None, 'figure_label':None},
#                             'line3':{'line_type':'errorbar', 'zT_m_plot':zT_m_input, 'T_plot':T_input_cc, 'zT_error_m_plot':zT_error_m_input, 'T_error_plot':T_error_input, 'fmt':'.b', 'alpha':1, 'markeredgecolor':None, 'figure_label':'a'}}
#                             k_plotting_dict = {'number_lines':2, 
#                             'line0':{'line_type':'strat_interp_lith_k', 'z0_m':layer_z0_m_subsampled, 'z1_m':layer_z1_m_subsampled, 'min_k':layer_min_k_subsampled, 'max_k':layer_max_k_subsampled, 'mean_k':layer_mean_k_subsampled, 'k_assigned_error':layer_mean_k_error_subsampled, 'color':'black', 'alpha':0.5, 'zorder':5, 'figure_label':None}, 
#                             'line1':{'line_type':'mc_input', 'layer_zk_m_input_plot':layer_zk_m_input_plot, 'layer_k_input_plot':layer_k_input_plot, 'color':'red', 'alpha':1, 'zorder':6, 'figure_label':'b'}}
#                             res_plotting_dict = {'number_lines':2, 
#                             'line0':{'line_type':'errorbar', 'zR_m_plot':zT_m_input, 'R_plot':R1_downward, 'zR_error_m_plot':zT_error_m_input, 'R_error_plot':sigma_R1_downward, 'fmt':'.r', 'alpha':1, 'markeredgecolor':None, 'figure_label':None},
#                             'line1':{'line_type':'errorbar', 'zR_m_plot':zT_m_input, 'R_plot':R1_upward, 'zR_error_m_plot':zT_error_m_input, 'R_error_plot':sigma_R1_upward, 'fmt':'.k', 'alpha':1, 'markeredgecolor':None, 'figure_label':'b'}}
#                             bullard_TvR_plotting_dict = None
#                             bullard_RvT_plotting_dict = None
#                             figure_name = figures_path + '_jsim' + str(jsim) + T_suffix_mc + '_cc' + pc_suffix_mc + rc_suffix_mc + k_suffix_mc + '_R'
#                             plotting_functions.plot_temperature_stratigraphy_conductivity(max_depth_m_plot, k_distribution, plot_lith_fill_dict, plot_lith_fill_dict_keyword, strat_interp_lith_k_calcs_df, T_plotting_dict, k_plotting_dict, res_plotting_dict, bullard_TvR_plotting_dict, bullard_RvT_plotting_dict, T_range, figure_name)
                            
                            
#                              # Plot perturbed and climate-corrected temperature and perturbed conductivity and resistivity against depth. Plot thermal resistivity against temperature and vice versa (Bullard plots)
#                             T_plotting_dict = {'number_lines':4, 
#                             'line0':{'line_type':'errorbar', 'zT_m_plot':zT_m, 'T_plot':T, 'zT_error_m_plot':zT_error_m, 'T_error_plot':T_error, 'fmt':'.k', 'alpha':0.125, 'markeredgecolor':None, 'figure_label':None},
#                             'line1':{'line_type':'errorbar', 'zT_m_plot':zT_m_subsampled, 'T_plot':T_subsampled, 'zT_error_m_plot':zT_error_m_subsampled, 'T_error_plot':T_error_subsampled, 'fmt':'.k', 'alpha':0.5, 'markeredgecolor':None, 'figure_label':None},
#                             'line2':{'line_type':'errorbar', 'zT_m_plot':zT_m_input, 'T_plot':T_input, 'zT_error_m_plot':zT_error_m_input, 'T_error_plot':T_error_input, 'fmt':'.r', 'alpha':1, 'markeredgecolor':None, 'figure_label':None},
#                             'line3':{'line_type':'errorbar', 'zT_m_plot':zT_m_input, 'T_plot':T_input_cc, 'zT_error_m_plot':zT_error_m_input, 'T_error_plot':T_error_input, 'fmt':'.b', 'alpha':1, 'markeredgecolor':None, 'figure_label':'a'}}
#                             k_plotting_dict = {'number_lines':2, 
#                             'line0':{'line_type':'strat_interp_lith_k', 'z0_m':layer_z0_m_subsampled, 'z1_m':layer_z1_m_subsampled, 'min_k':layer_min_k_subsampled, 'max_k':layer_max_k_subsampled, 'mean_k':layer_mean_k_subsampled, 'k_assigned_error':layer_mean_k_error_subsampled, 'color':'black', 'alpha':0.5, 'zorder':5, 'figure_label':None}, 
#                             'line1':{'line_type':'mc_input', 'layer_zk_m_input_plot':layer_zk_m_input_plot, 'layer_k_input_plot':layer_k_input_plot, 'color':'red', 'alpha':1, 'zorder':6, 'figure_label':'b'}}
#                             res_plotting_dict = {'number_lines':2, 
#                             'line0':{'line_type':'errorbar', 'zR_m_plot':zT_m_input, 'R_plot':R1_downward, 'zR_error_m_plot':zT_error_m_input, 'R_error_plot':sigma_R1_downward, 'fmt':'.r', 'alpha':1, 'markeredgecolor':None, 'figure_label':None},
#                             'line1':{'line_type':'errorbar', 'zR_m_plot':zT_m_input, 'R_plot':R1_upward, 'zR_error_m_plot':zT_error_m_input, 'R_error_plot':sigma_R1_upward, 'fmt':'.k', 'alpha':1, 'markeredgecolor':None, 'figure_label':'c'}}
#                             bullard_TvR_plotting_dict = {'number_lines':2, 
#                             'line0':{'line_type':'TvR_errorbar', 'T_plot':T_input, 'R_plot':R1_downward, 'T_error_plot':T_error_input, 'R_error_plot':sigma_R1_downward, 'fmt':'.r', 'alpha':1, 'markeredgecolor':None, 'figure_label':None},
#                             'line1':{'line_type':'TvR_errorbar', 'T_plot':T_input, 'R_plot':R1_upward, 'T_error_plot':T_error_input, 'R_error_plot':sigma_R1_upward, 'fmt':'.b', 'alpha':1, 'markeredgecolor':None, 'figure_label':'e'}}
#                             bullard_RvT_plotting_dict = {'number_lines':2, 
#                             'line0':{'line_type':'RvT_errorbar', 'T_plot':T_input, 'R_plot':R1_downward, 'T_error_plot':T_error_input, 'R_error_plot':sigma_R1_downward, 'fmt':'.r', 'alpha':1, 'markeredgecolor':None, 'figure_label':None},
#                             'line1':{'line_type':'RvT_errorbar', 'T_plot':T_input, 'R_plot':R1_upward, 'T_error_plot':T_error_input, 'R_error_plot':sigma_R1_upward, 'fmt':'.r', 'alpha':1, 'markeredgecolor':None, 'figure_label':'e'}}
#                             figure_name = figures_path + '_jsim' + str(jsim) + T_suffix_mc + '_cc' + pc_suffix_mc + rc_suffix_mc + k_suffix_mc + '_R_bull'
#                             plotting_functions.plot_temperature_stratigraphy_conductivity(max_depth_m_plot, k_distribution, plot_lith_fill_dict, plot_lith_fill_dict_keyword, strat_interp_lith_k_calcs_df, T_plotting_dict, k_plotting_dict, res_plotting_dict, bullard_TvR_plotting_dict, bullard_RvT_plotting_dict, T_range, figure_name)
                            
                            
#                              # Plot perturbed and climate-corrected temperature and perturbed conductivity and resistivity against depth. Plot thermal resistivity against temperature and vice versa (Bullard plots). Include fitted lines of heat flow.
#                             T_plotting_dict = {'number_lines':4, 
#                             'line0':{'line_type':'errorbar', 'zT_m_plot':zT_m, 'T_plot':T, 'zT_error_m_plot':zT_error_m, 'T_error_plot':T_error, 'fmt':'.k', 'alpha':0.125, 'markeredgecolor':None, 'figure_label':None},
#                             'line1':{'line_type':'errorbar', 'zT_m_plot':zT_m_subsampled, 'T_plot':T_subsampled, 'zT_error_m_plot':zT_error_m_subsampled, 'T_error_plot':T_error_subsampled, 'fmt':'.k', 'alpha':0.5, 'markeredgecolor':None, 'figure_label':None},
#                             'line2':{'line_type':'errorbar', 'zT_m_plot':zT_m_input, 'T_plot':T_input, 'zT_error_m_plot':zT_error_m_input, 'T_error_plot':T_error_input, 'fmt':'.r', 'alpha':1, 'markeredgecolor':None, 'figure_label':None},
#                             'line3':{'line_type':'errorbar', 'zT_m_plot':zT_m_input, 'T_plot':T_input_cc, 'zT_error_m_plot':zT_error_m_input, 'T_error_plot':T_error_input, 'fmt':'.b', 'alpha':1, 'markeredgecolor':None, 'figure_label':'a'}}
#                             k_plotting_dict = {'number_lines':2, 
#                             'line0':{'line_type':'strat_interp_lith_k', 'z0_m':layer_z0_m_subsampled, 'z1_m':layer_z1_m_subsampled, 'min_k':layer_min_k_subsampled, 'max_k':layer_max_k_subsampled, 'mean_k':layer_mean_k_subsampled, 'k_assigned_error':layer_mean_k_error_subsampled, 'color':'black', 'alpha':0.5, 'zorder':5, 'figure_label':None}, 
#                             'line1':{'line_type':'mc_input', 'layer_zk_m_input_plot':layer_zk_m_input_plot, 'layer_k_input_plot':layer_k_input_plot, 'color':'red', 'alpha':1, 'zorder':6, 'figure_label':'b'}}
#                             res_plotting_dict = {'number_lines':2, 
#                             'line0':{'line_type':'errorbar', 'zR_m_plot':zT_m_input, 'R_plot':R1_downward, 'zR_error_m_plot':zT_error_m_input, 'R_error_plot':sigma_R1_downward, 'fmt':'.r', 'alpha':1, 'markeredgecolor':None, 'figure_label':None},
#                             'line1':{'line_type':'errorbar', 'zR_m_plot':zT_m_input, 'R_plot':R1_upward, 'zR_error_m_plot':zT_error_m_input, 'R_error_plot':sigma_R1_upward, 'fmt':'.k', 'alpha':1, 'markeredgecolor':None, 'figure_label':'c'}}
#                             bullard_TvR_plotting_dict = {'number_lines':3, 
#                             'line0':{'line_type':'TvR_errorbar', 'T_plot':T_input, 'R_plot':R1_downward, 'T_error_plot':T_error_input, 'R_error_plot':sigma_R1_downward, 'fmt':'.r', 'alpha':1, 'markeredgecolor':None, 'figure_label':None},
#                             'line1':{'line_type':'TvR_errorbar', 'T_plot':T_input, 'R_plot':R1_upward, 'T_error_plot':T_error_input, 'R_error_plot':sigma_R1_upward, 'fmt':'.b', 'alpha':1, 'markeredgecolor':None, 'figure_label':None},
#                             'line2':{'line_type':'fitted_line', 'x_plot':R1_downward, 'gradient':q_downward_TvR, 'intercept':c_downward_TvR, 'color':'red', 'alpha':1, 'figure_label':'e'}}
#                             bullard_RvT_plotting_dict = {'number_lines':3, 
#                             'line0':{'line_type':'RvT_errorbar', 'T_plot':T_input, 'R_plot':R1_downward, 'T_error_plot':T_error_input, 'R_error_plot':sigma_R1_downward, 'fmt':'.r', 'alpha':1, 'markeredgecolor':None, 'figure_label':None},
#                             'line1':{'line_type':'RvT_errorbar', 'T_plot':T_input, 'R_plot':R1_upward, 'T_error_plot':T_error_input, 'R_error_plot':sigma_R1_upward, 'fmt':'.r', 'alpha':1, 'markeredgecolor':None, 'figure_label':None},
#                             'line2':{'line_type':'fitted_line', 'x_plot':T, 'gradient':invq_downward_RvT, 'intercept':invc_downward_RvT, 'color':'red', 'alpha':1, 'figure_label':'e'}}
#                             figure_name = figures_path + '_jsim' + str(jsim) + T_suffix_mc + '_cc' + pc_suffix_mc + rc_suffix_mc + k_suffix_mc + '_R_bull_fitted'
#                             plotting_functions.plot_temperature_stratigraphy_conductivity(max_depth_m_plot, k_distribution, plot_lith_fill_dict, plot_lith_fill_dict_keyword, strat_interp_lith_k_calcs_df, T_plotting_dict, k_plotting_dict, res_plotting_dict, bullard_TvR_plotting_dict, bullard_RvT_plotting_dict, T_range, figure_name)
                            
#                             ax1.plot(R1_downward, q_downward_TvR*R1_downward + c_downward_TvR)
#                             ax1.set_xlabel(r'$R_d$ / W$^{-1}$ K m$^2$')
#                             ax1.set_ylabel(r'$T$ / $^{\circ}$C')
#                             ax1.set_title('Q = ' + str(Q_round_downward_TvR) + r' $\pm$ ' + str(sigma_Q_round_downward_TvR) + r' W m$^{-2}$')
#                             ax2 = fig.add_subplot(222)
#                             ax2.errorbar(T_input, R1_downward, yerr=sigma_R1_downward, fmt='.k')
#                             ax2.plot(T_input, invq_downward_RvT*T_input + invc_downward_RvT)
                            
                            print(q_downward_TvR_cc, 'q_downward_TvR_cc')
                            print(sigma_q_downward_TvR_cc, 'sigma_q_downward_TvR_cc')
                            print(c_downward_TvR_cc, 'c_downward_TvR_cc')
                            print(sigma_c_downward_TvR_cc, 'sigma_c_downward_TvR_cc')
#                             print("\n")
#                             print(figure_name)
#                             print(strat_interp_name)
#                             print(strat_interp_lith_k_option)
#                             print(pc_suffix_mc)
#                             print(rc_suffix_mc)
#                             print(k_suffix_mc)
#                             print("\n")
                            
                            if 1==1: continue
                            

                            ### TODO Plot separate effects of recent climatic and palaeoclimatic corrections

                            # exit()
#
                            # plt.close()
                            # fig = plt.figure(figsize=(5,3.5))
                            # ax1 = fig.add_subplot(121)
                            # ax1.errorbar(T_cut, zT_m_cut, xerr=T_error_cut, yerr=zT_error_m_cut, fmt='.k', markeredgecolor='k', capsize=1)
                            # ax1.errorbar(T_input, zT_m_input, fmt='.r', markeredgecolor='r')
                            # ax1.invert_yaxis()
                            # ax2 = fig.add_subplot(122)
                            # if k_distribution == 'uniform':
                            #     ax2.fill_betweenx(layer_z_plot, layer_min_k_plot, layer_max_k_plot, where=layer_max_k_plot >= layer_min_k_plot, facecolor='red', alpha=0.25)
                            #     ax2.plot(layer_min_k_plot, layer_z_plot, color='red', linestyle='--', alpha=1, zorder=5)
                            #     ax2.plot(layer_max_k_plot, layer_z_plot, color='red', linestyle='--', alpha=1, zorder=5)
                            # if k_distribution == 'normal' or k_distribution == 'in_situ_normal':
                            #     ax2.fill_betweenx(layer_z_plot, layer_mean_k_plot - layer_stdev_k_plot, layer_mean_k_plot + layer_stdev_k_plot, where=layer_mean_k_plot+layer_stdev_k_plot >= layer_mean_k_plot-layer_stdev_k_plot, facecolor='red', alpha=0.25)
                            #     ax2.plot(layer_mean_k_plot, layer_z_plot, color='red', alpha=1, zorder=5)
                            #
                            #
                            # if k_distribution == 'normal' or k_distribution == 'in_situ_normal':
                            #     ax2.errorbar(layer_mean_k, layer_zmid_m, xerr=layer_mean_k_error, yerr=layer_zmid_error_m, fmt='.k', markeredgecolor='k', capsize=1)
                            # elif k_distribution == 'uniform':
                            #     xerr_arr = np.transpose(np.column_stack((layer_min_k, layer_max_k)))
                            #     print(xerr_arr)
                            #     print(np.shape(xerr_arr))
                            #     xmid_plot = (layer_min_k + layer_max_k)/2
                            #     xerr_arr = (layer_max_k - layer_min_k)/2
                            #     ax2.errorbar(xmid_plot, layer_zmid_m, xerr=xerr_arr, yerr=layer_zmid_error_m, fmt='none', ecolor='k', capsize=0)
                            # if monte_carlo_k_option == 'yes':
                            #     if  monte_carlo_in_situ_k_subsample_factor is not None:
                            #         if k_distribution == 'normal' or k_distribution == 'in_situ_normal':
                            #             ax2.errorbar(layer_mean_k_subsampled, layer_zmid_m_subsampled, xerr=layer_mean_k_error_subsampled, yerr=layer_zmid_error_m_subsampled, fmt='.g', markeredgecolor='g', capsize=1)
                            #
                            #         # ax2.errorbar(layer_k_input, layer_mean_k_subsampled, fmt='.b')
                            # #
                            # #
                            # #
                            # #
                            # #         layer_zmid_m_subsampled, layer_zmid_error_m_subsampled, layer_z0_m_subsampled, layer_z0_error_m_subsampled, layer_z1_m_subsampled, layer_z1_error_m_subsampled, layer_mean_k_subsampled, layer_mean_k_error_subsampled, layer_min_k_subsampled, layer_max_k_subsampled
                            #
                            #     ax2.errorbar(layer_k_input, layer_zmid_m_input, fmt='.b')
                            #
                            #     ax2.plot(layer_k_input_plot, layer_zk_m_input_plot, color='blue', alpha=0.5)
                            #
                            # ax2.invert_yaxis()
                            # fig.suptitle(str(strat_interp_name) + '_' + str(strat_interp_lith_k_option))
                            # plt.show()

                            # # Plot corrected temperatures
                            # if pcc_option == 'yes':
                            #     plt.close()
                            #     fig = plt.figure(figsize=(5,3.5))
                            #     ax1 = fig.add_subplot(111)
                            #     ax1.errorbar(T_input, zT_m_input, yerr=T_error_input, fmt='.k', markeredgecolor='k')
                            #     ax1.errorbar(T_input_pc, zT_m_input, yerr=T_error_input_pc, fmt='.r', markeredgecolor='r')
                            #     ax1.invert_yaxis()
                            #     plt.show()

                            # if rcc_option == 'yes':
                            #     plt.close()
                            #     fig = plt.figure(figsize=(5,3.5))
                            #     ax1 = fig.add_subplot(111)
                            #     ax1.errorbar(T_input, zT_m_input, yerr=T_error_input, fmt='.k', markeredgecolor='k')
                            #     ax1.errorbar(T_input_rc, zT_m_input, yerr=T_error_input_rc, fmt='.r', markeredgecolor='r')
                            #     ax1.invert_yaxis()
                            #     plt.show()


                            plt.close()
                            fig = plt.figure(figsize=(5,3.5))
                            ax1 = fig.add_subplot(221)
                            ax1.errorbar(R1_downward, T_input, yerr=T_error_input, fmt='.k')
                            ax1.plot(R1_downward, q_downward_TvR*R1_downward + c_downward_TvR)
                            ax1.set_xlabel(r'$R_d$ / W$^{-1}$ K m$^2$')
                            ax1.set_ylabel(r'$T$ / $^{\circ}$C')
                            ax1.set_title('Q = ' + str(Q_round_downward_TvR) + r' $\pm$ ' + str(sigma_Q_round_downward_TvR) + r' W m$^{-2}$')
                            ax2 = fig.add_subplot(222)
                            ax2.errorbar(T_input, R1_downward, yerr=sigma_R1_downward, fmt='.k')
                            ax2.plot(T_input, invq_downward_RvT*T_input + invc_downward_RvT)
                            ax2.set_ylabel(r'$R_d$ / W$^{-1}$ K m$^2$')
                            ax2.set_xlabel(r'$T$ / $^{\circ}$C')
                            ax2.set_title('Q = ' + str(Q_round_downward_RvT) + r' $\pm$ ' + str(sigma_Q_round_downward_RvT) + r' W m$^{-2}$')

                            # Check upward calculations
                            ax3 = fig.add_subplot(223)
                            ax3.errorbar(R1_upward, np.flip(T_input), yerr=np.flip(T_error_input), fmt='.k')
                            ax3.plot(R1_upward, -1*q_upward_TvR*R1_upward + c_upward_TvR)
                            ax3.set_xlabel(r'$R_u$ / W$^{-1}$ K m$^2$')
                            ax3.set_ylabel(r'$T$ / $^{\circ}$C')
                            ax3.set_title('Q = ' + str(Q_round_upward_TvR) + r' $\pm$ ' + str(sigma_Q_round_upward_TvR) + r' W m$^{-2}$')

                            ax4 = fig.add_subplot(224)
                            ax4.errorbar(np.flip(T_input), R1_upward, yerr=sigma_R1_upward, fmt='.k')
                            ax4.plot(T_input, invq_upward_RvT*T_input + invc_upward_RvT)
                            ax4.set_ylabel(r'$R_u$ / W$^{-1}$ K m$^2$')
                            ax4.set_xlabel(r'$T$ / $^{\circ}$C')
                            ax4.set_title('Q = ' + str(Q_round_upward_RvT) + r' $\pm$ ' + str(sigma_Q_round_upward_RvT) + r' W m$^{-2}$')

                            fig.suptitle(str(strat_interp_name) + '_' + str(strat_interp_lith_k_option))
                            plt.show()





                            plt.close()
                            fig = plt.figure(figsize=(5,3.5))

                            ax1 = fig.add_subplot(221, xlim=(0,150))

                            temp_ago = pc_deltaTs
                            temp_ago_ravel = np.column_stack([temp_ago, temp_ago]).ravel()
                            temp_ago_low = pc_deltaTs - pc_sigma_deltaTs
                            temp_ago_low_ravel = np.column_stack([temp_ago_low, temp_ago_low]).ravel()
                            temp_ago_high = pc_deltaTs + pc_sigma_deltaTs
                            temp_ago_high_ravel = np.column_stack([temp_ago_high, temp_ago_high]).ravel()

                            time0_ago = general_python_functions.s2y(pc_t0)
                            time1_ago = general_python_functions.s2y(pc_t1)
                            time1_ago_low = general_python_functions.s2y(pc_t1 - pc_sigma_t1)
                            time1_ago_high = general_python_functions.s2y(pc_t1 + pc_sigma_t1)

                            time_ago_ravel = general_python_functions.s2y(np.column_stack([pc_t0, pc_t1]).ravel())
                            time_ago_low_ravel = general_python_functions.s2y(np.column_stack([pc_t0 - pc_sigma_t0, pc_t1 - pc_sigma_t1]).ravel())
                            time_ago_high_ravel = general_python_functions.s2y(np.column_stack([pc_t0 + pc_sigma_t0, pc_t1 + pc_sigma_t1]).ravel())

                            ### TODO Make sure no white spaces in infill
                            ax1.fill_between(time0_ago/1e3, temp_ago_low, temp_ago_high, color='lightgray', step='post')
                            ax1.fill_between(time1_ago/1e3, temp_ago_low, temp_ago_high, color='lightgray', step='pre')

                            ax1.fill_betweenx(temp_ago, time1_ago_low/1e3, time1_ago_high/1e3, where=np.full(np.size(temp_ago), True), interpolate=True, color='lightgray', step='post')
                            ax1.fill_betweenx(temp_ago_low, time1_ago_low/1e3, time1_ago_high/1e3, where=np.full(np.size(temp_ago), True), interpolate=True, color='lightgray', step='post')
                            ax1.fill_betweenx(temp_ago_high, time1_ago_low/1e3, time1_ago_high/1e3, where=np.full(np.size(temp_ago), True), interpolate=True, color='lightgray', step='post')
                            # ax1.fill_betweenx(temp_ago, time1_ago/1e3, time1_ago_high/1e3, color='blue', step='post', alpha=0.5)



                            # ax1.fill_between(time_ago_ravel/1e3, temp_ago_low_ravel, temp_ago_high_ravel, color='gray', alpha=0.2)
#                             ax1.fill_between(time_ago_high/1e3, temp_ago_low, temp_ago_high, color='gray', alpha=1)
#                             ax1.fill_between(time_ago_low/1e3, temp_ago_low, temp_ago_high, color='gray', alpha=1)
                            # ax1.fill_betweenx(temp_ago, time_ago_low/1e3, time_ago_high/1e3, color='gray', alpha=1)
                            # ax1.fill_betweenx(temp_ago_high, time_ago_low/1e3, time_ago_high/1e3, color='gray', alpha=1)
                            ax1.plot(time_ago_ravel/1e3, temp_ago_ravel, 'k-', label=r'$\Delta T_c$')



                            temp_ago_mc = np.column_stack([pc_deltaTs_input, pc_deltaTs_input]).ravel()
                            time_ago_mc = np.column_stack([pc_t0_input, pc_t1_input]).ravel()
                            time_ago_mc = general_python_functions.s2y(time_ago_mc)
                            ax1.plot(time_ago_mc/1e3, temp_ago_mc, 'r-', label=r'$\Delta T_c$', alpha=0.5)
                            ax1.set_ylabel(r'$\Delta T_s$ / $^{\circ}$C')
                            ax1.set_xlabel('Time before present / ka')

                            ax2 = fig.add_subplot(222)
                            rc_deltaTs_ago = np.column_stack([rc_deltaTs, rc_deltaTs]).ravel()
                            rc_ty_ago = general_python_functions.s2y(np.column_stack([rc_t0, rc_t1]).ravel())
                            rc_deltaTs_ago_cut = np.column_stack([rc_deltaTs_cut, rc_deltaTs_cut]).ravel()
                            rc_ty_ago_cut = general_python_functions.s2y(np.column_stack([rc_t0_cut, rc_t1_cut]).ravel())
                            rc_deltaTs_ago_smoothed_cut = np.column_stack([rc_deltaTs_smoothed_cut, rc_deltaTs_smoothed_cut]).ravel()
                            rc_ty_ago_smoothing_cutoff_cut = general_python_functions.s2y(np.column_stack([rc_t0_smoothing_cutoff_cut, rc_t1_smoothing_cutoff_cut]).ravel())
                            rc_deltaTs_ago_input = np.column_stack([rc_deltaTs_input, rc_deltaTs_input]).ravel()
                            rc_ty_ago_input = general_python_functions.s2y(np.column_stack([rc_t0_input, rc_t1_input]).ravel())
                            ax2.set_ylabel(r'$\Delta T_s$ / $^{\circ}$C')
                            ax2.set_xlabel('Years before borehole drilled')
                            ax2.plot(rc_ty_ago, rc_deltaTs_ago, color='black', alpha=0.25)
                            ax2.plot(rc_ty_ago_cut, rc_deltaTs_ago_cut, color='black')
                            ax2.plot(rc_ty_ago_smoothing_cutoff_cut, rc_deltaTs_ago_smoothed_cut, color='red', alpha=0.25)
                            ax2.plot(rc_ty_ago_input, rc_deltaTs_ago_input, color='blue')
                            ax2year = ax2.twiny()
                            ax2year.set_xlabel('Year')
                            ax2year.invert_xaxis()
                            ax2year.plot(rc_ty, rc_deltaTs, alpha=0)

                            # ax3 = fig.add_subplot(223)
#                                                                 ax3.plot(palaeoclimatic_delta_T_zT_palaeoclimatic_background, z_m_palaeoclimatic_background, color='black')
#                                                                 ax3.plot(palaeoclimatic_delta_T_zT, zT_m_input, color='red')
#
#                                                                 ax4 = fig.add_subplot(224)
#                                                                 ax4.plot(recent_climatic_delta_T_zT_rc_background, zT_m_rc_background, color='black')
#                                                                 ax4.plot(recent_climatic_delta_T_zT, zT_m_input, color='red')
#                                                                 plt.show()

                            # exit()

                            if 1==1:
                                continue

                            ### PLOT RESULTS OF MONTE CARLO ANALYSIS ###
                            if monte_carlo_option is not None and jsim == monte_carlo_nsim-1:
                                print('estimating Monte Carlo averages\n')
                                # TODO Add weighted averages
                                whole_bullard_q_downward_TvR_mc_mean = np.nanmean(whole_bullard_q_downward_TvR_mc_all)
                                whole_bullard_q_downward_TvR_mc_stdev = np.nanstd(whole_bullard_q_downward_TvR_mc_all)
                                whole_bullard_c_downward_TvR_mc_median = np.nanmedian(whole_bullard_c_downward_TvR_mc_all)
                                whole_bullard_Q_downward_TvR_mc_mean_round, whole_bullard_Q_downward_TvR_mc_stdev_round = general_python_functions.round_to_sf(1e3*whole_bullard_q_downward_TvR_mc_mean, 1e3*whole_bullard_q_downward_TvR_mc_stdev)

                                whole_bullard_q_downward_RvT_mc_mean = np.nanmean(whole_bullard_q_downward_RvT_mc_all)
                                whole_bullard_q_downward_RvT_mc_stdev = np.nanstd(whole_bullard_q_downward_RvT_mc_all)
                                whole_bullard_invc_downward_RvT_mc_median = np.nanmedian(whole_bullard_invc_downward_RvT_mc_all)
                                whole_bullard_Q_downward_RvT_mc_mean_round, whole_bullard_Q_downward_RvT_mc_stdev_round = general_python_functions.round_to_sf(1e3*whole_bullard_q_downward_RvT_mc_mean, 1e3*whole_bullard_q_downward_RvT_mc_stdev)

                                whole_bullard_q_upward_TvR_mc_mean = np.nanmean(whole_bullard_q_upward_TvR_mc_all)
                                whole_bullard_q_upward_TvR_mc_stdev = np.nanstd(whole_bullard_q_upward_TvR_mc_all)
                                whole_bullard_c_upward_TvR_mc_median = np.nanmedian(whole_bullard_c_upward_TvR_mc_all)
                                whole_bullard_Q_upward_TvR_mc_mean_round, whole_bullard_Q_upward_TvR_mc_stdev_round = general_python_functions.round_to_sf(1e3*whole_bullard_q_upward_TvR_mc_mean, 1e3*whole_bullard_q_upward_TvR_mc_stdev)

                                whole_bullard_q_upward_RvT_mc_mean = np.nanmean(whole_bullard_q_upward_RvT_mc_all)
                                whole_bullard_q_upward_RvT_mc_stdev = np.nanstd(whole_bullard_q_upward_RvT_mc_all)
                                whole_bullard_invc_upward_RvT_mc_median = np.nanmedian(whole_bullard_invc_upward_RvT_mc_all)
                                whole_bullard_Q_upward_RvT_mc_mean_round, whole_bullard_Q_upward_RvT_mc_stdev_round = general_python_functions.round_to_sf(1e3*whole_bullard_q_upward_RvT_mc_mean, 1e3*whole_bullard_q_upward_RvT_mc_stdev)

                                print('plotting Monte Carlo\n')

                                # zT_m_input_mc_all[jsim, :] = zT_m_input
                                # T_input_mc_all[jsim, :] = T_input
                                # whole_bullard_R1_downward_mc_all[jsim, :] = R1_downward
                                # whole_bullard_sigma_R1_downward_mc_all[jsim, :]  = sigma_R1_downward



                                # Plot a heat map of perturbed temperature versus perturbed depth
                                heatmap, xedges, yedges = np.histogram2d(T_input_mc_all[~np.isnan(T_input_mc_all)].ravel(), zT_m_input_mc_all[~np.isnan(T_input_mc_all)].ravel(), bins=[1000,1000], density=True)
                                heatmap = heatmap.T
                                X, Y = np.meshgrid(xedges, yedges)
                                fig = plt.figure(figsize=(4,4))
                                ax1 = fig.add_subplot(111, xlabel=r'$T$ / $^{\circ}$C', ylabel='Depth / m')
                                ax1.pcolormesh(X, Y, heatmap, cmap='Reds')
                                ax1.invert_yaxis()
                                plt.show()

                                # Plot a heatmap of pertubed thermal conductivity versus perturbed conductivity depth
                                # Sample every 1 m to make smooth distribution of conductivity profile
                                heatmap, xedges, yedges = np.histogram2d(layer_k_input_plot_mc_all[~np.isnan(layer_k_input_plot_mc_all)].ravel(), layer_zk_m_input_plot_mc_all[~np.isnan(layer_zk_m_input_plot_mc_all)].ravel(), bins=[200,100], density=True)
                                heatmap = heatmap.T
                                X, Y = np.meshgrid(xedges, yedges)
                                fig = plt.figure(figsize=(4,4))
                                ax1 = fig.add_subplot(111, xlabel=r'$k$', ylabel='Depth / m')
                                ax1.pcolormesh(X, Y, heatmap, cmap='Reds')
                                ax1.invert_yaxis()
                                plt.show()

                                ### Plot thermal resistivities against depth
                                fig = plt.figure(figsize=(4,4))
                                # Plot a heatmap of pertubed downward thermal resistivity versus perturbed temperature depth
                                heatmap, xedges, yedges = np.histogram2d(whole_bullard_R1_downward_mc_all[~np.isnan(T_input_mc_all)].ravel(), zT_m_input_mc_all[~np.isnan(T_input_mc_all)].ravel(), bins=[200,100], density=True)
                                heatmap = heatmap.T
                                X, Y = np.meshgrid(xedges, yedges)
                                ax1 = fig.add_subplot(121, xlabel=r'$R_d$ / W$^{-1}$ K m$^2$', ylabel=r'$z_T$ / m')
                                ax1.pcolormesh(X, Y, heatmap, cmap='Reds')
                                ax1.invert_yaxis()
                                # Plot a heatmap of pertubed upward thermal resistivity versus perturbed temperature depth
                                heatmap, xedges, yedges = np.histogram2d(whole_bullard_R1_upward_mc_all[~np.isnan(T_input_mc_all)].ravel(), zT_m_input_mc_all[~np.isnan(T_input_mc_all)].ravel(), bins=[200,100], density=True)
                                heatmap = heatmap.T
                                X, Y = np.meshgrid(xedges, yedges)
                                ax1 = fig.add_subplot(122, xlabel=r'$R_u$ / W$^{-1}$ K m$^2$', ylabel=r'$z_T$ / m')
                                ax1.pcolormesh(X, Y, heatmap, cmap='Reds')
                                ax1.invert_yaxis()
                                plt.show()


                                ### Plot heat maps of perturbed temperatures and thermal resistivities
                                fig = plt.figure(figsize=(4,4))
                                # Plot a heatmap of perturbed temperature against perturbed downward thermal resistivity
                                heatmap, xedges, yedges = np.histogram2d(whole_bullard_R1_downward_mc_all[~np.isnan(T_input_mc_all)].ravel(), T_input_mc_all[~np.isnan(T_input_mc_all)].ravel(), bins=[100,100], density=True)
                                heatmap = heatmap.T
                                X, Y = np.meshgrid(xedges, yedges)
                                ax1 = fig.add_subplot(221, xlabel=r'$R_d$ / W$^{-1}$ K m$^2$', ylabel=r'$T$ / $^{\circ}$C')
                                ax1.pcolormesh(X, Y, heatmap, cmap='Reds')
                                minmax_R_plot = np.array([np.min(whole_bullard_R1_downward_mc_all), np.max(whole_bullard_R1_downward_mc_all)])
                                ax1.plot(minmax_R_plot, whole_bullard_q_downward_TvR_mc_mean * minmax_R_plot + whole_bullard_c_downward_TvR_mc_median, zorder=4, color='white')
                                textstr1 = r'$q = $' + str(whole_bullard_Q_downward_TvR_mc_mean_round) + r'$ \pm $' + str(whole_bullard_Q_downward_TvR_mc_stdev_round) + r' mW m$^{-2}$'
                                fig.text(0.1, 0.9, textstr1, color="green")


                                # whole_bullard_q_downward_TvR_mc_mean = np.nanmean(whole_bullard_q_downward_TvR_mc_all)
                                # whole_bullard_c_downward_TvR_mc_all_median = np.nanmedian(whole_bullard_c_downward_TvR_mc_all)
                                # whole_bullard_q_downward_RvT_mc_all_mean = np.nanmean(whole_bullard_q_downward_RvT_mc_all)
                                # whole_bullard_invc_downward_RvT_mc_all_median = np.nanmedian(whole_bullard_invc_downward_RvT_mc_all)

                                # Plot a heatmap of perturbed downward thermal resistivity against perturbed temperature
                                heatmap, xedges, yedges = np.histogram2d(T_input_mc_all[~np.isnan(T_input_mc_all)].ravel(), whole_bullard_R1_downward_mc_all[~np.isnan(T_input_mc_all)].ravel(), bins=[100,100], density=True)
                                heatmap = heatmap.T
                                X, Y = np.meshgrid(xedges, yedges)
                                ax1 = fig.add_subplot(222, ylabel=r'$R_d$ / W$^{-1}$ K m$^2$', xlabel=r'$T$ / $^{\circ}$C')
                                ax1.pcolormesh(X, Y, heatmap, cmap='Reds')
                                minmax_T_plot = np.array([np.min(T_input_mc_all), np.max(T_input_mc_all)])
                                ax1.plot(minmax_T_plot, 1/whole_bullard_q_downward_RvT_mc_mean * minmax_T_plot + whole_bullard_invc_downward_RvT_mc_median, zorder=4, color='white')
                                textstr1 = r'$q = $' + str(whole_bullard_Q_downward_RvT_mc_mean_round) + r'$ \pm $' + str(whole_bullard_Q_downward_RvT_mc_stdev_round) + r' mW m$^{-2}$'
                                fig.text(0.6, 0.9, textstr1, color="green")



                                # Plot a heatmap of perturbed temperature against perturbed upward thermal resistivity
                                heatmap, xedges, yedges = np.histogram2d(whole_bullard_R1_upward_mc_all[~np.isnan(T_input_mc_all)].ravel(), T_input_mc_all[~np.isnan(T_input_mc_all)].ravel(), bins=[100,100], density=True)
                                heatmap = heatmap.T
                                X, Y = np.meshgrid(xedges, yedges)
                                ax1 = fig.add_subplot(223, xlabel=r'$R_u$ / W$^{-1}$ K m$^2$', ylabel=r'$T$ / $^{\circ}$C')
                                ax1.pcolormesh(X, Y, heatmap, cmap='Reds')
                                # Plot a heatmap of perturbed upward thermal resistivity against perturbed temperature
                                heatmap, xedges, yedges = np.histogram2d(T_input_mc_all[~np.isnan(T_input_mc_all)].ravel(), whole_bullard_R1_upward_mc_all[~np.isnan(T_input_mc_all)].ravel(), bins=[100,100], density=True)
                                heatmap = heatmap.T
                                X, Y = np.meshgrid(xedges, yedges)
                                ax1 = fig.add_subplot(224, ylabel=r'$R_u$ / W$^{-1}$ K m$^2$', xlabel=r'$T$ / $^{\circ}$C')
                                ax1.pcolormesh(X, Y, heatmap, cmap='Reds')
                                plt.show()



                                # Plot distributions of heat-flow estimates
                                probdens_whole_bullard_q_downward_TvR_mc_all, edges_whole_bullard_q_downward_TvR_mc_all = np.histogram(whole_bullard_q_downward_TvR_mc_all * 1e3, bins=50, density=True)
                                # probdens_random_cc, edges_random_cc = np.histogram(qc_all * 1e3, bins=100, density=True)

                                fig = plt.figure(figsize=(4,4))
                                ax1 = fig.add_subplot(111, xlabel="$q_s \ / \ \mathrm{mW \, m^{-2}}$", ylabel='Probability Density')
                                ax1.bar(edges_whole_bullard_q_downward_TvR_mc_all[:-1], probdens_whole_bullard_q_downward_TvR_mc_all, width=np.diff(edges_whole_bullard_q_downward_TvR_mc_all), align="edge")
                                # ax1.bar(edges_random_cc[:-1], probdens_random_cc, width=np.diff(edges_random_cc), align="edge")
                                # fig.savefig(outfile_dir+"/"+borename+'_bullard_heat-flow_histogram.jpg', bbox_inches='tight', transparent=True, dpi=300)
    #                                                                 plt.close(fig)
                                plt.show()



silloth_no2 Silloth No. 2 1982
['in_situ_conds', 'ukogl_well_tops', 'bgs_borehole_scan']

 in_situ_conds
in_situ_conds
dropping conductivity na
('cst_borehole_conductivity', 'yes', 'yes', 2, 2, 100)
estimating Monte Carlo averages



AttributeError: 'int' object has no attribute 'ndim'